In [ ]:
"""Using 'all' file instead of 'wmo' file"""
import numpy as np 
import pandas as pd
from netCDF4 import Dataset,num2date
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import norm, poisson, lognorm, chisquare, linregress, ttest_ind, power_divergence, ks_2samp, chi2_contingency
from scipy.interpolate import interp1d
from scipy.special import factorial
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
#%matplotlib tk    #Uncomment for interactive figures
import geopy.distance as gd
from sklearn.metrics import mean_squared_error
import time
import os

In [ ]:
'''Read .nc file, credits: Theo Rashid'''
tic = time.time() #Expect runtime ~5 min

nc_ibtracs = 'Allstorms.ibtracs_all.v03r10.nc' #Using 'all' file
ibt = Dataset(nc_ibtracs)

name          = ibt.variables['name'][:] #Name of TC
storm_sn      = ibt.variables['storm_sn'][:] 
time_record   = ibt.variables['source_time'][:] #Time of record
landfall      = ibt.variables['landfall'][:]

genesis_basin = ibt.variables['genesis_basin'][:]
season        = ibt.variables['season'][:]

lat           = ibt.variables['source_lat'][:] #Latitude of TC centre
lon           = ibt.variables['source_lon'][:] #Longitude of TC centre
max_wind      = ibt.variables['source_wind'][:] #Max. wind speed
min_pres      = ibt.variables['source_pres'][:] #Central pressure
dist2land     = ibt.variables['dist2land'][:] #Distance to land
    

nc_landmask = 'ETOPO1_Ice_g_gmt4.nc' #Land mask from ETOPO1 Global Relief Model
land = Dataset(nc_landmask)

landmask_lon = land.variables['lon'][:] #Longitude points of land mask
landmask_lat = land.variables['lat'][:] #Latitude points
z = land.variables['z'][:] #Altitude
landmask_LAT, landmask_LON = np.meshgrid(landmask_lat, landmask_lon) #Create meshgrid for getting and plotting each point

source_list = [10,14,19,21] #These are data from ibt, 10 is jtwc_wp, #14 is cma, #19 is wmo tokyo, #21 is hko
LAT         = np.nan * np.ones(shape=(np.size(lat,0),np.size(lat,1),len(source_list)))
LON         = np.nan * np.ones(shape=(np.size(lat,0),np.size(lat,1),len(source_list)))
MAX_WIND    = np.nan * np.ones(shape=(np.size(lat,0),np.size(lat,1),len(source_list)))
MIN_PRES    = np.nan * np.ones(shape=(np.size(lat,0),np.size(lat,1),len(source_list)))
DIST2LAND   = np.nan * np.ones(shape=(np.size(lat,0),np.size(lat,1),len(source_list)))
TIME_RECORD = np.nan * np.ones(shape=(np.size(lat,0),np.size(lat,1),len(source_list)))
LANDFALL    = np.nan * np.ones(shape=(np.size(lat,0),np.size(lat,1),len(source_list)))

for INDEX_RECORD_SOURCE in source_list: 
    LAT      [:,:,source_list.index(INDEX_RECORD_SOURCE)] = lat[:,:,INDEX_RECORD_SOURCE] 
    LON      [:,:,source_list.index(INDEX_RECORD_SOURCE)] = lon[:,:,INDEX_RECORD_SOURCE]
    MAX_WIND [:,:,source_list.index(INDEX_RECORD_SOURCE)] = max_wind[:,:,INDEX_RECORD_SOURCE]
    MIN_PRES [:,:,source_list.index(INDEX_RECORD_SOURCE)] = min_pres[:,:,INDEX_RECORD_SOURCE]
    
    DIST2LAND   = dist2land.data[:]
    TIME_RECORD = time_record.data[:]
    LANDFALL    = landfall.data[:].astype(float)

TCinseason = [] #Index of TCs occuring on or after 2014, needed for rainfall
for i in range(len(list(season))):
    if list(season)[i] >= 2014:
        TCinseason.append(i)

toc = time.time()
print(toc - tic)

In [ ]:
#Extract PHI data
tic = time.time() #Expect runtime ~3min

#Input extreme point = [x,y]
extremeP_N = [122.9, 19.2]
extremeP_S = [122.6, 3.5]
extremeP_E = [128.2, 6.2]
extremeP_W = [117.4, 18.1]

#Slope of lines bounding region
m_top_P = (extremeP_N[1]-extremeP_W[1])/(extremeP_N[0]-extremeP_W[0])
m_bot_P = (extremeP_S[1]-extremeP_E[1])/(extremeP_S[0]-extremeP_E[0])
m_left_P = (extremeP_S[1]-extremeP_W[1])/(extremeP_S[0]-extremeP_W[0])
m_right_P = (extremeP_N[1]-extremeP_E[1])/(extremeP_N[0]-extremeP_E[0])

#Intercept of lines
c_top_P = extremeP_N[1]-m_top_P*extremeP_N[0]
c_bot_P = extremeP_S[1]-m_bot_P*extremeP_S[0]
c_left_P = extremeP_S[1]-m_left_P*extremeP_S[0]
c_right_P = extremeP_N[1]-m_right_P*extremeP_N[0]

#Get points in land mask file which are in PHI area
def philippine_landmask_points(landmask_LON,landmask_LAT):
    landmask_points = []
    
    for i in range(17400,18500): 
        for i2 in range(5600,6600):
            equation_top = m_top_P * landmask_LON[i,i2] + c_top_P
            equation_bot = m_bot_P * landmask_LON[i,i2] + c_bot_P 
            equation_left = m_left_P * landmask_LON[i,i2] + c_left_P 
            equation_right  = m_right_P * landmask_LON[i,i2] + c_right_P
    
            if landmask_LAT[i,i2] <= equation_top and landmask_LAT[i,i2] <= equation_right and landmask_LAT[i,i2] >= equation_bot and landmask_LAT[i,i2] >= equation_left:
                landmask_points.append(i)
                landmask_points.append(i2)
        
    return landmask_points
    
#Get TC centre points in PHI area
def philippine_data_extraction(LON,LAT,after2014=False,separate_NS=False): 
    #after2014: True means only include TC data after 2014
    #Separate NS: If you want to separate PHI into Northern and Southern parts, type 'N' or 'S' respectively,
    #    otherwise, or for GD, type 'False'
    
    philippine_data = []
    PHI_TCposition = {}
    
    philippine_data_N = []
    philippine_data_S = []
    PHI_TCposition_N = {}
    PHI_TCposition_S = {}
    
    if separate_NS == False:
        for i in range (len(LON)):
            PHI_TCposition[str(i)] = []
            for i2 in range (len(LON[i])): 
                equation_top = m_top_P * LON[i,i2] + c_top_P
                equation_bot = m_bot_P * LON[i,i2] + c_bot_P 
                equation_left = m_left_P * LON[i,i2] + c_left_P 
                equation_right  = m_right_P * LON[i,i2] + c_right_P

                if after2014 == True:
                    if i in TCinseason and LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                        philippine_data.append((i))
                        PHI_TCposition[str(i)].append(i2)
                else:
                    if LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                        philippine_data.append((i))
                        PHI_TCposition[str(i)].append(i2)
                        
        return np.unique(philippine_data), PHI_TCposition
                    
    elif separate_NS == True:
        NSlatboundary = 14.4 #Set latitude for dividing into North and South
        for i in range(len(LON)):
            PHI_TCposition_N[str(i)] = []
            PHI_TCposition_S[str(i)] = []
            for i2 in range (len(LON[i])): 
                equation_top = m_top_P * LON[i,i2] + c_top_P #Equation of straight line
                equation_bot = m_bot_P * LON[i,i2] + c_bot_P 
                equation_left = m_left_P * LON[i,i2] + c_left_P 
                equation_right  = m_right_P * LON[i,i2] + c_right_P
                
                #Check to see if a point is inside the boundary
                if after2014 == True:
                    if i in TCinseason and LAT[i,i2] > NSlatboundary and LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                        philippine_data_N.append((i))
                        PHI_TCposition_N[str(i)].append(i2)
                    elif i in TCinseason and LAT[i,i2] <= NSlatboundary and LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                        philippine_data_S.append((i))
                        PHI_TCposition_S[str(i)].append(i2)

                else:
                    if LAT[i,i2] > NSlatboundary and LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                        philippine_data_N.append((i))
                        PHI_TCposition_N[str(i)].append(i2)
                    elif LAT[i,i2] <= NSlatboundary and LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                        philippine_data_S.append((i))
                        PHI_TCposition_S[str(i)].append(i2)
                    
        return np.unique(philippine_data_N), np.unique(philippine_data_S), PHI_TCposition_N, PHI_TCposition_S
                

landmask_in_area_PHI = np.array(philippine_landmask_points(landmask_LON,landmask_LAT))
landmask_in_area_PHI = landmask_in_area_PHI.reshape(int(len(landmask_in_area_PHI)/2),2)

z_coastline_PHI = [] #Find PHI coastline from land mask (this list contains index of elements)
z_land_PHI = [] #Find PHI land points from land mask
for i in range(len(landmask_in_area_PHI)):
    if z[landmask_in_area_PHI[i,1],landmask_in_area_PHI[i,0]] == 0: #Coast if altitude == 0
        z_coastline_PHI.append(landmask_in_area_PHI[i,0])
        z_coastline_PHI.append(landmask_in_area_PHI[i,1])
    elif z[landmask_in_area_PHI[i,1],landmask_in_area_PHI[i,0]] >= 0: #Land if altitude >= 0
        z_land_PHI.append(landmask_in_area_PHI[i,0])
        z_land_PHI.append(landmask_in_area_PHI[i,1])
        
z_coastline_PHI = np.asarray(z_coastline_PHI).reshape(int(len(z_coastline_PHI)/2),2)
z_land_PHI = np.asarray(z_land_PHI).reshape(int(len(z_land_PHI)/2),2)

z_coastline_coord_PHI = np.zeros(z_coastline_PHI.shape) #list of coordinates of PHI coastline (change index to actual coordinates)
z_coastline_coord_PHI[:,0] = landmask_lon[z_coastline_PHI[:,0]]
z_coastline_coord_PHI[:,1] = landmask_lat[z_coastline_PHI[:,1]]

z_land_coord_PHI = np.zeros(z_land_PHI.shape) #List of coordinates of PHI land
z_land_coord_PHI[:,0] = landmask_lon[z_land_PHI[:,0]]
z_land_coord_PHI[:,1] = landmask_lat[z_land_PHI[:,1]]

landfallTC_PHI_jtwc = philippine_data_extraction(LON[:,:,0],LAT[:,:,0])[0] #list (keys)
landfallTC_PHI_jtwc_after2014 = philippine_data_extraction(LON[:,:,0],LAT[:,:,0], True)[0]
landfallTC_PHI_cma = philippine_data_extraction(LON[:,:,1],LAT[:,:,1])[0] #list (keys)
landfallTC_PHI_cma_after2014 = philippine_data_extraction(LON[:,:,1],LAT[:,:,1], True)[0]
landfallTC_PHI_wmo = philippine_data_extraction(LON[:,:,2],LAT[:,:,2])[0] #list (keys)
landfallTC_PHI_wmo_after2014 = philippine_data_extraction(LON[:,:,2],LAT[:,:,2], True)[0]

PHI_TCposition_jtwc = philippine_data_extraction(LON[:,:,0],LAT[:,:,0])[1] #dict (tc index and position index)
PHI_TCposition_jtwc_after2014 = philippine_data_extraction(LON[:,:,0],LAT[:,:,0],True)[1]
PHI_TCposition_cma = philippine_data_extraction(LON[:,:,1],LAT[:,:,1])[1] #dict (tc index and position index)
PHI_TCposition_cma_after2014 = philippine_data_extraction(LON[:,:,1],LAT[:,:,1],True)[1]
PHI_TCposition_wmo = philippine_data_extraction(LON[:,:,2],LAT[:,:,2])[1] #dict (tc index and position index)
PHI_TCposition_wmo_after2014 = philippine_data_extraction(LON[:,:,2],LAT[:,:,2],True)[1]

landfallTC_PHI_jtwc_N, landfallTC_PHI_jtwc_S, PHI_TCposition_jtwc_N, PHI_TCposition_jtwc_S = philippine_data_extraction(LON[:,:,0],LAT[:,:,0],False, True)
landfallTC_PHI_jtwc_N_after2014, landfallTC_PHI_jtwc_S_after2014, PHI_TCposition_jtwc_N_after2014, PHI_TCposition_jtwc_S_after2014 = philippine_data_extraction(LON[:,:,0],LAT[:,:,0],True, True)
landfallTC_PHI_cma_N, landfallTC_PHI_cma_S, PHI_TCposition_cma_N, PHI_TCposition_cma_S = philippine_data_extraction(LON[:,:,1],LAT[:,:,1],False, True)
landfallTC_PHI_cma_N_after2014, landfallTC_PHI_cma_S_after2014, PHI_TCposition_cma_N_after2014, PHI_TCposition_cma_S_after2014 = philippine_data_extraction(LON[:,:,1],LAT[:,:,1],True, True)
landfallTC_PHI_wmo_N, landfallTC_PHI_wmo_S, PHI_TCposition_wmo_N, PHI_TCposition_wmo_S = philippine_data_extraction(LON[:,:,2],LAT[:,:,2],False, True)
landfallTC_PHI_wmo_N_after2014, landfallTC_PHI_wmo_S_after2014, PHI_TCposition_wmo_N_after2014, PHI_TCposition_wmo_S_after2014 = philippine_data_extraction(LON[:,:,2],LAT[:,:,2],True, True)

toc = time.time()
print(toc - tic)

In [ ]:
#Extract GD data

tic = time.time() #Expect runtime ~2min

extremeG_N = [117.4, 24.2]
extremeG_S = [109.8, 19.9]
extremeG_E = [117.7, 22.9]
extremeG_W = [109.4, 22.2]

m_top_G = (extremeG_N[1]-extremeG_W[1])/(extremeG_N[0]-extremeG_W[0])
m_bot_G = (extremeG_S[1]-extremeG_E[1])/(extremeG_S[0]-extremeG_E[0])
m_left_G = (extremeG_S[1]-extremeG_W[1])/(extremeG_S[0]-extremeG_W[0])
m_right_G = (extremeG_N[1]-extremeG_E[1])/(extremeG_N[0]-extremeG_E[0])

c_top_G = extremeG_N[1]-m_top_G*extremeG_N[0]
c_bot_G = extremeG_S[1]-m_bot_G*extremeG_S[0]
c_left_G = extremeG_S[1]-m_left_G*extremeG_S[0]
c_right_G = extremeG_N[1]-m_right_G*extremeG_N[0]

def guangdong_landmask_points(landmask_LON,landmask_LAT):

    landmask_points = []

    for i in range(17200,17900): 
        for i2 in range(6500,7000):
            equation_top = m_top_G * landmask_LON[i,i2] + c_top_G
            equation_bot = m_bot_G * landmask_LON[i,i2] + c_bot_G 
            equation_left = m_left_G * landmask_LON[i,i2] + c_left_G
            equation_right  = m_right_G * landmask_LON[i,i2] + c_right_G
    
            if landmask_LAT[i,i2] <= equation_top and landmask_LAT[i,i2] <= equation_right and landmask_LAT[i,i2] >= equation_bot and landmask_LAT[i,i2] >= equation_left:
                landmask_points.append(i)
                landmask_points.append(i2)
        
    return landmask_points

def guangdong_data_extraction(LON,LAT,after2014=False): 
    guangdong_data = []
    GD_TCposition = {}
    
    for i in range (len(LON)):
        GD_TCposition[str(i)] = []
        for i2 in range (len(LON[i])):
            equation_top = m_top_G * LON[i,i2] + c_top_G
            equation_bot = m_bot_G * LON[i,i2] + c_bot_G 
            equation_left = m_left_G * LON[i,i2] + c_left_G 
            equation_right  = m_right_G * LON[i,i2] + c_right_G
            
            if after2014 == True:
                if i in TCinseason and LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                    guangdong_data.append((i))
                    GD_TCposition[str(i)].append(i2)
            else:
                if LAT[i,i2] <= equation_top and LAT[i,i2] <= equation_right and LAT[i,i2] >= equation_bot and LAT[i,i2] >= equation_left: 
                    guangdong_data.append((i))
                    GD_TCposition[str(i)].append(i2)
            
    return np.unique(guangdong_data), GD_TCposition

landmask_in_area_GD = np.asarray(guangdong_landmask_points(landmask_LON,landmask_LAT))
landmask_in_area_GD = landmask_in_area_GD.reshape(int(len(landmask_in_area_GD)/2),2)


z_coastline_GD = []
z_land_GD = []
for i in range(len(landmask_in_area_GD)):
    if z[landmask_in_area_GD[i,1],landmask_in_area_GD[i,0]] == 0: 
        z_coastline_GD.append(landmask_in_area_GD[i,0])
        z_coastline_GD.append(landmask_in_area_GD[i,1])
    elif z[landmask_in_area_GD[i,1],landmask_in_area_GD[i,0]] >= 0:
        z_land_GD.append(landmask_in_area_GD[i,0])
        z_land_GD.append(landmask_in_area_GD[i,1])
        
z_coastline_GD = np.asarray(z_coastline_GD).reshape(int(len(z_coastline_GD)/2),2)
z_land_GD = np.asarray(z_land_GD).reshape(int(len(z_land_GD)/2),2)

z_coastline_coord_GD = np.zeros(z_coastline_GD.shape) #list of coordinates of GD coastline
z_coastline_coord_GD[:,0] = landmask_lon[z_coastline_GD[:,0]]
z_coastline_coord_GD[:,1] = landmask_lat[z_coastline_GD[:,1]]

z_land_coord_GD = np.zeros(z_land_GD.shape)
z_land_coord_GD[:,0] = landmask_lon[z_land_GD[:,0]]
z_land_coord_GD[:,1] = landmask_lat[z_land_GD[:,1]]
        
landfallTC_GD_jtwc = guangdong_data_extraction(LON[:,:,0],LAT[:,:,0])[0] #list (index)
landfallTC_GD_jtwc_after2014 = guangdong_data_extraction(LON[:,:,0],LAT[:,:,0],True)[0]
landfallTC_GD_cma = guangdong_data_extraction(LON[:,:,1],LAT[:,:,1])[0] #list (index)
landfallTC_GD_cma_after2014 = guangdong_data_extraction(LON[:,:,1],LAT[:,:,1],True)[0]
landfallTC_GD_wmo = guangdong_data_extraction(LON[:,:,2],LAT[:,:,2])[0] #list (index)
landfallTC_GD_wmo_after2014 = guangdong_data_extraction(LON[:,:,2],LAT[:,:,2],True)[0]

GD_TCposition_jtwc = guangdong_data_extraction(LON[:,:,0],LAT[:,:,0])[1] #dict (tc index & position index)
GD_TCposition_jtwc_after2014 = guangdong_data_extraction(LON[:,:,0],LAT[:,:,0],True)[1]
GD_TCposition_cma = guangdong_data_extraction(LON[:,:,1],LAT[:,:,1])[1] #dict (tc index & position index)
GD_TCposition_cma_after2014 = guangdong_data_extraction(LON[:,:,1],LAT[:,:,1],True)[1]
GD_TCposition_wmo = guangdong_data_extraction(LON[:,:,2],LAT[:,:,2])[1] #dict (tc index & position index)
GD_TCposition_wmo_after2014 = guangdong_data_extraction(LON[:,:,2],LAT[:,:,2],True)[1]

toc = time.time()
print(toc - tic)

In [ ]:
#----------------------------------------------------
'''
Here, we are only looking at the landfall typhoon point and the point before that
'''

tic = time.time() #Expected runtime ~45s

#Gets the data for TCs at first landfall point, and closest point before landfall
def get_landfall_data_beforeandafter(area, source, after2014 = False, PHI_NS = False):
    #area: if PHI, type '0'; if GD, type '1'
    #source: if JTWC, type '0'; if CMA, type '1'; if WMO, type '2'
    #after2014: True means only include TC data after 2014
    #Separate NS: If you want to separate PHI into Northern and Southern parts, type 'N' or 'S' respectively,
    #    otherwise, or for GD, type 'False'
    
    pres = {}
    wind = {}
    coord = {}
    disttoland = {}
    list_of_index = {}
    epsilon = 1e-5 #To account for rounding errors when comparing numbers
    
    if area == 0: #PHI
        if source == 0: #jtwc
            if PHI_NS == False:
                if after2014 == True:
                    tcindex = landfallTC_PHI_jtwc_after2014
                    posindex = PHI_TCposition_jtwc_after2014
                else:
                    tcindex = landfallTC_PHI_jtwc
                    posindex = PHI_TCposition_jtwc
            elif PHI_NS == 'N':
                if after2014 == True:
                    tcindex = landfallTC_PHI_jtwc_N_after2014
                    posindex = PHI_TCposition_jtwc_N_after2014
                else:
                    tcindex = landfallTC_PHI_jtwc_N
                    posindex = PHI_TCposition_jtwc_N
            elif PHI_NS == 'S':
                if after2014 == True:
                    tcindex = landfallTC_PHI_jtwc_S_after2014
                    posindex = PHI_TCposition_jtwc_S_after2014
                else:
                    tcindex = landfallTC_PHI_jtwc_S
                    posindex = PHI_TCposition_jtwc_S
        elif source == 1: #cma
            if PHI_NS == False:
                if after2014 == True:
                    tcindex = landfallTC_PHI_cma_after2014
                    posindex = PHI_TCposition_cma_after2014
                else:
                    tcindex = landfallTC_PHI_cma
                    posindex = PHI_TCposition_cma
            elif PHI_NS == 'N':
                if after2014 == True:
                    tcindex = landfallTC_PHI_cma_N_after2014
                    posindex = PHI_TCposition_cma_N_after2014
                else:
                    tcindex = landfallTC_PHI_cma_N
                    posindex = PHI_TCposition_cma_N
            elif PHI_NS == 'S':
                if after2014 == True:
                    tcindex = landfallTC_PHI_cma_S_after2014
                    posindex = PHI_TCposition_cma_S_after2014
                else:
                    tcindex = landfallTC_PHI_cma_S
                    posindex = PHI_TCposition_cma_S
        elif source == 2: #wmo
            if PHI_NS == False:
                if after2014 == True:
                    tcindex = landfallTC_PHI_wmo_after2014
                    posindex = PHI_TCposition_wmo_after2014
                else:
                    tcindex = landfallTC_PHI_wmo
                    posindex = PHI_TCposition_wmo
            elif PHI_NS == 'N':
                if after2014 == True:
                    tcindex = landfallTC_PHI_wmo_N_after2014
                    posindex = PHI_TCposition_wmo_N_after2014
                else:
                    tcindex = landfallTC_PHI_wmo_N
                    posindex = PHI_TCposition_wmo_N
            elif PHI_NS == 'S':
                if after2014 == True:
                    tcindex = landfallTC_PHI_wmo_S_after2014
                    posindex = PHI_TCposition_wmo_S_after2014
                else:
                    tcindex = landfallTC_PHI_wmo_S
                    posindex = PHI_TCposition_wmo_S
            
            
        landmask_lat = z_land_coord_PHI[:,1]
        landmask_lon = z_land_coord_PHI[:,0]
        for i in list(tcindex): #Loop over each TC in list
            landindexinarea = []
            for j in posindex[str(i)]: #Loop over each data point for that TC
                TClat = LAT[:,:,source][i][j]
                TClon = LON[:,:,source][i][j]
                
                #Find all land points with same latitude
                landpoint_onlat_temp1 = np.where(landmask_lat <= TClat + epsilon)
                landpoint_onlat_temp2 = np.where(landmask_lat >= TClat - epsilon)
                landpoint_onlat_index = np.intersect1d(landpoint_onlat_temp1,landpoint_onlat_temp2) #Index of points in landmask array where latitude is same as TClat
                landpoint_onlat_lon = landmask_lon[landpoint_onlat_index] #Lon of landmask points with same lat as TClat
                
                if landpoint_onlat_lon.size != 0:
                    landpoint_onlat_maxlon = max(landpoint_onlat_lon) #Find easternmost PHI land point on same latitude
                    landpoint_onlat_minlon = min(landpoint_onlat_lon) #Find westernmost PHI land point on same latitude
                
                    #Check whether TC is on land or ocean
                    if landpoint_onlat_minlon-epsilon <= TClon <= landpoint_onlat_maxlon+epsilon:
                        landindexinarea.append(j)
            
            landindexinarea = np.asarray(landindexinarea)
            if landindexinarea.size != 0:
                first_land_index = landindexinarea[0]
                if first_land_index == 0:
                    indexlist = [first_land_index]
                else: 
                    indexlist = [first_land_index-1, first_land_index]

                pres[str(i)] = MIN_PRES[:,:,source][int(i)][indexlist]
                pres[str(i)] = [k for k in pres[str(i)] if k >= 0]
                if pres[str(i)] == []:
                    del pres[str(i)]
                wind[str(i)] = MAX_WIND[:,:,source][int(i)][indexlist]
                wind[str(i)] = [k for k in wind[str(i)] if k >= 0]
                if wind[str(i)] == []:
                    del wind[str(i)]
                coord[str(i)] = np.array([LAT[:,:,source][int(i),indexlist],LON[:,:,source][int(i),indexlist]]).transpose()
                if list(coord[str(i)][0]) == [-3e4,-3e4]: #Remove non-existant data
                    coord[str(i)] = np.delete(coord[str(i)],0,0)
                disttoland[str(i)] = DIST2LAND[int(i)][indexlist]
                list_of_index[str(i)] = indexlist

        if source == 1 and (PHI_NS == False or PHI_NS == 'N'):
            #Add Mangkhut (to the end of the dict), source: cma
            pres[str(len(list(season))+1)] = [910.0, 925.0] #Mangkhut
            wind[str(len(list(season))+1)] = [65*1.944, 58*1.944] #Change m/s to knots
            coord[str(len(list(season))+1)] = np.array([[17.8,123.4],[18.1,121.5]])    
                

    elif area == 1: #GD
        if source == 0: #jtwc
            if after2014 == True:
                tcindex = landfallTC_GD_jtwc_after2014
                posindex = GD_TCposition_jtwc_after2014
            else:
                tcindex = landfallTC_GD_jtwc
                posindex = GD_TCposition_jtwc
        elif source == 1: #cma
            if after2014 == True:
                tcindex = landfallTC_GD_cma_after2014
                posindex = GD_TCposition_cma_after2014
            else:
                tcindex = landfallTC_GD_cma
                posindex = GD_TCposition_cma
        elif source == 2: #wmo
            if after2014 == True:
                tcindex = landfallTC_GD_wmo_after2014
                posindex = GD_TCposition_wmo_after2014
            else:
                tcindex = landfallTC_GD_wmo
                posindex = GD_TCposition_wmo

        for i in list(tcindex):
            distlist = DIST2LAND[int(i)][DIST2LAND[int(i)]>=0]
            landindexes = np.where(distlist == 0)
            landindexinarea = np.intersect1d(posindex[str(i)],landindexes)

            if landindexinarea.size != 0:
                first_land_index = landindexinarea[0]
                if first_land_index == 0:
                    indexlist = [first_land_index]
                else: 
                    indexlist = [first_land_index-1, first_land_index]

                pres[str(i)] = MIN_PRES[:,:,source][int(i)][indexlist]
                pres[str(i)] = [k for k in pres[str(i)] if k >= 0] #Remove non-existant data
                if pres[str(i)] == []:
                    del pres[str(i)]
                wind[str(i)] = MAX_WIND[:,:,source][int(i)][indexlist]
                wind[str(i)] = [k for k in wind[str(i)] if k >= 0]
                if wind[str(i)] == []:
                    del wind[str(i)]
                coord[str(i)] = np.array([LAT[:,:,source][int(i),indexlist],LON[:,:,source][int(i),indexlist]]).transpose()
                if list(coord[str(i)][0]) == [-3e4,-3e4]: #Remove non-existant data
                    coord[str(i)] = np.delete(coord[str(i)],0,0)
                disttoland[str(i)] = DIST2LAND[int(i)][indexlist]
                list_of_index[str(i)] = indexlist
        
        if source == 0:
            #Add Hato (to the end of the dict), source: jtwc
            pres[str(len(list(season)))] = [956.0, 964.0] #Hato
            wind[str(len(list(season)))] = [90., 85.] #In knots
            coord[str(len(list(season)))] = np.array([[21.5,114.5],[22.2,112.8]])
            
        elif source == 1:        
            #Add Hato and Mangkhut (to the end of the dict), source: cma
            pres[str(len(list(season)))] = [950.0, 950.0] #Hato
            wind[str(len(list(season)))] = [42*1.944, 45*1.944] #Change m/s to knots
            coord[str(len(list(season)))] = np.array([[21.5,114.7],[22.0,113.2]])

            pres[str(len(list(season))+1)] = [950.0, 960.0] #Mangkhut
            wind[str(len(list(season))+1)] = [48*1.944, 42*1.944]
            coord[str(len(list(season))+1)] = np.array([[21.4, 113.8],[21.9, 112.0]])
            
    return pres, wind, coord, disttoland, list_of_index
            
##################### READ TILL HERE #################
    
landfalldata_pres_PHI_jtwc, landfalldata_wind_PHI_jtwc, landfalldata_coord_PHI_jtwc, landfalldata_dist2land_PHI_jtwc, landfalldata_indexlist_PHI_jtwc = get_landfall_data_beforeandafter(0,0)
landfalldata_pres_GD_jtwc, landfalldata_wind_GD_jtwc, landfalldata_coord_GD_jtwc, landfalldata_dist2land_GD_jtwc, landfalldata_indexlist_GD_jtwc = get_landfall_data_beforeandafter(1,0)
landfalldata_pres_PHI2014_jtwc, landfalldata_wind_PHI2014_jtwc, landfalldata_coord_PHI2014_jtwc, landfalldata_dist2land_PHI2014_jtwc, landfalldata_indexlist_PHI2014_jtwc = get_landfall_data_beforeandafter(0,0,True)
landfalldata_pres_GD2014_jtwc, landfalldata_wind_GD2014_jtwc, landfalldata_coord_GD2014_jtwc, landfalldata_dist2land_GD2014_jtwc, landfalldata_indexlist_GD2014_jtwc = get_landfall_data_beforeandafter(1,0,True)

landfalldata_pres_PHI_cma, landfalldata_wind_PHI_cma, landfalldata_coord_PHI_cma, landfalldata_dist2land_PHI_cma, landfalldata_indexlist_PHI_cma = get_landfall_data_beforeandafter(0,1)
landfalldata_pres_GD_cma, landfalldata_wind_GD_cma, landfalldata_coord_GD_cma, landfalldata_dist2land_GD_cma, landfalldata_indexlist_GD_cma = get_landfall_data_beforeandafter(1,1)
landfalldata_pres_PHI2014_cma, landfalldata_wind_PHI2014_cma, landfalldata_coord_PHI2014_cma, landfalldata_dist2land_PHI2014_cma, landfalldata_indexlist_PHI2014_cma = get_landfall_data_beforeandafter(0,1,True)
landfalldata_pres_GD2014_cma, landfalldata_wind_GD2014_cma, landfalldata_coord_GD2014_cma, landfalldata_dist2land_GD2014_cma, landfalldata_indexlist_GD2014_cma = get_landfall_data_beforeandafter(1,1,True)

landfalldata_pres_PHI_wmo, landfalldata_wind_PHI_wmo, landfalldata_coord_PHI_wmo, landfalldata_dist2land_PHI_wmo, landfalldata_indexlist_PHI_wmo = get_landfall_data_beforeandafter(0,2)
landfalldata_pres_GD_wmo, landfalldata_wind_GD_wmo, landfalldata_coord_GD_wmo, landfalldata_dist2land_GD_wmo, landfalldata_indexlist_GD_wmo = get_landfall_data_beforeandafter(1,2)
landfalldata_pres_PHI2014_wmo, landfalldata_wind_PHI2014_wmo, landfalldata_coord_PHI2014_wmo, landfalldata_dist2land_PHI2014_wmo, landfalldata_indexlist_PHI2014_wmo = get_landfall_data_beforeandafter(0,2,True)
landfalldata_pres_GD2014_wmo, landfalldata_wind_GD2014_wmo, landfalldata_coord_GD2014_wmo, landfalldata_dist2land_GD2014_wmo, landfalldata_indexlist_GD2014_wmo = get_landfall_data_beforeandafter(1,2,True)

landfalldata_pres_PHI_N_jtwc, landfalldata_wind_PHI_N_jtwc, landfalldata_coord_PHI_N_jtwc, landfalldata_dist2land_PHI_N_jtwc, landfalldata_indexlist_PHI_N_jtwc = get_landfall_data_beforeandafter(0,0,False,'N')
landfalldata_pres_PHI_N2014_jtwc, landfalldata_wind_PHI_N2014_jtwc, landfalldata_coord_PHI_N2014_jtwc, landfalldata_dist2land_PHI_N2014_jtwc, landfalldata_indexlist_PHI_N2014_jtwc = get_landfall_data_beforeandafter(0,0,True,'N')
landfalldata_pres_PHI_S_jtwc, landfalldata_wind_PHI_S_jtwc, landfalldata_coord_PHI_S_jtwc, landfalldata_dist2land_PHI_S_jtwc, landfalldata_indexlist_PHI_S_jtwc = get_landfall_data_beforeandafter(0,0,False,'S')
landfalldata_pres_PHI_S2014_jtwc, landfalldata_wind_PHI_S2014_jtwc, landfalldata_coord_PHI_S2014_jtwc, landfalldata_dist2land_PHI_S2014_jtwc, landfalldata_indexlist_PHI_S2014_jtwc = get_landfall_data_beforeandafter(0,0,True,'S')

landfalldata_pres_PHI_N_cma, landfalldata_wind_PHI_N_cma, landfalldata_coord_PHI_N_cma, landfalldata_dist2land_PHI_N_cma, landfalldata_indexlist_PHI_N_cma = get_landfall_data_beforeandafter(0,1,False,'N')
landfalldata_pres_PHI_N2014_cma, landfalldata_wind_PHI_N2014_cma, landfalldata_coord_PHI_N2014_cma, landfalldata_dist2land_PHI_N2014_cma, landfalldata_indexlist_PHI_N2014_cma = get_landfall_data_beforeandafter(0,1,True,'N')
landfalldata_pres_PHI_S_cma, landfalldata_wind_PHI_S_cma, landfalldata_coord_PHI_S_cma, landfalldata_dist2land_PHI_S_cma, landfalldata_indexlist_PHI_S_cma = get_landfall_data_beforeandafter(0,1,False,'S')
landfalldata_pres_PHI_S2014_cma, landfalldata_wind_PHI_S2014_cma, landfalldata_coord_PHI_S2014_cma, landfalldata_dist2land_PHI_S2014_cma, landfalldata_indexlist_PHI_S2014_cma = get_landfall_data_beforeandafter(0,1,True,'S')

landfalldata_pres_PHI_N_wmo, landfalldata_wind_PHI_N_wmo, landfalldata_coord_PHI_N_wmo, landfalldata_dist2land_PHI_N_wmo, landfalldata_indexlist_PHI_N_wmo = get_landfall_data_beforeandafter(0,2,False,'N')
landfalldata_pres_PHI_N2014_wmo, landfalldata_wind_PHI_N2014_wmo, landfalldata_coord_PHI_N2014_wmo, landfalldata_dist2land_PHI_N2014_wmo, landfalldata_indexlist_PHI_N2014_wmo = get_landfall_data_beforeandafter(0,2,True,'N')
landfalldata_pres_PHI_S_wmo, landfalldata_wind_PHI_S_wmo, landfalldata_coord_PHI_S_wmo, landfalldata_dist2land_PHI_S_wmo, landfalldata_indexlist_PHI_S_wmo = get_landfall_data_beforeandafter(0,2,False,'S')
landfalldata_pres_PHI_S2014_wmo, landfalldata_wind_PHI_S2014_wmo, landfalldata_coord_PHI_S2014_wmo, landfalldata_dist2land_PHI_S2014_wmo, landfalldata_indexlist_PHI_S2014_wmo = get_landfall_data_beforeandafter(0,2,True,'S')

toc = time.time()
print(toc - tic)

In [ ]:
#Calculate Pressure deficit

def pressure_deficit(area,source,separate_PHI = False, intersect = False):
    #separate_PHI: 'N','S' or False
    #intersect: 'True' if only using data of TCs appearing in all 3 agencies (to compare like to like events), 'False' otherwise
    
    pressure_dict = get_landfall_data_beforeandafter(area,source,False,separate_PHI)[0]
    pres_def_before_list = []
    pres_def_after_list = []
    
    if intersect == True:
        key_a = list(get_landfall_data_beforeandafter(area,0,False,separate_PHI)[0].keys())
        key_b = list(get_landfall_data_beforeandafter(area,1,False,separate_PHI)[0].keys())
        key_c = list(get_landfall_data_beforeandafter(area,2,False,separate_PHI)[0].keys())
        key_intersect_list = np.intersect1d(np.intersect1d(key_a,key_b),key_c) #Intersected list of TCs
    else:
        key_intersect_list = list(pressure_dict.keys())
    
    for key in key_intersect_list:
        if len(pressure_dict[key]) == 1: 
            pres_def_after = 1010 - pressure_dict[key][0]
            pres_def_after_list.append(pres_def_after)
        else: 
            pres_def_before = 1010 - pressure_dict[key][0]
            pres_def_after = 1010 - pressure_dict[key][1]
            pres_def_before_list.append(pres_def_before)
            pres_def_after_list.append(pres_def_after)
    return pres_def_before_list, pres_def_after_list #pressure_deficit(area)[0] for the first list and pressure_deficit(area)[1] for the second list

In [ ]:
#Calculating probability of pressure deficit with log-normal fit
def presdef_probability(area, source, binsize, separate_PHI=False, intersect = False):
    #binsize: range of data grouped into a single data point
    x_range = list(range(0,101,binsize))
    before_list, after_list = pressure_deficit(area, source, separate_PHI, intersect)
    #this returns all the pres deficits for the typhoons just before reaching lands (-1 typhoon), and just reach the lands (0 typhoon)

    prob_before = [] #Distribution before landfall
    prob_after = [] #Distribution after landfall
    for i in range(len(x_range)-1) : 
        count_before = 0 
        count_after = 0
        for j in range(len(after_list)):
            if i < len(x_range)-1 and x_range[i+1] > after_list[j] >= x_range[i]: # to make sure that it only includes one range each time but not more than one
                count_after = count_after + 1 
        for j in range(len(before_list)):
            if i < len(x_range)-1 and x_range[i+1] > before_list[j] >= x_range[i]:
                count_before = count_before + 1 
        prob_before.append((count_before/len(before_list))/(100/len(x_range)))
        prob_after.append((count_after/len(after_list))/(100/len(x_range)))
    
    prob_before = np.asarray(prob_before)
    prob_after = np.asarray(prob_after)
    
    #Plot figure for before landfall
    plt.figure()
    plt.xlabel('Pressure deficit (hPa)')
    plt.ylabel('Probability')
    x_axis = list(np.arange(binsize/2,101,binsize))
    plt.scatter(x_axis, prob_before,color = 'k')
    s, loc, scale = lognorm.fit(before_list, loc=0) #Fit log-normal
    mu, var, skew = lognorm.stats(s, loc, scale, moments='mvs') #Get log-normal statistics
    std = np.sqrt(var)
    x = np.arange(0, 101, 1)
    p = lognorm.pdf(x, s, loc, scale)
    p = np.asarray(p)
    zeroindexlist = np.where(p == 0)
    p[zeroindexlist] = 1e-80 #Make zero to a very small number for chi-square test
    
    x_at_datapt_index = [i for i in range(len(x)) if x[i] in x_axis] #Find index in x where there is a data point in x_axis
    p_mse = p[x_at_datapt_index]
    p_chisq = p[x_at_datapt_index]
    chisq, pvalue = chisquare(prob_before*100/len(x_range)*len(before_list),p_chisq*100/len(x_range)*len(before_list), ddof=2) #Chi-sq test
#    print(str(source), chisq, pvalue)
    rmserr = np.sqrt(mean_squared_error(prob_before, p_mse))

    samplesize = len(before_list)
    
    plt.plot(x, p, 'b-')
    
    #Plot title
    if area == 0:
        if source == 0:
            if separate_PHI == False:
                plt.title('Probability distribution of central pressure deficit of typhoons before their landfalls in the Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
            elif separate_PHI == 'N':
                plt.title('Probability distribution of central pressure deficit of typhoons before their landfalls in the Northern Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
            elif separate_PHI == 'S':
                plt.title('Probability distribution of central pressure deficit of typhoons before their landfalls in the Southern Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
        elif source == 1:
            if separate_PHI == False:
                plt.title('Probability distribution of central pressure deficit of typhoons before their landfalls in the Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
            elif separate_PHI == 'N':
                plt.title('Probability distribution of central pressure deficit of typhoons before their landfalls in the Northern Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
            elif separate_PHI == 'S':
                plt.title('Probability distribution of central pressure deficit of typhoons before their landfalls in the Southern Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
        elif source == 2: 
            if separate_PHI == False:
                plt.title('Probability distribution of central pressure deficit of typhoons before their landfalls in the Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
            elif separate_PHI == 'N':
                plt.title('Probability distribution of central pressure deficit of typhoons before their landfalls in the Northern Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
            elif separate_PHI == 'S':
                plt.title('Probability distribution of central pressure deficit of typhoons before their landfalls in the Southern Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
    else: 
        if source == 0:
            plt.title('Probability distribution of central pressure deficit of typhoons before their landfalls in Guangdong \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
        elif source == 1:
            plt.title('Probability distribution of central pressure deficit of typhoons before their landfalls in Guangdong \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
        elif source == 2:
            plt.title('Probability distribution of central pressure deficit of typhoons before their landfalls in Guangdong \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
    plt.xlim([0,100])
    plt.ylim([0,0.05])
    plt.show()
    
    
    #Plot figure for after landfall
    plt.figure()
    plt.xlabel('Pressure deficit (hPa)')
    plt.ylabel('Probability')
    x_axis = list(np.arange(binsize/2,101,binsize))
    plt.scatter(x_axis, prob_after,color = 'k')
    s, loc, scale = lognorm.fit(after_list, loc=0)
    mu, var, skew = lognorm.stats(s, loc, scale, moments='mvs')
    std = np.sqrt(var)
    
    x = np.arange(0, 101, 1)
    p = lognorm.pdf(x, s, loc, scale)
    p = np.asarray(p)
    zeroindexlist = np.where(p == 0)
    p[zeroindexlist] = 1e-80 #Make zero to a very small number for chi-square test
    
    x_at_datapt_index = [i for i in range(len(x)) if x[i] in x_axis] #Find index in x where there is a data point in x_axis
    p_mse = p[x_at_datapt_index]
    p_chisq = p[x_at_datapt_index]#*(100/len(x_range))
    chisq, pvalue = chisquare(prob_after*100/len(x_range)*len(after_list),p_chisq*100/len(x_range)*len(after_list), ddof=2)
#    print(str(source), chisq, pvalue)

    rmserr = np.sqrt(mean_squared_error(prob_after, p_mse))

    samplesize = len(after_list)
    plt.plot(x, p, 'b-')

#     print(prob_after*100/len(x_range)*len(after_list))
#     print(p_chisq*100/len(x_range)*len(after_list))
    
    if area == 0:
        if source == 0:
            if separate_PHI == False:
                plt.title('Probability distribution of central pressure deficit of typhoons at their landfalls in the Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
            elif separate_PHI == 'N':
                plt.title('Probability distribution of central pressure deficit of typhoons at their landfalls in the Northern Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
            elif separate_PHI == 'S':
                plt.title('Probability distribution of central pressure deficit of typhoons at their landfalls in the Southern Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
        elif source == 1:
            if separate_PHI == False:
                plt.title('Probability distribution of central pressure deficit of typhoons at their landfalls in the Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
            elif separate_PHI == 'N':
                plt.title('Probability distribution of central pressure deficit of typhoons at their landfalls in the Northern Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
            elif separate_PHI == 'S':
                plt.title('Probability distribution of central pressure deficit of typhoons at their landfalls in the Southern Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
        elif source == 2:
            if separate_PHI == False:
                plt.title('Probability distribution of central pressure deficit of typhoons at their landfalls in the Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
            elif separate_PHI == 'N':
                plt.title('Probability distribution of central pressure deficit of typhoons at their landfalls in the Northern Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
            elif separate_PHI == 'S':
                plt.title('Probability distribution of central pressure deficit of typhoons at their landfalls in the Southern Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
    
    else: 
        if source == 0:
            plt.title('Probability distribution of central pressure deficit of typhoons at their landfalls in Guangdong \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
        elif source == 1:
            plt.title('Probability distribution of central pressure deficit of typhoons at their landfalls in Guangdong \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
        elif source == 2: 
            plt.title('Probability distribution of central pressure deficit of typhoons at their landfalls in Guangdong \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ' , RMS error = ' + str(rmserr),fontsize='small')
    plt.xlim([0,100])
    plt.ylim([0,0.05])
    plt.show()
    
        
    return prob_before, prob_after

In [ ]:
tic = time.time()
# presdef_probability(area=0,source=0,binsize=10)
# presdef_probability(0,1,10)
# presdef_probability(0,2,10)
# presdef_probability(1,0,10, False,False)
# presdef_probability(1,1,10, False,False)
# presdef_probability(1,2,10, False,False)
# presdef_probability(0,0,10,'N')
# presdef_probability(0,0,10,'S')
# presdef_probability(0,1,10,'N')
# presdef_probability(0,1,10,'S')
#presdef_probability(0,2,10,'N')
#presdef_probability(0,2,10,'S')
toc = time.time()
print(toc-tic)

In [ ]:
#----------------------------------------------------
'''
calculating probability of max wind speed with log-normal fit
'''
def wind_speed(area, source, separate_PHI=False, intersect = False, pd_wind_int = False):
    #pd_wind_int: 'True' if same TCs for plotting pd prob (in above) and wind prob, 'False' otherwise
    windspeed_dict = get_landfall_data_beforeandafter(area, source, False,separate_PHI)[1]
    windspeed_before_list = []
    windspeed_after_list = []
    
    if pd_wind_int == True:
        pressure_dict = get_landfall_data_beforeandafter(area, source, False,separate_PHI)[0]
        key_a = list(pressure_dict.keys())
        key_b = list(windspeed_dict.keys())
        key_intersect_list = np.intersect1d(key_a,key_b)
    
    elif intersect == True:
        key_a = list(get_landfall_data_beforeandafter(area,0,False,separate_PHI)[1].keys())
        key_b = list(get_landfall_data_beforeandafter(area,1,False,separate_PHI)[1].keys())
        key_c = list(get_landfall_data_beforeandafter(area,2,False,separate_PHI)[1].keys())
        key_intersect_list = np.intersect1d(np.intersect1d(key_a,key_b),key_c)
    else:
        key_intersect_list = list(windspeed_dict.keys())
        
    
    for key in key_intersect_list:
        if len(windspeed_dict[key]) == 1: 
            windspeed_after_list. append(windspeed_dict[key][0])
        else: 
            windspeed_before_list.append(windspeed_dict[key][0])
            windspeed_after_list.append(windspeed_dict[key][1])

    return windspeed_before_list, windspeed_after_list #before landfall, after landfall

In [ ]:
#----------------------------------------------------
'''
calculating probability of max wind speed 
'''
def wind_probability(area, source, separate_PHI = False, fitnorm = False, intersect = False, pd_wind_int = False):
    #fitnorm: 'True' if fitting normal function, 'False' if fitting log-normal
    x_range = list(range(0,131,10))
    b, a = wind_speed(area, source, separate_PHI, intersect, pd_wind_int)
    b = [i for i in b if i != 0] #Remove 0 values as they are actually missing data
    a = [j for j in a if j != 0]
    before_list = [int(i) for i in b]
    after_list = [int(j) for j in a]
    

    
    prob_before = []
    prob_after = []
    for i in range(len(x_range)-1) : 
        count_before = 0 
        count_after = 0
        for j in range(len(after_list)):
            if i < len(x_range)-1 and x_range[i+1] > after_list[j] >= x_range[i]: # to make sure that it only includes one range each time but not more than one
                count_after = count_after + 1 
        for j in range(len(before_list)):
            if i < len(x_range)-1 and x_range[i+1] > before_list[j] >= x_range[i]:
                count_before = count_before + 1 
        prob_before.append((count_before/len(before_list))/(130/len(x_range)))
        prob_after.append((count_after/len(after_list))/(130/len(x_range)))
        
    prob_before = np.asarray(prob_before)
    prob_after = np.asarray(prob_after)
        
    #Plot before landfall
    plt.figure()
    plt.xlabel('Maximum Wind Speed (kt)')
    plt.ylabel('Probability') 
    plt.ylim(0,0.06)
    x_axis = list(np.arange(5,130,10))
    plt.scatter(x_axis, prob_before,color = 'k')

    x = np.arange(0, 131, 1)
    
    if fitnorm == True:
        mu, std = norm.fit(after_list)
        p = norm.pdf(x, mu,std)
        skew = 0
    else:
        s, loc, scale = lognorm.fit(before_list, loc = 0)
        mu, var, skew = lognorm.stats(s, loc, scale, moments='mvs')
        std = np.sqrt(var)
        p = lognorm.pdf(x, s, loc, scale)
    p = np.asarray(p)
    zeroindexlist = np.where(p == 0)
    p[zeroindexlist] = 1e-80 #Make zero to a very small number for chi-square test
    
    x_at_datapt_index = [i for i in range(len(x)) if x[i] in x_axis] #Find index in x where there is a data point in x_axis
    p_mse = p[x_at_datapt_index]
    chisq, pvalue = chisquare(prob_before*130/len(x_range)*len(before_list), p_mse*130/len(x_range)*len(before_list), ddof=2) #Perform chi-sq test
#    print(str(source), chisq, pvalue)

    rmserr = np.sqrt(mean_squared_error(prob_before, p_mse))
    
    samplesize = len(before_list)
    plt.plot(x, p,'b-')
    
    #Plot title
    if area == 0:
        if source == 0:
            if separate_PHI == False:
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
            elif separate_PHI == 'N':
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Northern Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
            elif separate_PHI == 'S':
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Southern Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
        elif source == 1:
            if separate_PHI == False:
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
            elif separate_PHI == 'N':
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Northern Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
            elif separate_PHI == 'S':
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Southern Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
        elif source == 2:
            if separate_PHI == False:
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
            elif separate_PHI == 'N':
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Northern Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
            elif separate_PHI == 'S':
                plt.title('Probability distribution of max wind speed of typhoons before their landfalls in the Southern Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
    
    else: 
        if source == 0:
            plt.title('Probability distribution of max wind speed of typhoons before their landfalls in Guangdong \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
        elif source == 1:
            plt.title('Probability distribution of max wind speed of typhoons before their landfalls in Guangdong \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
        elif source == 2:
            plt.title('Probability distribution of max wind speed of typhoons before their landfalls in Guangdong \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
    plt.show()
    
    
    #Plot after landfall
    plt.figure()
    plt.xlabel('Maximum Wind Speed (kt)')  
    plt.ylabel('Probability') 
    plt.ylim(0,0.06)
    x_axis = list(np.arange(5,130,10))
    plt.scatter(x_axis, prob_after,color = 'r')
    
    x = np.arange(0, 131, 1)
    if fitnorm == True:
        mu, std = norm.fit(after_list)
        p = norm.pdf(x, mu,std)
        skew = 0
    else:
        s, loc, scale = lognorm.fit(after_list, loc = 0)
        mu, var, skew = lognorm.stats(s, loc, scale, moments='mvs')
        std = np.sqrt(var)
        p = lognorm.pdf(x, s, loc, scale)
    p = np.asarray(p)
    zeroindexlist = np.where(p == 0)
    p[zeroindexlist] = 1e-80 #Make zero to a very small number for chi-square test
    
    x_at_datapt_index = [i for i in range(len(x)) if x[i] in x_axis] #Find index in x where there is a data point in x_axis
    p_mse = p[x_at_datapt_index]
    chisq, pvalue = chisquare(prob_after*130/len(x_range)*len(after_list), p_mse*130/len(x_range)*len(after_list), ddof=2)
#    print(str(source), chisq, pvalue)

    rmserr = np.sqrt(mean_squared_error(prob_after, p_mse))

    samplesize = len(after_list)
    plt.plot(x, p, 'b-')
    
    print(prob_after*130/len(x_range)*len(after_list))
    print(p_mse*130/len(x_range)*len(after_list))

    
    if area == 0:
        if source == 0:
            if separate_PHI == False:
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
            elif separate_PHI == 'N':
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Northern Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
            elif separate_PHI == 'S':
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Southern Philippines \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
        elif source == 1:
            if separate_PHI == False:
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
            elif separate_PHI == 'N':
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Northern Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
            elif separate_PHI == 'S':
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Southern Philippines \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
        elif source == 2: 
            if separate_PHI == False:
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
            elif separate_PHI == 'N':
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Northern Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
            elif separate_PHI == 'S':
                plt.title('Probability distribution of max wind speed of typhoons at their landfalls in the Southern Philippines \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small') 
    else: 
        if source == 0:
            plt.title('Probability distribution of max wind speed of typhoons at their landfalls in Guangdong \n source: JTWC, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
        elif source == 1:
            plt.title('Probability distribution of max wind speed of typhoons at their landfalls in Guangdong \n source: CMA, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
        elif source == 2:
            plt.title('Probability distribution of max wind speed of typhoons at their landfalls in Guangdong \n source: WMO, sample size = ' + str(samplesize) + ', mu = ' + str(mu) + '\n s.d. = ' + str(std) + ', skewness = ' + str(skew) + ', RMS error = ' + str(rmserr), fontsize='small')
    plt.show()
    
    return prob_before, prob_after

In [ ]:
tic = time.time()
# wind_probability(0,0,False)
# wind_probability(0,1,False)
# wind_probability(0,2,False)
# wind_probability(1,0,False,False,False,True)
# wind_probability(1,0,False,False,False,False)
# wind_probability(1,1,False,False,False,True)
# wind_probability(1,2,False,False,False,True)
# wind_probability(0, 0,'N')
# wind_probability(0, 1,'N')
# wind_probability(0, 2,'N')
# wind_probability(0, 0,'S')
# wind_probability(0, 1,'S')
# wind_probability(0, 2,'S')
toc = time.time()
print(toc-tic)

In [ ]:
#Visualise TC coordinates
#Plots the coastline and the landfall TC coordinates of whole area
def plot_points(area, source, after2014 = False, PHI_NS = False): 
    if area == 0: #PHI
        landcoord = z_land_coord_PHI
        if source == 0:
            if after2014 == True:
                if PHI_NS == False:
                    coordlist = landfalldata_coord_PHI2014_jtwc #List of coordinates of TC centre
                    keys = landfalldata_coord_PHI2014_jtwc.keys() #Keys of dict (TC indexes)
                elif PHI_NS == 'N':
                    coordlist = landfalldata_coord_PHI_N2014_jtwc
                    keys = landfalldata_coord_PHI_N2014_jtwc.keys()
                elif PHI_NS == 'S':
                    coordlist = landfalldata_coord_PHI_S2014_jtwc
                    keys = landfalldata_coord_PHI_S2014_jtwc.keys()
            elif after2014 == False:
                if PHI_NS == False:
                    coordlist = landfalldata_coord_PHI_jtwc
                    keys = landfalldata_coord_PHI_jtwc.keys()
                elif PHI_NS == 'N':
                    coordlist = landfalldata_coord_PHI_N_jtwc
                    keys = landfalldata_coord_PHI_N_jtwc.keys()
                elif PHI_NS == 'S':
                    coordlist = landfalldata_coord_PHI_S_jtwc
                    keys = landfalldata_coord_PHI_S_jtwc.keys()
                    
        elif source == 1:
            if after2014 == True:
                if PHI_NS == False:
                    coordlist = landfalldata_coord_PHI2014_cma
                    keys = landfalldata_coord_PHI2014_cma.keys()
                elif PHI_NS == 'N':
                    coordlist = landfalldata_coord_PHI_N2014_cma
                    keys = landfalldata_coord_PHI_N2014_cma.keys()
                elif PHI_NS == 'S':
                    coordlist = landfalldata_coord_PHI_S2014_cma
                    keys = landfalldata_coord_PHI_S2014_cma.keys()
            elif after2014 == False:
                if PHI_NS == False:
                    coordlist = landfalldata_coord_PHI_cma
                    keys = landfalldata_coord_PHI_cma.keys()
                elif PHI_NS == 'N':
                    coordlist = landfalldata_coord_PHI_N_cma
                    keys = landfalldata_coord_PHI_N_cma.keys()
                elif PHI_NS == 'S':
                    coordlist = landfalldata_coord_PHI_S_cma
                    keys = landfalldata_coord_PHI_S_cma.keys()
        elif source == 2:
            if after2014 == True:
                if PHI_NS == False:
                    coordlist = landfalldata_coord_PHI2014_wmo
                    keys = landfalldata_coord_PHI2014_wmo.keys()
                elif PHI_NS == 'N':
                    coordlist = landfalldata_coord_PHI_N2014_wmo
                    keys = landfalldata_coord_PHI_N2014_wmo.keys()
                elif PHI_NS == 'S':
                    coordlist = landfalldata_coord_PHI_S2014_wmo
                    keys = landfalldata_coord_PHI_S2014_wmo.keys()
            elif after2014 == False:
                if PHI_NS == False:
                    coordlist = landfalldata_coord_PHI_wmo
                    keys = landfalldata_coord_PHI_wmo.keys()
                elif PHI_NS == 'N':
                    coordlist = landfalldata_coord_PHI_N_wmo
                    keys = landfalldata_coord_PHI_N_wmo.keys()
                elif PHI_NS == 'S':
                    coordlist = landfalldata_coord_PHI_S_wmo
                    keys = landfalldata_coord_PHI_S_wmo.keys()
                    
                    
                    
    elif area == 1: #GD
        landcoord = z_land_coord_GD
        if source == 0:
            if after2014 == True:
                coordlist = landfalldata_coord_GD2014_jtwc
                keys = landfalldata_coord_GD2014_jtwc.keys()
            elif after2014 == False:
                coordlist = landfalldata_coord_GD_jtwc
                keys = landfalldata_coord_GD_jtwc.keys()
        elif source == 1:
            if after2014 == True:
                coordlist = landfalldata_coord_GD2014_cma
                keys = landfalldata_coord_GD2014_cma.keys()
            elif after2014 == False:
                coordlist = landfalldata_coord_GD_cma
                keys = landfalldata_coord_GD_cma.keys()
        elif source == 2:
            if after2014 == True:
                coordlist = landfalldata_coord_GD2014_wmo
                keys = landfalldata_coord_GD2014_wmo.keys()
            elif after2014 == False:
                coordlist = landfalldata_coord_GD_wmo
                keys = landfalldata_coord_GD_wmo.keys()    
    color = iter(cm.rainbow(np.linspace(0,1,len(keys))))
    plt.figure()  
    if area == 0: 
        if source == 0:
            if PHI_NS == False:
                plt.title('Tracks of typhoons with landfall in the Philippines, source: JTWC') 
            elif PHI_NS == 'N':
                plt.title('Tracks of typhoons with landfall in the Northern Philippines, source: JTWC') 
            elif PHI_NS == 'S':
                plt.title('Tracks of typhoons with landfall in the Southern Philippines, source: JTWC')
        elif source == 1:
            if PHI_NS == False:
                plt.title('Tracks of typhoons with landfall in the Philippines, source: CMA') 
            elif PHI_NS == 'N':
                plt.title('Tracks of typhoons with landfall in the Northern Philippines, source: CMA') 
            elif PHI_NS == 'S':
                plt.title('Tracks of typhoons with landfall in the Southern Philippines, source: CMA')
        elif source == 2:
            if PHI_NS == False:
                plt.title('Tracks of typhoons with landfall in the Philippines, source: WMO') 
            elif PHI_NS == 'N':
                plt.title('Tracks of typhoons with landfall in the Northern Philippines, source: WMO') 
            elif PHI_NS == 'S':
                plt.title('Tracks of typhoons with landfall in the Southern Philippines, source: WMO')
    else:
        if source == 0:
            plt.title('Tracks of typhoons with landfall in Guangdong, source: JTWC')
        elif source == 1:
            plt.title('Tracks of typhoons with landfall in Guangdong, source: CMA')
        elif source == 2:   
            plt.title('Tracks of typhoons with landfall in Guangdong, source: WMO')
    plt.xlabel('Longitude (E)') 
    plt.ylabel('Latitude (N)')
    plt.scatter(landcoord[:,0],landcoord[:,1],color = 'k', s=5)
    if after2014 == True:
        size = 20
    else:
        size = 5
    for index in list(keys):
        c=next(color)
        plt.scatter(coordlist[index][:,1],coordlist[index][:,0],color = c,s=size, label = index)
    if after2014 == False:
        plt.legend(fontsize=4)
    else:
        plt.legend(fontsize='x-small')
    plt.axis('equal')


#Plots the coastline and the landfall TC coordinates of individual TC
def plot_points_landfall_indiv(source, tcindex):
    plt.figure()
    plt.title(str(tcindex))
    if source == 0:
        coorddictPHI = landfalldata_coord_PHI_jtwc
        coorddictGD = landfalldata_coord_GD_jtwc
    else:
        coorddictPHI = landfalldata_coord_PHI_cma
        coorddictGD = landfalldata_coord_GD_cma
    if str(tcindex) in landfalldata_coord_PHI:
        plt.scatter(z_land_coord_PHI[:,0],z_land_coord_PHI[:,1],color = 'k')
        plt.scatter(coorddictPHI[str(tcindex)][:,1],coorddictPHI[str(tcindex)][:,0], color = 'r')
    if str(tcindex) in landfalldata_coord_GD:
        plt.scatter(z_land_coord_GD[:,0],z_land_coord_GD[:,1],color = 'k')
        plt.scatter(coorddictGD[str(tcindex)][:,1],coorddictGD[str(tcindex)][:,0], color = 'b')

In [ ]:
tic = time.time()
plot_points(0, 0, True, False)
plot_points(0, 1, True, False)
plot_points(0, 2, True, False)
plot_points(0, 0, True, 'N')
plot_points(0, 1, True, 'N')
plot_points(0, 0, True, 'S')
plot_points(0, 1, True, 'S')
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
plot_points(0, 0, False)
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
plot_points(1, 1, True)
#plot_points(1, 1, True)
#plot_points(1, 0, False)
#plot_points(1, 1, False)
toc = time.time()
print(toc-tic)

In [ ]:
#------------------------------------------------------
'''
extract dates of typhoon for rainfall data
'''

time_units = ibt.variables['source_time'].getncattr('units')

#Change dates of TCs to common understandable format
def getdates(tcindex): 
    datelist = list(num2date(time_record,time_units)[tcindex])

    return datelist #returns a list in (year, month, day, hour, minute, second) format

#Get dates of TCs at landfall
def getdates_landfall(area, after2014 = False):
    landfall_dates = {}
    if after2014 == True:
        indexlist = get_landfall_data_beforeandafter(area, True)[4]
    else:
        indexlist = get_landfall_data_beforeandafter(area, False)[4]
        
    for key in indexlist.keys():
        print(key)
        landfall_dates[key] = list(num2date(time_record,time_units)[int(key), indexlist[key]])
    return landfall_dates

#WARNING: Extremely long runtime, dates are pasted in cell below so code does not have to be rerun again

tic = time.time()
#landfallTC_dates_GD_2014 = getdates_landfall(1, after2014 = True)
landfallTC_dates_GD = getdates_landfall(1, after2014 = False)
print(landfallTC_dates_GD)
toc = time.time()
print(toc - tic)

'''For GD:
{'12594': [datetime.datetime(2014, 6, 15, 6, 0), datetime.datetime(2014, 6, 15, 12, 0)],
 '12600': [datetime.datetime(2014, 7, 18, 6, 0), datetime.datetime(2014, 7, 18, 12, 0)], 
 '12621': [datetime.datetime(2014, 9, 8, 0, 0), datetime.datetime(2014, 9, 8, 6, 0)], 
 '12624': [datetime.datetime(2014, 9, 16, 0, 0), datetime.datetime(2014, 9, 16, 6, 0)], 
 '12688': [datetime.datetime(2015, 7, 9, 0, 0), datetime.datetime(2015, 7, 9, 6, 0)], 
 '12733': [datetime.datetime(2015, 10, 4, 0, 0), datetime.datetime(2015, 10, 4, 6, 0)], 
 '12776': [datetime.datetime(2016, 5, 27, 6, 0), datetime.datetime(2016, 5, 27, 12, 0)], 
 '12793': [datetime.datetime(2016, 8, 1, 18, 0), datetime.datetime(2016, 8, 2, 0, 0)], 
 '12837': [datetime.datetime(2016, 10, 21, 0, 0), datetime.datetime(2016, 10, 21, 6, 0)]
 '12861': [datetime.datetime(2017, 8, 23, 0, 0), datetime.datetime(2017, 8, 23, 6, 0)
 '12862': [datetime.datetime(2018, 9, 16, 6, 0), datetime.datetime(2016, 9, 16, 12, 0)}
'''

'''For GD: {
'4872': [datetime.datetime(1945, 7, 7, 12, 0), datetime.datetime(1945, 7, 7, 18, 0)], 
'4968': [datetime.datetime(1946, 7, 18, 12, 0), datetime.datetime(1946, 7, 18, 18, 0)], 
'4981': [datetime.datetime(1946, 9, 12, 6, 0), datetime.datetime(1946, 9, 12, 12, 0)], 
'5066': [datetime.datetime(1947, 8, 13, 0, 0), datetime.datetime(1947, 8, 13, 6, 0)], 
'5087': [datetime.datetime(1947, 10, 7, 6, 0), datetime.datetime(1947, 10, 7, 12, 0)], 
'5156': [datetime.datetime(1948, 6, 10, 6, 0), datetime.datetime(1948, 6, 10, 12, 0)], 
'5164': [datetime.datetime(1948, 7, 27, 6, 0), datetime.datetime(1948, 7, 27, 12, 0)], 
'5173': [datetime.datetime(1948, 9, 3, 0, 0), datetime.datetime(1948, 9, 3, 6, 0)], 
'5183': [datetime.datetime(1948, 9, 26, 18, 0), datetime.datetime(1948, 9, 27, 0, 0)], 
'5252': [datetime.datetime(1949, 7, 10, 6, 0), datetime.datetime(1949, 7, 10, 12, 0)], 
'5275': [datetime.datetime(1949, 9, 8, 0, 0), datetime.datetime(1949, 9, 8, 6, 0)], 
'5285': [datetime.datetime(1949, 9, 14, 18, 0), datetime.datetime(1949, 9, 15, 0, 0)], 
'5504': [datetime.datetime(1951, 9, 22, 0, 0), datetime.datetime(1951, 9, 22, 6, 0)], 
'5563': [datetime.datetime(1952, 6, 13, 12, 0), datetime.datetime(1952, 6, 13, 18, 0)], 
'5596': [datetime.datetime(1952, 9, 13, 6, 0), datetime.datetime(1952, 9, 13, 12, 0)], 
'5684': [datetime.datetime(1953, 9, 1, 12, 0), datetime.datetime(1953, 9, 1, 18, 0)], 
'5693': [datetime.datetime(1953, 9, 19, 6, 0), datetime.datetime(1953, 9, 19, 12, 0)], 
'5769': [datetime.datetime(1954, 8, 29, 12, 0), datetime.datetime(1954, 8, 29, 18, 0)], 
'5852': [datetime.datetime(1955, 6, 5, 12, 0), datetime.datetime(1955, 6, 5, 18, 0)], 
'6078': [datetime.datetime(1957, 7, 16, 6, 0), datetime.datetime(1957, 7, 16, 12, 0)], 
'6085': [datetime.datetime(1957, 8, 20, 12, 0), datetime.datetime(1957, 8, 20, 18, 0)], 
'6111': [datetime.datetime(1957, 10, 15, 6, 0), datetime.datetime(1957, 10, 15, 12, 0)], 
'6287': [datetime.datetime(1959, 7, 5, 18, 0), datetime.datetime(1959, 7, 6, 0, 0)], 
'6318': [datetime.datetime(1959, 9, 10, 18, 0), datetime.datetime(1959, 9, 11, 0, 0)], 
'6382': [datetime.datetime(1960, 6, 8, 18, 0), datetime.datetime(1960, 6, 9, 0, 0)], 
'6385': [datetime.datetime(1960, 6, 29, 18, 0), datetime.datetime(1960, 6, 30, 0, 0)], 
'6399': [datetime.datetime(1960, 8, 9, 0, 0), datetime.datetime(1960, 8, 9, 6, 0)], 
'6413': [datetime.datetime(1960, 8, 25, 0, 0), datetime.datetime(1960, 8, 25, 6, 0)], 
'6492': [datetime.datetime(1961, 5, 19, 0, 0), datetime.datetime(1961, 5, 19, 6, 0)], 
'6504': [datetime.datetime(1961, 7, 1, 18, 0), datetime.datetime(1961, 7, 2, 0, 0)], 
'6513': [datetime.datetime(1961, 7, 14, 18, 0), datetime.datetime(1961, 7, 15, 0, 0)], 
'6515': [datetime.datetime(1961, 7, 19, 6, 0), datetime.datetime(1961, 7, 19, 12, 0)], 
'6542': [datetime.datetime(1961, 9, 9, 12, 0), datetime.datetime(1961, 9, 9, 18, 0)], 
'6552': [datetime.datetime(1961, 9, 29, 0, 0), datetime.datetime(1961, 9, 29, 6, 0)], 
'6640': [datetime.datetime(1962, 8, 10, 12, 0), datetime.datetime(1962, 8, 10, 18, 0)], 
'6648': [datetime.datetime(1962, 9, 1, 6, 0), datetime.datetime(1962, 9, 1, 12, 0)], 
'6666': [datetime.datetime(1962, 10, 3, 12, 0), datetime.datetime(1962, 10, 3, 18, 0)], 
'6748': [datetime.datetime(1963, 6, 30, 18, 0), datetime.datetime(1963, 7, 1, 0, 0)], 
'6757': [datetime.datetime(1963, 7, 22, 12, 0), datetime.datetime(1963, 7, 22, 18, 0)], 
'6854': [datetime.datetime(1964, 5, 28, 0, 0), datetime.datetime(1964, 5, 28, 6, 0)], 
'6872': [datetime.datetime(1964, 8, 8, 12, 0), datetime.datetime(1964, 8, 8, 18, 0)], 
'6892': [datetime.datetime(1964, 9, 5, 6, 0), datetime.datetime(1964, 9, 5, 12, 0)], 
'6911': [datetime.datetime(1964, 10, 12, 18, 0), datetime.datetime(1964, 10, 13, 0, 0)], 
'6988': [datetime.datetime(1965, 6, 4, 6, 0), datetime.datetime(1965, 6, 4, 12, 0)], 
'7003': [datetime.datetime(1965, 7, 23, 0, 0), datetime.datetime(1965, 7, 23, 6, 0)], 
'7030': [datetime.datetime(1965, 9, 5, 6, 0), datetime.datetime(1965, 9, 5, 12, 0)], 
'7044': [datetime.datetime(1965, 9, 27, 12, 0), datetime.datetime(1965, 9, 27, 18, 0)], 
'7104': [datetime.datetime(1966, 7, 13, 12, 0), datetime.datetime(1966, 7, 13, 18, 0)], 
'7105': [datetime.datetime(1966, 7, 17, 12, 0), datetime.datetime(1966, 7, 17, 18, 0)], 
'7112': [datetime.datetime(1966, 7, 26, 0, 0), datetime.datetime(1966, 7, 26, 6, 0)], 
'7128': [datetime.datetime(1966, 9, 3, 18, 0), datetime.datetime(1966, 9, 4, 0, 0)], 
'7233': [datetime.datetime(1967, 6, 30, 0, 0), datetime.datetime(1967, 6, 30, 6, 0)], 
'7259': [datetime.datetime(1967, 8, 16, 6, 0), datetime.datetime(1967, 8, 16, 12, 0)], 
'7264': [datetime.datetime(1967, 8, 21, 6, 0), datetime.datetime(1967, 8, 21, 12, 0)], 
'7321': [datetime.datetime(1967, 11, 7, 18, 0), datetime.datetime(1967, 11, 8, 0, 0)], 
'7405': [datetime.datetime(1968, 8, 21, 12, 0), datetime.datetime(1968, 8, 21, 18, 0)], 
'7418': [datetime.datetime(1968, 9, 8, 12, 0), datetime.datetime(1968, 9, 8, 18, 0)], 
'7433': [datetime.datetime(1968, 10, 1, 0, 0), datetime.datetime(1968, 10, 1, 6, 0)], 
'7513': [datetime.datetime(1969, 7, 28, 0, 0), datetime.datetime(1969, 7, 28, 6, 0)], 
'7643': [datetime.datetime(1970, 7, 16, 0, 0), datetime.datetime(1970, 7, 16, 6, 0)], 
'7662': [datetime.datetime(1970, 8, 9, 0, 0), datetime.datetime(1970, 8, 9, 6, 0)], 
'7691': [datetime.datetime(1970, 9, 13, 18, 0), datetime.datetime(1970, 9, 14, 0, 0)], 
'7712': [datetime.datetime(1970, 10, 17, 6, 0), datetime.datetime(1970, 10, 17, 12, 0)], 
'7775': [datetime.datetime(1971, 5, 3, 18, 0), datetime.datetime(1971, 5, 4, 0, 0)], 
'7791': [datetime.datetime(1971, 6, 27, 18, 0), datetime.datetime(1971, 6, 28, 0, 0)], 
'7802': [datetime.datetime(1971, 7, 22, 0, 0), datetime.datetime(1971, 7, 22, 6, 0)], 
'7816': [datetime.datetime(1971, 8, 16, 18, 0), datetime.datetime(1971, 8, 17, 0, 0)], 
'7930': [datetime.datetime(1972, 6, 5, 6, 0), datetime.datetime(1972, 6, 5, 12, 0)], 
'7935': [datetime.datetime(1972, 6, 27, 0, 0), datetime.datetime(1972, 6, 27, 6, 0)], 
'8003': [datetime.datetime(1972, 11, 8, 6, 0), datetime.datetime(1972, 11, 8, 12, 0)], 
'8083': [datetime.datetime(1973, 7, 17, 0, 0), datetime.datetime(1973, 7, 17, 6, 0)], 
'8095': [datetime.datetime(1973, 8, 12, 6, 0), datetime.datetime(1973, 8, 12, 12, 0)], 
'8101': [datetime.datetime(1973, 8, 21, 12, 0), datetime.datetime(1973, 8, 21, 18, 0)], 
'8109': [datetime.datetime(1973, 9, 6, 6, 0), datetime.datetime(1973, 9, 6, 12, 0)], 
'8196': [datetime.datetime(1974, 6, 8, 0, 0), datetime.datetime(1974, 6, 8, 6, 0)], 
'8213': [datetime.datetime(1974, 7, 22, 6, 0), datetime.datetime(1974, 7, 22, 12, 0)], 
'8250': [datetime.datetime(1974, 9, 6, 6, 0), datetime.datetime(1974, 9, 6, 12, 0)], 
'8284': [datetime.datetime(1974, 12, 2, 0, 0), datetime.datetime(1974, 12, 2, 6, 0)], 
'8390': [datetime.datetime(1975, 9, 23, 6, 0), datetime.datetime(1975, 9, 23, 12, 0)], 
'8399': [datetime.datetime(1975, 10, 5, 18, 0), datetime.datetime(1975, 10, 6, 0, 0)], 
'8402': [datetime.datetime(1975, 10, 14, 12, 0), datetime.datetime(1975, 10, 14, 18, 0)], 
'8498': [datetime.datetime(1976, 7, 25, 18, 0), datetime.datetime(1976, 7, 26, 0, 0)], 
'8507': [datetime.datetime(1976, 8, 6, 12, 0), datetime.datetime(1976, 8, 6, 18, 0)], 
'8520': [datetime.datetime(1976, 8, 23, 18, 0), datetime.datetime(1976, 8, 24, 0, 0)], 
'8534': [datetime.datetime(1976, 9, 19, 18, 0), datetime.datetime(1976, 9, 20, 0, 0)], 
'8644': [datetime.datetime(1977, 9, 24, 18, 0), datetime.datetime(1977, 9, 25, 0, 0)], 
'8729': [datetime.datetime(1978, 7, 29, 18, 0), datetime.datetime(1978, 7, 30, 0, 0)], 
'8744': [datetime.datetime(1978, 8, 27, 6, 0), datetime.datetime(1978, 8, 27, 12, 0)], 
'8850': [datetime.datetime(1979, 7, 6, 0, 0), datetime.datetime(1979, 7, 6, 6, 0)], 
'8861': [datetime.datetime(1979, 8, 2, 0, 0), datetime.datetime(1979, 8, 2, 6, 0)], 
'8863': [datetime.datetime(1979, 7, 29, 0, 0), datetime.datetime(1979, 7, 29, 6, 0)], 
'8888': [datetime.datetime(1979, 9, 23, 12, 0), datetime.datetime(1979, 9, 23, 18, 0)], 
'8958': [datetime.datetime(1980, 5, 23, 18, 0), datetime.datetime(1980, 5, 24, 0, 0)], 
'8965': [datetime.datetime(1980, 7, 11, 12, 0), datetime.datetime(1980, 7, 11, 18, 0)], 
'8967': [datetime.datetime(1980, 7, 22, 6, 0), datetime.datetime(1980, 7, 22, 12, 0)], 
'8968': [datetime.datetime(1980, 7, 19, 0, 0), datetime.datetime(1980, 7, 19, 6, 0)], 
'8972': [datetime.datetime(1980, 7, 27, 6, 0), datetime.datetime(1980, 7, 27, 12, 0)], 
'9075': [datetime.datetime(1981, 7, 6, 18, 0), datetime.datetime(1981, 7, 7, 0, 0)], 
'9112': [datetime.datetime(1981, 9, 21, 18, 0), datetime.datetime(1981, 9, 22, 0, 0)], 
'9194': [datetime.datetime(1982, 7, 17, 0, 0), datetime.datetime(1982, 7, 17, 6, 0)], 
'9217': [datetime.datetime(1982, 9, 14, 18, 0), datetime.datetime(1982, 9, 15, 0, 0)],
'9283': [datetime.datetime(1983, 7, 13, 12, 0), datetime.datetime(1983, 7, 13, 18, 0)],
'9304': [datetime.datetime(1983, 9, 9, 0, 0), datetime.datetime(1983, 9, 9, 6, 0)],
'9314': [datetime.datetime(1983, 9, 30, 0, 0), datetime.datetime(1983, 9, 30, 6, 0)],
'9319': [datetime.datetime(1983, 10, 13, 12, 0), datetime.datetime(1983, 10, 13, 18, 0)],
'9386': [datetime.datetime(1984, 6, 25, 6, 0), datetime.datetime(1984, 6, 25, 12, 0)],
'9391': [datetime.datetime(1984, 7, 9, 0, 0), datetime.datetime(1984, 7, 9, 6, 0)],
'9406': [datetime.datetime(1984, 8, 21, 0, 0), datetime.datetime(1984, 8, 21, 6, 0)],
'9509': [datetime.datetime(1985, 6, 24, 0, 0), datetime.datetime(1985, 6, 24, 6, 0)],
'9542': [datetime.datetime(1985, 9, 5, 18, 0), datetime.datetime(1985, 9, 6, 0, 0)],
'9554': [datetime.datetime(1985, 9, 21, 18, 0), datetime.datetime(1985, 9, 22, 0, 0)],
'9634': [datetime.datetime(1986, 7, 11, 6, 0), datetime.datetime(1986, 7, 11, 12, 0)],
'9728': [datetime.datetime(1987, 6, 19, 0, 0), datetime.datetime(1987, 6, 19, 6, 0)],
'9782': [datetime.datetime(1987, 10, 28, 0, 0), datetime.datetime(1987, 10, 28, 6, 0)],
'9830': [datetime.datetime(1988, 7, 19, 6, 0), datetime.datetime(1988, 7, 19, 12, 0)],
'9874': [datetime.datetime(1988, 9, 21, 18, 0), datetime.datetime(1988, 9, 22, 0, 0)],
'9944': [datetime.datetime(1989, 5, 20, 12, 0), datetime.datetime(1989, 5, 20, 18, 0)],
'9961': [datetime.datetime(1989, 7, 18, 0, 0), datetime.datetime(1989, 7, 18, 6, 0)],
'10069': [datetime.datetime(1990, 6, 17, 18, 0), datetime.datetime(1990, 6, 18, 0, 0)],
'10074': [datetime.datetime(1990, 6, 29, 6, 0), datetime.datetime(1990, 6, 29, 12, 0)],
'10083': [datetime.datetime(1990, 7, 30, 18, 0), datetime.datetime(1990, 7, 31, 0, 0)],
'10191': [datetime.datetime(1991, 7, 19, 6, 0), datetime.datetime(1991, 7, 19, 12, 0)],
'10192': [datetime.datetime(1991, 7, 24, 0, 0), datetime.datetime(1991, 7, 24, 6, 0)],
'10212': [datetime.datetime(1991, 9, 6, 12, 0), datetime.datetime(1991, 9, 6, 18, 0)],
'10302': [datetime.datetime(1992, 7, 18, 0, 0), datetime.datetime(1992, 7, 18, 6, 0)],
'10318': [datetime.datetime(1992, 8, 18, 18, 0), datetime.datetime(1992, 8, 19, 0, 0)],
'10417': [datetime.datetime(1993, 6, 27, 12, 0), datetime.datetime(1993, 6, 27, 18, 0)],
'10436': [datetime.datetime(1993, 8, 20, 18, 0), datetime.datetime(1993, 8, 21, 0, 0)],
'10449': [datetime.datetime(1993, 9, 13, 18, 0), datetime.datetime(1993, 9, 14, 0, 0)],
'10452': [datetime.datetime(1993, 9, 17, 0, 0), datetime.datetime(1993, 9, 17, 6, 0)],
'10454': [datetime.datetime(1993, 9, 26, 6, 0), datetime.datetime(1993, 9, 26, 12, 0)],
'10466': [datetime.datetime(1993, 11, 4, 12, 0), datetime.datetime(1993, 11, 4, 18, 0)],
'10518': [datetime.datetime(1994, 6, 8, 6, 0), datetime.datetime(1994, 6, 8, 12, 0)],
'10520': [datetime.datetime(1994, 6, 24, 18, 0), datetime.datetime(1994, 6, 25, 0, 0)],
'10524': [datetime.datetime(1994, 7, 4, 6, 0), datetime.datetime(1994, 7, 4, 12, 0)],
'10553': [datetime.datetime(1994, 8, 27, 12, 0), datetime.datetime(1994, 8, 27, 18, 0)],
'10636': [datetime.datetime(1995, 7, 31, 0, 0), datetime.datetime(1995, 7, 31, 6, 0)],
'10640': [datetime.datetime(1995, 8, 12, 0, 0), datetime.datetime(1995, 8, 12, 6, 0)],
'10645': [datetime.datetime(1995, 8, 19, 12, 0), datetime.datetime(1995, 8, 19, 18, 0)],
'10653': [datetime.datetime(1995, 8, 31, 0, 0), datetime.datetime(1995, 8, 31, 6, 0)],
'10657': [datetime.datetime(1995, 9, 7, 0, 0), datetime.datetime(1995, 9, 7, 6, 0)],
'10669': [datetime.datetime(1995, 10, 3, 0, 0), datetime.datetime(1995, 10, 3, 6, 0)],
'10675': [datetime.datetime(1995, 10, 14, 6, 0), datetime.datetime(1995, 10, 14, 12, 0)],
'10779': [datetime.datetime(1996, 9, 20, 0, 0), datetime.datetime(1996, 9, 20, 6, 0)],
'10881': [datetime.datetime(1997, 8, 2, 6, 0), datetime.datetime(1997, 8, 2, 12, 0)],
'10891': [datetime.datetime(1997, 8, 22, 0, 0), datetime.datetime(1997, 8, 22, 6, 0)],
'10980': [datetime.datetime(1998, 8, 10, 18, 0), datetime.datetime(1998, 8, 11, 0, 0)],
'10996': [datetime.datetime(1998, 9, 13, 6, 0), datetime.datetime(1998, 9, 13, 12, 0)],
'11074': [datetime.datetime(1999, 5, 2, 12, 0), datetime.datetime(1999, 5, 2, 18, 0)],
'11077': [datetime.datetime(1999, 6, 6, 12, 0), datetime.datetime(1999, 6, 6, 18, 0)],
'11092': [datetime.datetime(1999, 7, 27, 0, 0), datetime.datetime(1999, 7, 27, 6, 0)],
'11106': [datetime.datetime(1999, 8, 22, 6, 0), datetime.datetime(1999, 8, 22, 12, 0)],
'11115': [datetime.datetime(1999, 9, 3, 18, 0), datetime.datetime(1999, 9, 4, 0, 0)],
'11119': [datetime.datetime(1999, 9, 16, 6, 0), datetime.datetime(1999, 9, 16, 12, 0)],
'11190': [datetime.datetime(2000, 7, 17, 0, 0), datetime.datetime(2000, 7, 17, 6, 0)],
'11218': [datetime.datetime(2000, 8, 31, 18, 0), datetime.datetime(2000, 9, 1, 0, 0)],
'11290': [datetime.datetime(2001, 7, 5, 18, 0), datetime.datetime(2001, 7, 6, 0, 0)], 
'11298': [datetime.datetime(2001, 7, 25, 18, 0), datetime.datetime(2001, 7, 26, 0, 0)],
'11318': [datetime.datetime(2001, 9, 20, 0, 0), datetime.datetime(2001, 9, 20, 6, 0)],
'11410': [datetime.datetime(2002, 8, 4, 18, 0), datetime.datetime(2002, 8, 5, 0, 0)],
'11417': [datetime.datetime(2002, 8, 19, 6, 0), datetime.datetime(2002, 8, 19, 12, 0)],
'11431': [datetime.datetime(2002, 9, 11, 18, 0), datetime.datetime(2002, 9, 12, 0, 0)],
'11508': [datetime.datetime(2003, 7, 24, 0, 0), datetime.datetime(2003, 7, 24, 6, 0)],
'11520': [datetime.datetime(2003, 8, 24, 18, 0), datetime.datetime(2003, 8, 25, 0, 0)],
'11528': [datetime.datetime(2003, 9, 2, 6, 0), datetime.datetime(2003, 9, 2, 12, 0)],
'11605': [datetime.datetime(2004, 7, 16, 6, 0), datetime.datetime(2004, 7, 16, 12, 0)],
'11623': [datetime.datetime(2004, 8, 26, 0, 0), datetime.datetime(2004, 8, 26, 6, 0)],
'11725': [datetime.datetime(2005, 8, 13, 0, 0), datetime.datetime(2005, 8, 13, 6, 0)],
'11810': [datetime.datetime(2006, 5, 17, 12, 0), datetime.datetime(2006, 5, 17, 18, 0)],
'11814': [datetime.datetime(2006, 6, 28, 18, 0), datetime.datetime(2006, 6, 29, 0, 0)],
'11826': [datetime.datetime(2006, 8, 3, 6, 0), datetime.datetime(2006, 8, 3, 12, 0)],
'11842': [datetime.datetime(2006, 8, 24, 18, 0), datetime.datetime(2006, 8, 25, 0, 0)],
'11850': [datetime.datetime(2006, 9, 13, 6, 0), datetime.datetime(2006, 9, 13, 12, 0)],
'11999': [datetime.datetime(2008, 4, 19, 0, 0), datetime.datetime(2008, 4, 19, 6, 0)],
'12012': [datetime.datetime(2008, 6, 24, 18, 0), datetime.datetime(2008, 6, 25, 0, 0)],
'12025': [datetime.datetime(2008, 8, 6, 6, 0), datetime.datetime(2008, 8, 6, 12, 0)],
'12034': [datetime.datetime(2008, 8, 22, 6, 0), datetime.datetime(2008, 8, 22, 12, 0)],
'12048': [datetime.datetime(2008, 9, 23, 18, 0), datetime.datetime(2008, 9, 24, 0, 0)],
'12052': [datetime.datetime(2008, 10, 4, 6, 0), datetime.datetime(2008, 10, 4, 12, 0)],
'12110': [datetime.datetime(2009, 6, 26, 12, 0), datetime.datetime(2009, 6, 26, 18, 0)],
'12114': [datetime.datetime(2009, 7, 11, 18, 0), datetime.datetime(2009, 7, 12, 0, 0)],
'12118': [datetime.datetime(2009, 7, 18, 12, 0), datetime.datetime(2009, 7, 18, 18, 0)],
'12121': [datetime.datetime(2009, 8, 5, 0, 0), datetime.datetime(2009, 8, 5, 6, 0)],
'12146': [datetime.datetime(2009, 9, 14, 18, 0), datetime.datetime(2009, 9, 15, 0, 0)],
'12215': [datetime.datetime(2010, 7, 22, 0, 0), datetime.datetime(2010, 7, 22, 6, 0)],
'12238': [datetime.datetime(2010, 9, 20, 0, 0), datetime.datetime(2010, 9, 20, 6, 0)],
'12245': [datetime.datetime(2010, 10, 5, 12, 0), datetime.datetime(2010, 10, 5, 18, 0)],
'12294': [datetime.datetime(2011, 6, 10, 18, 0), datetime.datetime(2011, 6, 11, 0, 0)],
'12296': [datetime.datetime(2011, 6, 23, 6, 0), datetime.datetime(2011, 6, 23, 12, 0)],
'12393': [datetime.datetime(2012, 6, 29, 18, 0), datetime.datetime(2012, 6, 30, 0, 0)],
'12399': [datetime.datetime(2012, 7, 23, 18, 0), datetime.datetime(2012, 7, 24, 0, 0)],
'12408': [datetime.datetime(2012, 8, 17, 0, 0), datetime.datetime(2012, 8, 17, 6, 0)],
'12485': [datetime.datetime(2013, 7, 1, 18, 0), datetime.datetime(2013, 7, 2, 0, 0)],
'12498': [datetime.datetime(2013, 8, 14, 6, 0), datetime.datetime(2013, 8, 14, 12, 0)],
'12520': [datetime.datetime(2013, 9, 22, 6, 0), datetime.datetime(2013, 9, 22, 12, 0)],
'12594': [datetime.datetime(2014, 6, 15, 6, 0), datetime.datetime(2014, 6, 15, 12, 0)],
'12600': [datetime.datetime(2014, 7, 18, 6, 0), datetime.datetime(2014, 7, 18, 12, 0)],
'12621': [datetime.datetime(2014, 9, 8, 0, 0), datetime.datetime(2014, 9, 8, 6, 0)],
'12624': [datetime.datetime(2014, 9, 16, 0, 0), datetime.datetime(2014, 9, 16, 6, 0)],
'12688': [datetime.datetime(2015, 7, 9, 0, 0), datetime.datetime(2015, 7, 9, 6, 0)],
'12733': [datetime.datetime(2015, 10, 4, 0, 0), datetime.datetime(2015, 10, 4, 6, 0)],
'12776': [datetime.datetime(2016, 5, 27, 6, 0), datetime.datetime(2016, 5, 27, 12, 0)],
'12793': [datetime.datetime(2016, 8, 1, 18, 0), datetime.datetime(2016, 8, 2, 0, 0)],
'12837': [datetime.datetime(2016, 10, 21, 0, 0), datetime.datetime(2016, 10, 21, 6, 0)]}'''

tic = time.time()
#landfallTC_dates_PHI_2014 = getdates_landfall(0, after2014 = True)
landfallTC_dates_PHI = getdates_landfall(0, after2014 = False)
print(landfallTC_dates_PHI)
toc = time.time()
print(toc - tic)

'''For PHI:
{'12566': [datetime.datetime(2014, 1, 31, 12, 0), datetime.datetime(2014, 1, 31, 18, 0)],
'12580': [datetime.datetime(2014, 3, 22, 6, 0), datetime.datetime(2014, 3, 22, 12, 0)],
'12600': [datetime.datetime(2014, 7, 15, 6, 0), datetime.datetime(2014, 7, 15, 12, 0)],
'12624': [datetime.datetime(2014, 9, 14, 6, 0), datetime.datetime(2014, 9, 14, 12, 0)],
'12627': [datetime.datetime(2014, 9, 19, 0, 0), datetime.datetime(2014, 9, 19, 6, 0)],
'12645': [datetime.datetime(2014, 11, 26, 6, 0), datetime.datetime(2014, 11, 26, 12, 0)],
'12646': [datetime.datetime(2014, 12, 6, 12, 0), datetime.datetime(2014, 12, 6, 18, 0)],
'12649': [datetime.datetime(2014, 12, 28, 18, 0), datetime.datetime(2014, 12, 29, 0, 0)],
'12651': [datetime.datetime(2015, 1, 17, 6, 0), datetime.datetime(2015, 1, 17, 12, 0)],
'12669': [datetime.datetime(2015, 4, 5, 0, 0), datetime.datetime(2015, 4, 5, 6, 0)],
'12688': [datetime.datetime(2015, 7, 4, 18, 0), datetime.datetime(2015, 7, 5, 0, 0)],
'12733': [datetime.datetime(2015, 10, 1, 12, 0), datetime.datetime(2015, 10, 1, 18, 0)],
'12738': [datetime.datetime(2015, 10, 17, 18, 0), datetime.datetime(2015, 10, 18, 0, 0)],
'12754': [datetime.datetime(2015, 12, 14, 6, 0), datetime.datetime(2015, 12, 14, 12, 0)],
'12836': [datetime.datetime(2016, 10, 15, 18, 0), datetime.datetime(2016, 10, 16, 0, 0)],
'12837': [datetime.datetime(2016, 10, 19, 12, 0), datetime.datetime(2016, 10, 19, 18, 0)],
'12846': [datetime.datetime(2016, 11, 24, 12, 0), datetime.datetime(2016, 11, 24, 18, 0)],
'12852': [datetime.datetime(2016, 12, 25, 12, 0), datetime.datetime(2016, 12, 25, 18, 0)]
'12862': [datetime.datetime(2018, 9, 14, 16, 0), datetime.datetime(2016, 9, 14, 22, 0)}'''

'''For PHI: {
'4883': [datetime.datetime(1945, 8, 5, 18, 0), datetime.datetime(1945, 8, 6, 0, 0)], 
'4903': [datetime.datetime(1945, 9, 21, 6, 0)], 
'4912': [datetime.datetime(1945, 11, 4, 0, 0), datetime.datetime(1945, 11, 4, 6, 0)], 
'4953': [datetime.datetime(1946, 4, 4, 0, 0), datetime.datetime(1946, 4, 4, 6, 0)],
'4968': [datetime.datetime(1946, 7, 16, 6, 0), datetime.datetime(1946, 7, 16, 12, 0)],
'4981': [datetime.datetime(1946, 9, 10, 12, 0), datetime.datetime(1946, 9, 10, 18, 0)],
'5039': [datetime.datetime(1947, 3, 19, 18, 0), datetime.datetime(1947, 3, 20, 0, 0)],
'5081': [datetime.datetime(1947, 9, 23, 0, 0), datetime.datetime(1947, 9, 23, 6, 0)],
'5087': [datetime.datetime(1947, 10, 5, 12, 0), datetime.datetime(1947, 10, 5, 18, 0)],
'5099': [datetime.datetime(1947, 10, 30, 0, 0), datetime.datetime(1947, 10, 30, 6, 0)],
'5103': [datetime.datetime(1947, 11, 7, 12, 0), datetime.datetime(1947, 11, 7, 18, 0)],
'5105': [datetime.datetime(1947, 11, 14, 18, 0), datetime.datetime(1947, 11, 15, 0, 0)],
'5110': [datetime.datetime(1947, 11, 30, 0, 0)],
'5114': [datetime.datetime(1947, 12, 25, 12, 0), datetime.datetime(1947, 12, 25, 18, 0)],
'5164': [datetime.datetime(1948, 7, 24, 18, 0), datetime.datetime(1948, 7, 25, 0, 0)],
'5173': [datetime.datetime(1948, 8, 31, 12, 0), datetime.datetime(1948, 8, 31, 18, 0)],
'5183': [datetime.datetime(1948, 9, 24, 12, 0), datetime.datetime(1948, 9, 24, 18, 0)],
'5205': [datetime.datetime(1948, 11, 29, 6, 0), datetime.datetime(1948, 11, 29, 12, 0)],
'5207': [datetime.datetime(1948, 12, 8, 6, 0), datetime.datetime(1948, 12, 8, 12, 0)],
'5210': [datetime.datetime(1948, 12, 14, 0, 0), datetime.datetime(1948, 12, 14, 6, 0)],
'5215': [datetime.datetime(1949, 1, 20, 12, 0), datetime.datetime(1949, 1, 20, 18, 0)],
'5252': [datetime.datetime(1949, 7, 6, 6, 0), datetime.datetime(1949, 7, 6, 12, 0)],
'5309': [datetime.datetime(1949, 11, 1, 6, 0), datetime.datetime(1949, 11, 1, 12, 0)],
'5313': [datetime.datetime(1949, 11, 11, 0, 0), datetime.datetime(1949, 11, 11, 6, 0)],
'5315': [datetime.datetime(1949, 12, 4, 0, 0), datetime.datetime(1949, 12, 4, 6, 0)],
'5316': [datetime.datetime(1949, 12, 11, 12, 0), datetime.datetime(1949, 12, 11, 18, 0)],
'5401': [datetime.datetime(1950, 10, 1, 12, 0), datetime.datetime(1950, 10, 1, 18, 0)],
'5427': [datetime.datetime(1950, 11, 20, 0, 0), datetime.datetime(1950, 11, 20, 6, 0)],
'5434': [datetime.datetime(1950, 12, 29, 12, 0), datetime.datetime(1950, 12, 29, 18, 0)],
'5459': [datetime.datetime(1951, 5, 4, 12, 0), datetime.datetime(1951, 5, 4, 18, 0)],
'5481': [datetime.datetime(1951, 7, 29, 18, 0), datetime.datetime(1951, 7, 30, 0, 0)],
'5495': [datetime.datetime(1951, 9, 1, 0, 0), datetime.datetime(1951, 9, 1, 6, 0)],
'5506': [datetime.datetime(1951, 9, 24, 0, 0), datetime.datetime(1951, 9, 24, 6, 0)],
'5521': [datetime.datetime(1951, 11, 20, 6, 0), datetime.datetime(1951, 11, 20, 12, 0)],
'5527': [datetime.datetime(1951, 12, 9, 6, 0), datetime.datetime(1951, 12, 9, 12, 0)],
'5568': [datetime.datetime(1952, 7, 2, 0, 0), datetime.datetime(1952, 7, 2, 6, 0)],
'5588': [datetime.datetime(1952, 8, 25, 6, 0), datetime.datetime(1952, 8, 25, 12, 0)],
'5593': [datetime.datetime(1952, 9, 4, 0, 0), datetime.datetime(1952, 9, 4, 6, 0)],
'5609': [datetime.datetime(1952, 10, 21, 12, 0), datetime.datetime(1952, 10, 21, 18, 0)],
'5617': [datetime.datetime(1952, 10, 27, 0, 0), datetime.datetime(1952, 10, 27, 6, 0)],
'5624': [datetime.datetime(1952, 11, 24, 18, 0), datetime.datetime(1952, 11, 25, 0, 0)],
'5636': [datetime.datetime(1952, 12, 20, 12, 0), datetime.datetime(1952, 12, 20, 18, 0)],
'5664': [datetime.datetime(1953, 6, 3, 18, 0), datetime.datetime(1953, 6, 4, 0, 0)],
'5667': [datetime.datetime(1953, 6, 25, 12, 0), datetime.datetime(1953, 6, 25, 18, 0)],
'5679': [datetime.datetime(1953, 8, 11, 18, 0), datetime.datetime(1953, 8, 12, 0, 0)],
'5712': [datetime.datetime(1953, 10, 27, 6, 0), datetime.datetime(1953, 10, 27, 12, 0)],
'5715': [datetime.datetime(1953, 11, 16, 12, 0), datetime.datetime(1953, 11, 16, 18, 0)],
'5735': [datetime.datetime(1954, 3, 3, 12, 0), datetime.datetime(1954, 3, 3, 18, 0)],
'5743': [datetime.datetime(1954, 5, 6, 12, 0), datetime.datetime(1954, 5, 6, 18, 0)],
'5800': [datetime.datetime(1954, 10, 7, 18, 0), datetime.datetime(1954, 10, 8, 0, 0)],
'5813': [datetime.datetime(1954, 11, 8, 0, 0), datetime.datetime(1954, 11, 8, 6, 0)],
'5818': [datetime.datetime(1954, 11, 29, 6, 0), datetime.datetime(1954, 11, 29, 12, 0)],
'5826': [datetime.datetime(1955, 1, 4, 12, 0), datetime.datetime(1955, 1, 4, 18, 0)],
'5892': [datetime.datetime(1955, 9, 23, 0, 0), datetime.datetime(1955, 9, 23, 6, 0)],
'5915': [datetime.datetime(1955, 11, 29, 6, 0), datetime.datetime(1955, 11, 29, 12, 0)],
'5958': [datetime.datetime(1956, 4, 9, 0, 0)],
'5959': [datetime.datetime(1956, 4, 21, 6, 0), datetime.datetime(1956, 4, 21, 12, 0)],
'5979': [datetime.datetime(1956, 7, 5, 6, 0), datetime.datetime(1956, 7, 5, 12, 0)],
'5991': [datetime.datetime(1956, 8, 10, 0, 0), datetime.datetime(1956, 8, 10, 6, 0)],
'6000': [datetime.datetime(1956, 8, 28, 12, 0), datetime.datetime(1956, 8, 28, 18, 0)],
'6014': [datetime.datetime(1956, 9, 18, 6, 0), datetime.datetime(1956, 9, 18, 12, 0)],
'6021': [datetime.datetime(1956, 10, 20, 18, 0), datetime.datetime(1956, 10, 21, 0, 0)],
'6028': [datetime.datetime(1956, 11, 15, 6, 0), datetime.datetime(1956, 11, 15, 12, 0)],
'6037': [datetime.datetime(1956, 11, 28, 18, 0), datetime.datetime(1956, 11, 29, 0, 0)],
'6038': [datetime.datetime(1956, 12, 8, 18, 0), datetime.datetime(1956, 12, 9, 0, 0)],
'6044': [datetime.datetime(1957, 1, 6, 0, 0), datetime.datetime(1957, 1, 6, 6, 0)],
'6078': [datetime.datetime(1957, 7, 14, 12, 0), datetime.datetime(1957, 7, 14, 18, 0)],
'6102': [datetime.datetime(1957, 9, 20, 6, 0), datetime.datetime(1957, 9, 20, 12, 0)],
'6111': [datetime.datetime(1957, 10, 12, 6, 0), datetime.datetime(1957, 10, 12, 12, 0)],
'6118': [datetime.datetime(1957, 11, 11, 18, 0), datetime.datetime(1957, 11, 12, 0, 0)],
'6229': [datetime.datetime(1958, 10, 20, 18, 0), datetime.datetime(1958, 10, 21, 0, 0)],
'6318': [datetime.datetime(1959, 9, 7, 18, 0), datetime.datetime(1959, 9, 8, 0, 0)],
'6334': [datetime.datetime(1959, 10, 6, 0, 0), datetime.datetime(1959, 10, 6, 6, 0)],
'6347': [datetime.datetime(1959, 11, 16, 0, 0), datetime.datetime(1959, 11, 16, 6, 0)], 
'6352': [datetime.datetime(1959, 12, 18, 0, 0), datetime.datetime(1959, 12, 18, 6, 0)], 
'6354': [datetime.datetime(1959, 12, 30, 18, 0), datetime.datetime(1959, 12, 31, 0, 0)], 
'6374': [datetime.datetime(1960, 4, 21, 18, 0), datetime.datetime(1960, 4, 22, 0, 0)], 
'6377': [datetime.datetime(1960, 5, 27, 6, 0), datetime.datetime(1960, 5, 27, 12, 0)], 
'6385': [datetime.datetime(1960, 6, 25, 18, 0), datetime.datetime(1960, 6, 26, 0, 0)], 
'6419': [datetime.datetime(1960, 9, 1, 12, 0), datetime.datetime(1960, 9, 1, 18, 0)], 
'6432': [datetime.datetime(1960, 10, 6, 6, 0), datetime.datetime(1960, 10, 6, 12, 0)], 
'6435': [datetime.datetime(1960, 10, 13, 6, 0), datetime.datetime(1960, 10, 13, 12, 0)], 
'6515': [datetime.datetime(1961, 7, 17, 6, 0), datetime.datetime(1961, 7, 17, 12, 0)], 
'6619': [datetime.datetime(1962, 5, 16, 12, 0), datetime.datetime(1962, 5, 16, 18, 0)], 
'6640': [datetime.datetime(1962, 8, 7, 18, 0), datetime.datetime(1962, 8, 8, 0, 0)], 
'6665': [datetime.datetime(1962, 9, 23, 18, 0), datetime.datetime(1962, 9, 24, 0, 0)], 
'6681': [datetime.datetime(1962, 11, 6, 0, 0), datetime.datetime(1962, 11, 6, 6, 0)], 
'6684': [datetime.datetime(1962, 11, 27, 0, 0), datetime.datetime(1962, 11, 27, 6, 0)], 
'6748': [datetime.datetime(1963, 6, 27, 18, 0), datetime.datetime(1963, 6, 28, 0, 0)], 
'6757': [datetime.datetime(1963, 7, 20, 6, 0), datetime.datetime(1963, 7, 20, 12, 0)], 
'6765': [datetime.datetime(1963, 8, 12, 18, 0), datetime.datetime(1963, 8, 13, 0, 0)], 
'6857': [datetime.datetime(1964, 6, 29, 12, 0), datetime.datetime(1964, 6, 29, 18, 0)], 
'6864': [datetime.datetime(1964, 7, 17, 18, 0), datetime.datetime(1964, 7, 18, 0, 0)], 
'6872': [datetime.datetime(1964, 8, 6, 18, 0), datetime.datetime(1964, 8, 7, 0, 0)], 
'6907': [datetime.datetime(1964, 9, 28, 18, 0), datetime.datetime(1964, 9, 29, 0, 0)], 
'6910': [datetime.datetime(1964, 10, 4, 18, 0), datetime.datetime(1964, 10, 5, 0, 0)], 
'6911': [datetime.datetime(1964, 10, 9, 0, 0), datetime.datetime(1964, 10, 9, 6, 0)], 
'6917': [datetime.datetime(1964, 10, 21, 0, 0), datetime.datetime(1964, 10, 21, 6, 0)], 
'6928': [datetime.datetime(1964, 11, 19, 0, 0), datetime.datetime(1964, 11, 19, 6, 0)], 
'6932': [datetime.datetime(1964, 11, 26, 18, 0), datetime.datetime(1964, 11, 27, 0, 0)], 
'6937': [datetime.datetime(1964, 12, 14, 12, 0), datetime.datetime(1964, 12, 14, 18, 0)], 
'6954': [datetime.datetime(1965, 1, 23, 0, 0), datetime.datetime(1965, 1, 23, 6, 0)], 
'6977': [datetime.datetime(1965, 3, 7, 0, 0), datetime.datetime(1965, 3, 7, 6, 0)], 
'7000': [datetime.datetime(1965, 7, 13, 6, 0), datetime.datetime(1965, 7, 13, 12, 0)], 
'7003': [datetime.datetime(1965, 7, 19, 0, 0), datetime.datetime(1965, 7, 19, 6, 0)], 
'7037': [datetime.datetime(1965, 9, 14, 0, 0), datetime.datetime(1965, 9, 14, 6, 0)], 
'7094': [datetime.datetime(1966, 5, 15, 12, 0), datetime.datetime(1966, 5, 15, 18, 0)], 
'7168': [datetime.datetime(1966, 10, 31, 6, 0), datetime.datetime(1966, 10, 31, 12, 0)], 
'7177': [datetime.datetime(1966, 11, 19, 12, 0), datetime.datetime(1966, 11, 19, 18, 0)],
'7186': [datetime.datetime(1966, 12, 18, 18, 0), datetime.datetime(1966, 12, 19, 0, 0)], 
'7190': [datetime.datetime(1966, 12, 26, 12, 0), datetime.datetime(1966, 12, 26, 18, 0)], 
'7210': [datetime.datetime(1967, 3, 3, 0, 0), datetime.datetime(1967, 3, 3, 6, 0)], 
'7219': [datetime.datetime(1967, 4, 8, 0, 0), datetime.datetime(1967, 4, 8, 6, 0)], 
'7264': [datetime.datetime(1967, 8, 19, 0, 0), datetime.datetime(1967, 8, 19, 6, 0)],
'7311': [datetime.datetime(1967, 10, 16, 18, 0), datetime.datetime(1967, 10, 17, 0, 0)],
'7321': [datetime.datetime(1967, 11, 3, 12, 0), datetime.datetime(1967, 11, 3, 18, 0)], 
'7324': [datetime.datetime(1967, 11, 7, 0, 0), datetime.datetime(1967, 11, 7, 6, 0)], 
'7398': [datetime.datetime(1968, 8, 10, 0, 0), datetime.datetime(1968, 8, 10, 6, 0)], 
'7405': [datetime.datetime(1968, 8, 19, 0, 0), datetime.datetime(1968, 8, 19, 6, 0)], 
'7433': [datetime.datetime(1968, 9, 28, 6, 0), datetime.datetime(1968, 9, 28, 12, 0)], 
'7457': [datetime.datetime(1968, 11, 18, 12, 0), datetime.datetime(1968, 11, 18, 18, 0)], 
'7459': [datetime.datetime(1968, 11, 23, 12, 0), datetime.datetime(1968, 11, 23, 18, 0)], 
'7460': [datetime.datetime(1968, 11, 29, 6, 0), datetime.datetime(1968, 11, 29, 12, 0)], 
'7498': [datetime.datetime(1969, 4, 23, 12, 0), datetime.datetime(1969, 4, 23, 18, 0)], 
'7510': [datetime.datetime(1969, 7, 8, 12, 0), datetime.datetime(1969, 7, 8, 18, 0)], 
'7636': [datetime.datetime(1970, 6, 29, 18, 0), datetime.datetime(1970, 6, 30, 0, 0)], 
'7643': [datetime.datetime(1970, 7, 13, 18, 0), datetime.datetime(1970, 7, 14, 0, 0)], 
'7662': [datetime.datetime(1970, 8, 6, 18, 0), datetime.datetime(1970, 8, 7, 0, 0)], 
'7691': [datetime.datetime(1970, 9, 11, 0, 0), datetime.datetime(1970, 9, 11, 6, 0)], 
'7712': [datetime.datetime(1970, 10, 13, 0, 0), datetime.datetime(1970, 10, 13, 6, 0)], 
'7714': [datetime.datetime(1970, 10, 18, 6, 0), datetime.datetime(1970, 10, 18, 12, 0)], 
'7721': [datetime.datetime(1970, 10, 31, 18, 0), datetime.datetime(1970, 11, 1, 0, 0)], 
'7727': [datetime.datetime(1970, 11, 18, 18, 0), datetime.datetime(1970, 11, 19, 0, 0)], 
'7775': [datetime.datetime(1971, 4, 24, 0, 0), datetime.datetime(1971, 4, 24, 6, 0)], 
'7781': [datetime.datetime(1971, 5, 25, 18, 0), datetime.datetime(1971, 5, 26, 0, 0)], 
'7786': [datetime.datetime(1971, 6, 15, 6, 0), datetime.datetime(1971, 6, 15, 12, 0)], 
'7791': [datetime.datetime(1971, 6, 24, 18, 0), datetime.datetime(1971, 6, 25, 0, 0)], 
'7792': [datetime.datetime(1971, 7, 2, 18, 0), datetime.datetime(1971, 7, 3, 0, 0)], 
'7800': [datetime.datetime(1971, 7, 14, 0, 0), datetime.datetime(1971, 7, 14, 6, 0)], 
'7816': [datetime.datetime(1971, 8, 13, 12, 0), datetime.datetime(1971, 8, 13, 18, 0)], 
'7854': [datetime.datetime(1971, 9, 26, 18, 0), datetime.datetime(1971, 9, 27, 0, 0)], 
'7859': [datetime.datetime(1971, 10, 3, 6, 0), datetime.datetime(1971, 10, 3, 12, 0)], 
'7861': [datetime.datetime(1971, 10, 10, 12, 0), datetime.datetime(1971, 10, 10, 18, 0)], 
'7866': [datetime.datetime(1971, 10, 9, 18, 0), datetime.datetime(1971, 10, 10, 0, 0)], 
'7868': [datetime.datetime(1971, 10, 20, 0, 0), datetime.datetime(1971, 10, 20, 6, 0)], 
'7896': [datetime.datetime(1972, 1, 7, 12, 0), datetime.datetime(1972, 1, 7, 18, 0)], 
'7935': [datetime.datetime(1972, 6, 24, 12, 0), datetime.datetime(1972, 6, 24, 18, 0)], 
'7940': [datetime.datetime(1972, 7, 5, 0, 0), datetime.datetime(1972, 7, 5, 6, 0)], 
'7972': [datetime.datetime(1972, 8, 30, 12, 0)], 
'7976': [datetime.datetime(1972, 9, 9, 18, 0), datetime.datetime(1972, 9, 10, 0, 0)], 
'8003': [datetime.datetime(1972, 11, 4, 18, 0), datetime.datetime(1972, 11, 5, 0, 0)], 
'8014': [datetime.datetime(1972, 12, 3, 0, 0), datetime.datetime(1972, 12, 3, 6, 0)], 
'8075': [datetime.datetime(1973, 6, 29, 18, 0), datetime.datetime(1973, 6, 30, 0, 0)], 
'8114': [datetime.datetime(1973, 9, 11, 12, 0), datetime.datetime(1973, 9, 11, 18, 0)], 
'8131': [datetime.datetime(1973, 10, 15, 12, 0), datetime.datetime(1973, 10, 15, 18, 0)], 
'8142': [datetime.datetime(1973, 11, 19, 12, 0), datetime.datetime(1973, 11, 19, 18, 0)], 
'8194': [datetime.datetime(1974, 6, 10, 0, 0), datetime.datetime(1974, 6, 10, 6, 0)], 
'8213': [datetime.datetime(1974, 7, 20, 0, 0), datetime.datetime(1974, 7, 20, 6, 0)], 
'8267': [datetime.datetime(1974, 10, 10, 18, 0), datetime.datetime(1974, 10, 11, 0, 0)], 
'8269': [datetime.datetime(1974, 10, 16, 6, 0), datetime.datetime(1974, 10, 16, 12, 0)], 
'8275': [datetime.datetime(1974, 10, 27, 18, 0), datetime.datetime(1974, 10, 28, 0, 0)], 
'8278': [datetime.datetime(1974, 11, 1, 0, 0), datetime.datetime(1974, 11, 1, 6, 0)], '8280': [datetime.datetime(1974, 11, 6, 18, 0), datetime.datetime(1974, 11, 7, 0, 0)], '8282': [datetime.datetime(1974, 11, 12, 18, 0), datetime.datetime(1974, 11, 13, 0, 0)], '8284': [datetime.datetime(1974, 11, 28, 0, 0), datetime.datetime(1974, 11, 28, 6, 0)], '8292': [datetime.datetime(1974, 12, 20, 6, 0), datetime.datetime(1974, 12, 20, 12, 0)], '8305': [datetime.datetime(1975, 1, 24, 6, 0), datetime.datetime(1975, 1, 24, 12, 0)], '8330': [datetime.datetime(1975, 4, 21, 18, 0), datetime.datetime(1975, 4, 22, 0, 0)], '8389': [datetime.datetime(1975, 9, 17, 18, 0), datetime.datetime(1975, 9, 18, 0, 0)], '8415': [datetime.datetime(1975, 11, 1, 12, 0), datetime.datetime(1975, 11, 1, 18, 0)], '8435': [datetime.datetime(1975, 12, 27, 0, 0), datetime.datetime(1975, 12, 27, 6, 0)], '8477': [datetime.datetime(1976, 5, 20, 18, 0), datetime.datetime(1976, 5, 21, 0, 0)], '8487': [datetime.datetime(1976, 6, 25, 0, 0), datetime.datetime(1976, 6, 25, 6, 0)], '8520': [datetime.datetime(1976, 8, 22, 6, 0), datetime.datetime(1976, 8, 22, 12, 0)], '8561': [datetime.datetime(1976, 12, 4, 12, 0), datetime.datetime(1976, 12, 4, 18, 0)], '8612': [datetime.datetime(1977, 7, 16, 18, 0), datetime.datetime(1977, 7, 17, 0, 0)], '8639': [datetime.datetime(1977, 9, 15, 12, 0), datetime.datetime(1977, 9, 15, 18, 0)], '8664': [datetime.datetime(1977, 11, 13, 12, 0), datetime.datetime(1977, 11, 13, 18, 0)], '8674': [datetime.datetime(1978, 1, 1, 18, 0), datetime.datetime(1978, 1, 2, 0, 0)], '8708': [datetime.datetime(1978, 4, 19, 18, 0), datetime.datetime(1978, 4, 20, 0, 0)], '8717': [datetime.datetime(1978, 6, 26, 18, 0), datetime.datetime(1978, 6, 27, 0, 0)], '8744': [datetime.datetime(1978, 8, 23, 12, 0), datetime.datetime(1978, 8, 23, 18, 0)], '8771': [datetime.datetime(1978, 9, 26, 12, 0), datetime.datetime(1978, 9, 26, 18, 0)], '8777': [datetime.datetime(1978, 10, 9, 12, 0), datetime.datetime(1978, 10, 9, 18, 0)], '8785': [datetime.datetime(1978, 10, 26, 12, 0), datetime.datetime(1978, 10, 26, 18, 0)], '8798': [datetime.datetime(1978, 11, 19, 18, 0), datetime.datetime(1978, 11, 20, 0, 0)], '8835': [datetime.datetime(1979, 4, 15, 6, 0), datetime.datetime(1979, 4, 15, 12, 0)], '8839': [datetime.datetime(1979, 5, 10, 18, 0), datetime.datetime(1979, 5, 11, 0, 0)], '8888': [datetime.datetime(1979, 9, 17, 18, 0), datetime.datetime(1979, 9, 18, 0, 0)], '8897': [datetime.datetime(1979, 10, 1, 6, 0), datetime.datetime(1979, 10, 1, 12, 0)], '8906': [datetime.datetime(1979, 11, 6, 6, 0), datetime.datetime(1979, 11, 6, 12, 0)], '8908': [datetime.datetime(1979, 11, 13, 12, 0), datetime.datetime(1979, 11, 13, 18, 0)], '8918': [datetime.datetime(1979, 12, 21, 6, 0), datetime.datetime(1979, 12, 21, 12, 0)], '8945': [datetime.datetime(1980, 3, 23, 12, 0), datetime.datetime(1980, 3, 23, 18, 0)], '8957': [datetime.datetime(1980, 5, 24, 6, 0), datetime.datetime(1980, 5, 24, 12, 0)], '8961': [datetime.datetime(1980, 6, 23, 0, 0), datetime.datetime(1980, 6, 23, 6, 0)], '8967': [datetime.datetime(1980, 7, 20, 12, 0), datetime.datetime(1980, 7, 20, 18, 0)], '8968': [datetime.datetime(1980, 7, 15, 18, 0), datetime.datetime(1980, 7, 16, 0, 0)], '8972': [datetime.datetime(1980, 7, 25, 0, 0), datetime.datetime(1980, 7, 25, 6, 0)], '8980': [datetime.datetime(1980, 8, 15, 0, 0), datetime.datetime(1980, 8, 15, 6, 0)], '9011': [datetime.datetime(1980, 11, 4, 12, 0), datetime.datetime(1980, 11, 4, 18, 0)], '9012': [datetime.datetime(1980, 10, 28, 18, 0), datetime.datetime(1980, 10, 29, 0, 0)], '9074': [datetime.datetime(1981, 6, 30, 6, 0), datetime.datetime(1981, 6, 30, 12, 0)], '9075': [datetime.datetime(1981, 7, 3, 18, 0), datetime.datetime(1981, 7, 4, 0, 0)], '9124': [datetime.datetime(1981, 10, 11, 18, 0), datetime.datetime(1981, 10, 12, 0, 0)], '9134': [datetime.datetime(1981, 11, 19, 12, 0), datetime.datetime(1981, 11, 19, 18, 0)], '9136': [datetime.datetime(1981, 11, 24, 6, 0), datetime.datetime(1981, 11, 24, 12, 0)], '9148': [datetime.datetime(1981, 12, 25, 6, 0), datetime.datetime(1981, 12, 25, 12, 0)], '9168': [datetime.datetime(1982, 3, 19, 6, 0), datetime.datetime(1982, 3, 19, 12, 0)], '9171': [datetime.datetime(1982, 3, 25, 12, 0), datetime.datetime(1982, 3, 25, 18, 0)], '9194': [datetime.datetime(1982, 7, 15, 0, 0), datetime.datetime(1982, 7, 15, 6, 0)], '9209': [datetime.datetime(1982, 8, 19, 18, 0), datetime.datetime(1982, 8, 20, 0, 0)], '9217': [datetime.datetime(1982, 9, 8, 6, 0), datetime.datetime(1982, 9, 8, 12, 0)], '9234': [datetime.datetime(1982, 10, 14, 6, 0), datetime.datetime(1982, 10, 14, 12, 0)], '9247': [datetime.datetime(1982, 12, 7, 6, 0), datetime.datetime(1982, 12, 7, 12, 0)], '9249': [datetime.datetime(1982, 12, 8, 0, 0), datetime.datetime(1982, 12, 8, 6, 0)], '9278': [datetime.datetime(1983, 6, 21, 12, 0), datetime.datetime(1983, 6, 21, 18, 0)], '9281': [datetime.datetime(1983, 7, 14, 6, 0), datetime.datetime(1983, 7, 14, 12, 0)], '9283': [datetime.datetime(1983, 7, 9, 12, 0), datetime.datetime(1983, 7, 9, 18, 0)], '9319': [datetime.datetime(1983, 10, 10, 18, 0), datetime.datetime(1983, 10, 11, 0, 0)], '9336': [datetime.datetime(1983, 11, 24, 6, 0), datetime.datetime(1983, 11, 24, 12, 0)], '9391': [datetime.datetime(1984, 7, 5, 18, 0), datetime.datetime(1984, 7, 6, 0, 0)], '9415': [datetime.datetime(1984, 8, 28, 12, 0), datetime.datetime(1984, 8, 28, 18, 0)], '9416': [datetime.datetime(1984, 9, 1, 12, 0), datetime.datetime(1984, 9, 1, 18, 0)], '9450': [datetime.datetime(1984, 11, 4, 18, 0), datetime.datetime(1984, 11, 5, 0, 0)], '9542': [datetime.datetime(1985, 9, 3, 0, 0), datetime.datetime(1985, 9, 3, 6, 0)], '9564': [datetime.datetime(1985, 10, 12, 0, 0), datetime.datetime(1985, 10, 12, 6, 0)], '9567': [datetime.datetime(1985, 10, 18, 12, 0), datetime.datetime(1985, 10, 18, 18, 0)], '9570': [datetime.datetime(1985, 10, 23, 6, 0), datetime.datetime(1985, 10, 23, 12, 0)], '9634': [datetime.datetime(1986, 7, 8, 18, 0), datetime.datetime(1986, 7, 9, 0, 0)], '9673': [datetime.datetime(1986, 10, 11, 0, 0), datetime.datetime(1986, 10, 11, 6, 0)], '9675': [datetime.datetime(1986, 10, 18, 12, 0), datetime.datetime(1986, 10, 18, 18, 0)], '9677': [datetime.datetime(1986, 11, 7, 12, 0), datetime.datetime(1986, 11, 7, 18, 0)], '9679': [datetime.datetime(1986, 11, 12, 6, 0), datetime.datetime(1986, 11, 12, 12, 0)], '9689': [datetime.datetime(1986, 12, 21, 6, 0), datetime.datetime(1986, 12, 21, 12, 0)], '9692': [datetime.datetime(1986, 12, 31, 6, 0), datetime.datetime(1986, 12, 31, 12, 0)], '9744': [datetime.datetime(1987, 8, 12, 6, 0), datetime.datetime(1987, 8, 12, 12, 0)], '9745': [datetime.datetime(1987, 8, 18, 0, 0), datetime.datetime(1987, 8, 18, 6, 0)], '9787': [datetime.datetime(1987, 11, 14, 12, 0), datetime.datetime(1987, 11, 14, 18, 0)], '9789': [datetime.datetime(1987, 11, 25, 12, 0), datetime.datetime(1987, 11, 25, 18, 0)], '9790': [datetime.datetime(1987, 11, 21, 0, 0), datetime.datetime(1987, 11, 21, 6, 0)], '9794': [datetime.datetime(1987, 12, 15, 18, 0), datetime.datetime(1987, 12, 16, 0, 0)], '9800': [datetime.datetime(1988, 1, 16, 0, 0), datetime.datetime(1988, 1, 16, 6, 0)], '9826': [datetime.datetime(1988, 6, 27, 6, 0), datetime.datetime(1988, 6, 27, 12, 0)], '9874': [datetime.datetime(1988, 9, 19, 12, 0), datetime.datetime(1988, 9, 19, 18, 0)], '9888': [datetime.datetime(1988, 10, 20, 12, 0), datetime.datetime(1988, 10, 20, 18, 0)], '9892': [datetime.datetime(1988, 10, 24, 0, 0), datetime.datetime(1988, 10, 24, 6, 0)], '9894': [datetime.datetime(1988, 11, 1, 18, 0), datetime.datetime(1988, 11, 2, 0, 0)], '9896': [datetime.datetime(1988, 11, 7, 0, 0), datetime.datetime(1988, 11, 7, 6, 0)], '9910': [datetime.datetime(1989, 1, 21, 6, 0), datetime.datetime(1989, 1, 21, 12, 0)], '9944': [datetime.datetime(1989, 5, 16, 12, 0), datetime.datetime(1989, 5, 16, 18, 0)], '9948': [datetime.datetime(1989, 6, 6, 0, 0), datetime.datetime(1989, 6, 6, 6, 0)], '9956': [datetime.datetime(1989, 7, 8, 6, 0), datetime.datetime(1989, 7, 8, 12, 0)], '9961': [datetime.datetime(1989, 7, 15, 18, 0), datetime.datetime(1989, 7, 16, 0, 0)], '9968': [datetime.datetime(1989, 7, 19, 18, 0), datetime.datetime(1989, 7, 20, 0, 0)], '10010': [datetime.datetime(1989, 10, 5, 12, 0), datetime.datetime(1989, 10, 5, 18, 0)], '10015': [datetime.datetime(1989, 10, 10, 12, 0), datetime.datetime(1989, 10, 10, 18, 0)], '10017': [datetime.datetime(1989, 10, 19, 0, 0), datetime.datetime(1989, 10, 19, 6, 0)], '10025': [datetime.datetime(1989, 11, 21, 18, 0), datetime.datetime(1989, 11, 22, 0, 0)], '10061': [datetime.datetime(1990, 5, 13, 6, 0), datetime.datetime(1990, 5, 13, 12, 0)], '10069': [datetime.datetime(1990, 6, 14, 6, 0), datetime.datetime(1990, 6, 14, 12, 0)], '10074': [datetime.datetime(1990, 6, 27, 0, 0), datetime.datetime(1990, 6, 27, 6, 0)], '10103': [datetime.datetime(1990, 8, 26, 6, 0), datetime.datetime(1990, 8, 26, 12, 0)], '10133': [datetime.datetime(1990, 10, 10, 12, 0), datetime.datetime(1990, 10, 10, 18, 0)], '10144': [datetime.datetime(1990, 11, 12, 12, 0), datetime.datetime(1990, 11, 12, 18, 0)], '10145': [datetime.datetime(1990, 11, 8, 6, 0)], '10147': [datetime.datetime(1990, 12, 4, 6, 0), datetime.datetime(1990, 12, 4, 12, 0)], '10164': [datetime.datetime(1991, 3, 12, 18, 0), datetime.datetime(1991, 3, 13, 0, 0)], '10173': [datetime.datetime(1991, 4, 24, 6, 0), datetime.datetime(1991, 4, 24, 12, 0)], '10181': [datetime.datetime(1991, 6, 15, 0, 0), datetime.datetime(1991, 6, 15, 6, 0)], '10188': [datetime.datetime(1991, 7, 9, 0, 0), datetime.datetime(1991, 7, 9, 6, 0)], '10192': [datetime.datetime(1991, 7, 22, 0, 0), datetime.datetime(1991, 7, 22, 6, 0)], '10200': [datetime.datetime(1991, 8, 12, 6, 0), datetime.datetime(1991, 8, 12, 12, 0)], '10212': [datetime.datetime(1991, 9, 1, 12, 0), datetime.datetime(1991, 9, 1, 18, 0)], '10233': [datetime.datetime(1991, 10, 27, 6, 0), datetime.datetime(1991, 10, 27, 12, 0)], '10238': [datetime.datetime(1991, 11, 12, 18, 0), datetime.datetime(1991, 11, 13, 0, 0)], '10241': [datetime.datetime(1991, 11, 4, 18, 0), datetime.datetime(1991, 11, 5, 0, 0)], '10246': [datetime.datetime(1991, 11, 16, 6, 0), datetime.datetime(1991, 11, 16, 12, 0)], '10293': [datetime.datetime(1992, 6, 22, 6, 0), datetime.datetime(1992, 6, 22, 12, 0)], '10299': [datetime.datetime(1992, 7, 11, 0, 0), datetime.datetime(1992, 7, 11, 6, 0)], '10302': [datetime.datetime(1992, 7, 15, 0, 0), datetime.datetime(1992, 7, 15, 6, 0)], '10305': [datetime.datetime(1992, 7, 19, 18, 0), datetime.datetime(1992, 7, 20, 0, 0)], '10362': [datetime.datetime(1992, 10, 25, 18, 0), datetime.datetime(1992, 10, 26, 0, 0)], '10373': [datetime.datetime(1992, 11, 10, 18, 0), datetime.datetime(1992, 11, 11, 0, 0)], '10401': [datetime.datetime(1993, 3, 1, 0, 0), datetime.datetime(1993, 3, 1, 6, 0)], '10408': [datetime.datetime(1993, 4, 13, 6, 0), datetime.datetime(1993, 4, 13, 12, 0)], '10410': [datetime.datetime(1993, 4, 27, 18, 0), datetime.datetime(1993, 4, 28, 0, 0)], '10417': [datetime.datetime(1993, 6, 25, 18, 0), datetime.datetime(1993, 6, 26, 0, 0)], '10418': [datetime.datetime(1993, 6, 19, 0, 0), datetime.datetime(1993, 6, 19, 6, 0)], '10422': [datetime.datetime(1993, 7, 7, 18, 0), datetime.datetime(1993, 7, 8, 0, 0)], '10441': [datetime.datetime(1993, 8, 21, 18, 0), datetime.datetime(1993, 8, 22, 0, 0)], '10449': [datetime.datetime(1993, 9, 8, 6, 0), datetime.datetime(1993, 9, 8, 12, 0)], '10452': [datetime.datetime(1993, 9, 15, 6, 0), datetime.datetime(1993, 9, 15, 12, 0)], '10454': [datetime.datetime(1993, 9, 20, 0, 0), datetime.datetime(1993, 9, 20, 6, 0)], '10458': [datetime.datetime(1993, 10, 4, 0, 0), datetime.datetime(1993, 10, 4, 6, 0)], '10463': [datetime.datetime(1993, 10, 7, 6, 0), datetime.datetime(1993, 10, 7, 12, 0)], '10466': [datetime.datetime(1993, 11, 1, 0, 0), datetime.datetime(1993, 11, 1, 6, 0)], '10473': [datetime.datetime(1993, 11, 18, 12, 0), datetime.datetime(1993, 11, 18, 18, 0)], '10474': [datetime.datetime(1993, 11, 20, 0, 0), datetime.datetime(1993, 11, 20, 6, 0)], '10476': [datetime.datetime(1993, 12, 5, 6, 0), datetime.datetime(1993, 12, 5, 12, 0)], '10478': [datetime.datetime(1993, 12, 9, 18, 0), datetime.datetime(1993, 12, 10, 0, 0)], '10481': [datetime.datetime(1993, 12, 26, 0, 0), datetime.datetime(1993, 12, 26, 6, 0)], '10485': [datetime.datetime(1994, 1, 5, 12, 0), datetime.datetime(1994, 1, 5, 18, 0)], '10511': [datetime.datetime(1994, 4, 3, 18, 0), datetime.datetime(1994, 4, 4, 0, 0)], '10517': [datetime.datetime(1994, 5, 24, 0, 0), datetime.datetime(1994, 5, 24, 6, 0)], '10520': [datetime.datetime(1994, 6, 22, 12, 0), datetime.datetime(1994, 6, 22, 18, 0)], '10524': [datetime.datetime(1994, 7, 1, 18, 0), datetime.datetime(1994, 7, 2, 0, 0)], '10532': [datetime.datetime(1994, 7, 19, 0, 0), datetime.datetime(1994, 7, 19, 6, 0)], '10553': [datetime.datetime(1994, 8, 24, 12, 0), datetime.datetime(1994, 8, 24, 18, 0)], '10563': [datetime.datetime(1994, 9, 10, 0, 0), datetime.datetime(1994, 9, 10, 6, 0)], '10583': [datetime.datetime(1994, 10, 20, 18, 0), datetime.datetime(1994, 10, 21, 0, 0)], '10595': [datetime.datetime(1994, 12, 21, 12, 0), datetime.datetime(1994, 12, 21, 18, 0)], '10625': [datetime.datetime(1995, 6, 1, 12, 0), datetime.datetime(1995, 6, 1, 18, 0)], '10635': [datetime.datetime(1995, 7, 27, 0, 0), datetime.datetime(1995, 7, 27, 6, 0)], '10645': [datetime.datetime(1995, 8, 13, 6, 0)], '10657': [datetime.datetime(1995, 9, 3, 18, 0), datetime.datetime(1995, 9, 4, 0, 0)], '10659': [datetime.datetime(1995, 9, 7, 0, 0), datetime.datetime(1995, 9, 7, 6, 0)], '10661': [datetime.datetime(1995, 9, 11, 18, 0), datetime.datetime(1995, 9, 12, 0, 0)], '10668': [datetime.datetime(1995, 9, 25, 6, 0), datetime.datetime(1995, 9, 25, 12, 0)], '10669': [datetime.datetime(1995, 9, 29, 6, 0), datetime.datetime(1995, 9, 29, 12, 0)], '10674': [datetime.datetime(1995, 10, 3, 6, 0)], '10675': [datetime.datetime(1995, 10, 7, 6, 0), datetime.datetime(1995, 10, 7, 12, 0)], '10681': [datetime.datetime(1995, 10, 23, 6, 0), datetime.datetime(1995, 10, 23, 12, 0)], '10682': [datetime.datetime(1995, 11, 2, 6, 0), datetime.datetime(1995, 11, 2, 12, 0)], '10683': [datetime.datetime(1995, 10, 28, 0, 0), datetime.datetime(1995, 10, 28, 6, 0)], '10691': [datetime.datetime(1995, 12, 4, 6, 0), datetime.datetime(1995, 12, 4, 12, 0)], '10713': [datetime.datetime(1996, 2, 29, 6, 0), datetime.datetime(1996, 2, 29, 12, 0)], '10721': [datetime.datetime(1996, 4, 6, 18, 0), datetime.datetime(1996, 4, 7, 0, 0)], '10758': [datetime.datetime(1996, 8, 19, 12, 0), datetime.datetime(1996, 8, 19, 18, 0)], '10788': [datetime.datetime(1996, 10, 17, 12, 0), datetime.datetime(1996, 10, 17, 18, 0)], '10789': [datetime.datetime(1996, 10, 11, 18, 0), datetime.datetime(1996, 10, 12, 0, 0)], '10799': [datetime.datetime(1996, 11, 5, 18, 0), datetime.datetime(1996, 11, 6, 0, 0)], '10805': [datetime.datetime(1996, 11, 8, 0, 0), datetime.datetime(1996, 11, 8, 6, 0)], '10810': [datetime.datetime(1996, 12, 1, 6, 0), datetime.datetime(1996, 12, 1, 12, 0)], '10853': [datetime.datetime(1997, 5, 26, 0, 0), datetime.datetime(1997, 5, 26, 6, 0)], '10916': [datetime.datetime(1997, 10, 19, 18, 0), datetime.datetime(1997, 10, 20, 0, 0)], '10921': [datetime.datetime(1997, 10, 29, 12, 0), datetime.datetime(1997, 10, 29, 18, 0)], '10928': [datetime.datetime(1997, 11, 16, 6, 0), datetime.datetime(1997, 11, 16, 12, 0)], '10980': [datetime.datetime(1998, 8, 7, 18, 0), datetime.datetime(1998, 8, 8, 0, 0)], '10999': [datetime.datetime(1998, 9, 18, 12, 0), datetime.datetime(1998, 9, 18, 18, 0)], '11016': [datetime.datetime(1998, 10, 14, 0, 0), datetime.datetime(1998, 10, 14, 6, 0)], '11020': [datetime.datetime(1998, 10, 21, 6, 0), datetime.datetime(1998, 10, 21, 12, 0)], '11035': [datetime.datetime(1998, 12, 10, 6, 0), datetime.datetime(1998, 12, 10, 12, 0)], '11039': [datetime.datetime(1998, 12, 17, 0, 0)], '11071': [datetime.datetime(1999, 4, 9, 18, 0), datetime.datetime(1999, 4, 10, 0, 0)], '11073': [datetime.datetime(1999, 4, 21, 0, 0)], '11106': [datetime.datetime(1999, 8, 20, 0, 0), datetime.datetime(1999, 8, 20, 6, 0)], '11128': [datetime.datetime(1999, 10, 5, 0, 0), datetime.datetime(1999, 10, 5, 6, 0)], '11133': [datetime.datetime(1999, 10, 15, 18, 0), datetime.datetime(1999, 10, 16, 0, 0)], '11139': [datetime.datetime(1999, 11, 8, 0, 0), datetime.datetime(1999, 11, 8, 6, 0)], '11178': [datetime.datetime(2000, 5, 17, 12, 0), datetime.datetime(2000, 5, 17, 18, 0)], '11188': [datetime.datetime(2000, 7, 4, 12, 0), datetime.datetime(2000, 7, 4, 18, 0)], '11191': [datetime.datetime(2000, 7, 14, 0, 0), datetime.datetime(2000, 7, 14, 6, 0)], '11194': [datetime.datetime(2000, 7, 22, 18, 0), datetime.datetime(2000, 7, 23, 0, 0)], '11224': [datetime.datetime(2000, 9, 11, 0, 0), datetime.datetime(2000, 9, 11, 6, 0)], '11245': [datetime.datetime(2000, 10, 27, 12, 0), datetime.datetime(2000, 10, 27, 18, 0)], '11249': [datetime.datetime(2000, 11, 2, 0, 0), datetime.datetime(2000, 11, 2, 6, 0)], '11254': [datetime.datetime(2000, 11, 29, 18, 0), datetime.datetime(2000, 11, 30, 0, 0)], '11268': [datetime.datetime(2001, 2, 18, 0, 0), datetime.datetime(2001, 2, 18, 6, 0)], '11281': [datetime.datetime(2001, 5, 7, 12, 0), datetime.datetime(2001, 5, 7, 18, 0)], '11350': [datetime.datetime(2001, 11, 6, 12, 0), datetime.datetime(2001, 11, 6, 18, 0)], '11360': [datetime.datetime(2001, 12, 5, 0, 0), datetime.datetime(2001, 12, 5, 6, 0)], '11382': [datetime.datetime(2002, 3, 21, 6, 0), datetime.datetime(2002, 3, 21, 12, 0)], '11406': [datetime.datetime(2002, 7, 18, 18, 0), datetime.datetime(2002, 7, 19, 0, 0)], '11416': [datetime.datetime(2002, 8, 13, 0, 0), datetime.datetime(2002, 8, 13, 6, 0)], '11496': [datetime.datetime(2003, 5, 26, 18, 0), datetime.datetime(2003, 5, 27, 0, 0)], '11508': [datetime.datetime(2003, 7, 22, 0, 0), datetime.datetime(2003, 7, 22, 6, 0)], '11509': [datetime.datetime(2003, 7, 16, 18, 0), datetime.datetime(2003, 7, 17, 0, 0)], '11520': [datetime.datetime(2003, 8, 22, 6, 0), datetime.datetime(2003, 8, 22, 12, 0)], '11554': [datetime.datetime(2003, 10, 23, 12, 0), datetime.datetime(2003, 10, 23, 18, 0)], '11556': [datetime.datetime(2003, 10, 31, 18, 0), datetime.datetime(2003, 11, 1, 0, 0)], '11559': [datetime.datetime(2003, 11, 13, 12, 0), datetime.datetime(2003, 11, 13, 18, 0)], '11567': [datetime.datetime(2003, 12, 27, 12, 0), datetime.datetime(2003, 12, 27, 18, 0)], '11591': [datetime.datetime(2004, 5, 16, 18, 0), datetime.datetime(2004, 5, 17, 0, 0)], '11597': [datetime.datetime(2004, 6, 9, 0, 0), datetime.datetime(2004, 6, 9, 6, 0)], '11660': [datetime.datetime(2004, 11, 19, 12, 0), datetime.datetime(2004, 11, 19, 18, 0)], '11694': [datetime.datetime(2005, 3, 16, 12, 0), datetime.datetime(2005, 3, 16, 18, 0)], '11775': [datetime.datetime(2005, 11, 10, 12, 0), datetime.datetime(2005, 11, 10, 18, 0)], '11776': [datetime.datetime(2005, 11, 20, 0, 0), datetime.datetime(2005, 11, 20, 6, 0)], '11785': [datetime.datetime(2005, 12, 15, 18, 0), datetime.datetime(2005, 12, 16, 0, 0)], '11810': [datetime.datetime(2006, 5, 11, 12, 0), datetime.datetime(2006, 5, 11, 18, 0)], '11814': [datetime.datetime(2006, 6, 24, 12, 0), datetime.datetime(2006, 6, 24, 18, 0)], '11826': [datetime.datetime(2006, 7, 31, 6, 0), datetime.datetime(2006, 7, 31, 12, 0)], '11861': [datetime.datetime(2006, 9, 27, 6, 0), datetime.datetime(2006, 9, 27, 12, 0)], '11872': [datetime.datetime(2006, 10, 29, 12, 0), datetime.datetime(2006, 10, 29, 18, 0)], '11875': [datetime.datetime(2006, 11, 10, 18, 0), datetime.datetime(2006, 11, 11, 0, 0)], '11880': [datetime.datetime(2006, 11, 30, 0, 0), datetime.datetime(2006, 11, 30, 6, 0)], '11883': [datetime.datetime(2006, 12, 9, 0, 0), datetime.datetime(2006, 12, 9, 6, 0)], '11965': [datetime.datetime(2007, 11, 4, 12, 0), datetime.datetime(2007, 11, 4, 18, 0)], '11971': [datetime.datetime(2007, 11, 19, 0, 0), datetime.datetime(2007, 11, 19, 6, 0)], '11972': [datetime.datetime(2007, 11, 25, 18, 0), datetime.datetime(2007, 11, 26, 0, 0)], '11983': [datetime.datetime(2008, 1, 12, 6, 0)], '11999': [datetime.datetime(2008, 4, 13, 0, 0), datetime.datetime(2008, 4, 13, 6, 0)], '12005': [datetime.datetime(2008, 5, 17, 6, 0), datetime.datetime(2008, 5, 17, 12, 0)], '12012': [datetime.datetime(2008, 6, 20, 0, 0), datetime.datetime(2008, 6, 20, 6, 0)], '12052': [datetime.datetime(2008, 9, 30, 12, 0), datetime.datetime(2008, 9, 30, 18, 0)], '12068': [datetime.datetime(2008, 11, 5, 12, 0), datetime.datetime(2008, 11, 5, 18, 0)], '12072': [datetime.datetime(2008, 11, 14, 18, 0)], '12103': [datetime.datetime(2009, 5, 1, 0, 0), datetime.datetime(2009, 5, 1, 6, 0)], '12104': [datetime.datetime(2009, 5, 7, 6, 0), datetime.datetime(2009, 5, 7, 12, 0)], '12107': [datetime.datetime(2009, 6, 15, 12, 0), datetime.datetime(2009, 6, 15, 18, 0)], '12110': [datetime.datetime(2009, 6, 23, 6, 0), datetime.datetime(2009, 6, 23, 12, 0)], '12150': [datetime.datetime(2009, 9, 26, 0, 0), datetime.datetime(2009, 9, 26, 6, 0)], '12153': [datetime.datetime(2009, 10, 3, 0, 0), datetime.datetime(2009, 10, 3, 6, 0)], '12164': [datetime.datetime(2009, 10, 30, 12, 0), datetime.datetime(2009, 10, 30, 18, 0)], '12170': [datetime.datetime(2009, 11, 24, 12, 0), datetime.datetime(2009, 11, 24, 18, 0)], '12213': [datetime.datetime(2010, 7, 13, 12, 0), datetime.datetime(2010, 7, 13, 18, 0)], '12215': [datetime.datetime(2010, 7, 18, 0, 0), datetime.datetime(2010, 7, 18, 6, 0)], '12249': [datetime.datetime(2010, 10, 18, 0, 0), datetime.datetime(2010, 10, 18, 6, 0)], '12290': [datetime.datetime(2011, 5, 7, 18, 0), datetime.datetime(2011, 5, 8, 0, 0)], '12294': [datetime.datetime(2011, 6, 8, 0, 0)], '12308': [datetime.datetime(2011, 7, 25, 18, 0), datetime.datetime(2011, 7, 26, 0, 0)], '12339': [datetime.datetime(2011, 9, 26, 18, 0), datetime.datetime(2011, 9, 27, 0, 0)], '12341': [datetime.datetime(2011, 10, 1, 0, 0), datetime.datetime(2011, 10, 1, 6, 0)], '12345': [datetime.datetime(2011, 10, 11, 6, 0), datetime.datetime(2011, 10, 11, 12, 0)], '12358': [datetime.datetime(2011, 12, 16, 6, 0), datetime.datetime(2011, 12, 16, 12, 0)], '12408': [datetime.datetime(2012, 8, 14, 18, 0), datetime.datetime(2012, 8, 15, 0, 0)], '12439': [datetime.datetime(2012, 10, 24, 0, 0), datetime.datetime(2012, 10, 24, 6, 0)], '12448': [datetime.datetime(2012, 12, 3, 18, 0), datetime.datetime(2012, 12, 4, 0, 0)], '12452': [datetime.datetime(2012, 12, 25, 12, 0), datetime.datetime(2012, 12, 25, 18, 0)], '12456': [datetime.datetime(2013, 1, 2, 12, 0), datetime.datetime(2013, 1, 2, 18, 0)], '12485': [datetime.datetime(2013, 6, 28, 18, 0), datetime.datetime(2013, 6, 29, 0, 0)], '12490': [datetime.datetime(2013, 7, 16, 18, 0), datetime.datetime(2013, 7, 17, 0, 0)], '12498': [datetime.datetime(2013, 8, 11, 18, 0), datetime.datetime(2013, 8, 12, 0, 0)], '12531': [datetime.datetime(2013, 10, 11, 12, 0), datetime.datetime(2013, 10, 11, 18, 0)], '12541': [datetime.datetime(2013, 10, 31, 6, 0), datetime.datetime(2013, 10, 31, 12, 0)], '12542': [datetime.datetime(2013, 11, 4, 0, 0), datetime.datetime(2013, 11, 4, 6, 0)], '12544': [datetime.datetime(2013, 11, 7, 18, 0), datetime.datetime(2013, 11, 8, 0, 0)], '12546': [datetime.datetime(2013, 11, 12, 0, 0), datetime.datetime(2013, 11, 12, 6, 0)], '12566': [datetime.datetime(2014, 1, 31, 12, 0), datetime.datetime(2014, 1, 31, 18, 0)], '12580': [datetime.datetime(2014, 3, 22, 6, 0), datetime.datetime(2014, 3, 22, 12, 0)], '12600': [datetime.datetime(2014, 7, 15, 6, 0), datetime.datetime(2014, 7, 15, 12, 0)], '12624': [datetime.datetime(2014, 9, 14, 6, 0), datetime.datetime(2014, 9, 14, 12, 0)], '12627': [datetime.datetime(2014, 9, 19, 0, 0), datetime.datetime(2014, 9, 19, 6, 0)], '12645': [datetime.datetime(2014, 11, 26, 6, 0), datetime.datetime(2014, 11, 26, 12, 0)], '12646': [datetime.datetime(2014, 12, 6, 12, 0), datetime.datetime(2014, 12, 6, 18, 0)], '12649': [datetime.datetime(2014, 12, 28, 18, 0), datetime.datetime(2014, 12, 29, 0, 0)], '12651': [datetime.datetime(2015, 1, 17, 6, 0), datetime.datetime(2015, 1, 17, 12, 0)], '12669': [datetime.datetime(2015, 4, 5, 0, 0), datetime.datetime(2015, 4, 5, 6, 0)], '12688': [datetime.datetime(2015, 7, 4, 18, 0), datetime.datetime(2015, 7, 5, 0, 0)], '12733': [datetime.datetime(2015, 10, 1, 12, 0), datetime.datetime(2015, 10, 1, 18, 0)], '12738': [datetime.datetime(2015, 10, 17, 18, 0), datetime.datetime(2015, 10, 18, 0, 0)], '12754': [datetime.datetime(2015, 12, 14, 6, 0), datetime.datetime(2015, 12, 14, 12, 0)], '12836': [datetime.datetime(2016, 10, 15, 18, 0), datetime.datetime(2016, 10, 16, 0, 0)], '12837': [datetime.datetime(2016, 10, 19, 12, 0), datetime.datetime(2016, 10, 19, 18, 0)], '12846': [datetime.datetime(2016, 11, 24, 12, 0), datetime.datetime(2016, 11, 24, 18, 0)], '12852': [datetime.datetime(2016, 12, 25, 12, 0), datetime.datetime(2016, 12, 25, 18, 0)]}'''

In [ ]:
# ------------------------------------------------------
'''
Probability of landfall in GD and PHI 
'''
tic = time.time() #WARNING: Expected runtime ~30min

#Creates a dictionary recording data
def landfall_probability(area, source):
    yeardict = {}
    yeardicttotal = {}
    
    if source == 0:
        startyear = 1945
    elif source == 1:
        startyear = 1949
    elif source == 2: 
        startyear = 1951
        
    for i in range(startyear, 2017): #No data before startyear, and 2017 data not complete
        yeardict[str(i)] = []
        yeardicttotal[str(i)] = []
    for tcindex in range(len(list(season))):
        if tcindex%100 == 0: #Just to check whether code is running + progress
            print(tcindex)
        i = list(season)[tcindex] #i is the year
        if i in list(range(startyear,2017)):
            yeardicttotal[str(i)].append(tcindex)
            if area == 0:
                if source == 0:
                    if str(tcindex) in landfalldata_coord_PHI_jtwc.keys():
                        yeardict[str(i)].append(tcindex)
                elif source == 1:
                    if str(tcindex) in landfalldata_coord_PHI_cma.keys():
                        yeardict[str(i)].append(tcindex)
                elif source == 2:
                    if str(tcindex) in landfalldata_coord_PHI_wmo.keys():
                        yeardict[str(i)].append(tcindex)
            else:
                if source == 0:
                    if str(tcindex) in landfalldata_coord_GD_jtwc.keys():
                        yeardict[str(i)].append(tcindex)
                elif source == 1:
                    if str(tcindex) in landfalldata_coord_GD_cma.keys():
                        yeardict[str(i)].append(tcindex)
                elif source == 2:
                    if str(tcindex) in landfalldata_coord_GD_wmo.keys():
                        yeardict[str(i)].append(tcindex)

    landfall_number_list = []
    landfall_number_list_total = []
    landfall_counts = {}
    landfall_counts_total = {}
    for year in yeardict.keys():
        landfall_number_list.append(len(yeardict[year])) #Append the number of landfalls in year
#        landfall_number_list_total.append(len(yeardicttotal[year]))
    for n in range(max(landfall_number_list)+1):
        landfall_counts[str(n)] = landfall_number_list.count(n)
#        landfall_counts_total[str(n)] = landfall_number_list_total.count(n)
        
#     print(yeardict, landfall_number_list)
    return landfall_counts #, yeardict, yeardicttotal
    
landfall_no_PHI_jtwc = landfall_probability(0,0)
landfall_no_PHI_cma = landfall_probability(0,1)
landfall_no_PHI_wmo = landfall_probability(0,2)
landfall_no_GD_jtwc = landfall_probability(1,0)
landfall_no_GD_cma = landfall_probability(1,1)
landfall_no_GD_wmo = landfall_probability(1,2)

toc = time.time()
print(toc-tic)

In [ ]:
print(landfall_no_PHI_wmo)
print(landfall_no_GD_wmo)
'''
No. of landfalls in each year:
PHI WMO:[6, 9, 5, 6, 3, 7, 6, 3, 5, 5, 2, 6, 5, 13, 4, 4, 7, 6, 3, 9, 13, 7, 5, 
11, 3, 4, 5, 6, 6, 6, 7, 8, 5, 3, 5, 8, 5, 8, 8, 4, 8, 4, 10, 6, 7, 5, 2, 6, 5, 
5, 3, 0, 6, 4, 3, 9, 4, 5, 7, 3, 6, 3, 9, 7, 7, 4]

GD WMO: [2, 3, 2, 3, 1, 0, 5, 2, 2, 3, 6, 2, 2, 6, 4, 3, 5, 3, 1, 4, 4, 2, 3, 
3, 5, 4, 1, 3, 4, 2, 3, 3, 4, 4, 3, 4, 2, 4, 2, 3, 4, 3, 6, 3, 6, 1, 2, 1, 6, 
1, 4, 4, 3, 2, 1, 3, 1, 6, 5, 3, 2, 3, 3, 3, 2, 2]'''

In [ ]:
def poisson_func(x,mu): #Define Poisson distribution for curve fitting
    P = np.exp(-mu) * mu**x/factorial(x)
    return P

#Plot landfall probability
def landfall_prob_plot(area, source):
    plt.figure()
    if area == 0:
        if source == 0:
            keys = landfall_no_PHI_jtwc.keys()
            values = landfall_no_PHI_jtwc.values()
            startyear = 1945 #Year which data starts to exist
        elif source == 1:
            keys = landfall_no_PHI_cma.keys()
            values = landfall_no_PHI_cma.values()
            startyear = 1949
        elif source == 2:
            keys = landfall_no_PHI_wmo.keys()
            values = landfall_no_PHI_wmo.values()
            startyear = 1951
    else: 
        if source == 0:
            keys = landfall_no_GD_jtwc.keys()
            values = landfall_no_GD_jtwc.values()
            startyear = 1945
        elif source == 1:
            keys = landfall_no_GD_cma.keys()
            values = landfall_no_GD_cma.values()
            startyear = 1949
        elif source == 2:
            keys = landfall_no_GD_wmo.keys()
            values = landfall_no_GD_wmo.values()
            startyear = 1951
            
    plt.xlabel('Number of landfalls per year')  
    plt.ylabel('Probability') #(number of years/total number of years)')
        
    prob = np.asarray(list(values)) / (2017 - startyear)
    k = np.asarray(list(keys),dtype=int)
    plt.scatter(k, prob,color = 'r')
    thesum = 0
    for i in range(len(values)):
        thesum += np.asarray(list(keys),dtype=int)[i]*list(values)[i]
        
    mu = thesum / (2017-startyear)
#    mu = curve_fit(poisson_func,k,prob)[0][0]

    pmf = poisson.pmf(k,mu) #Plot Poisson distribution
    
    chisq, pvalue = chisquare(np.array(list(values)),pmf*(2017 - startyear),ddof=1) #Find chi-sq and p-value using chi-square test
#    print(chisq, pvalue)
#    print(list(values))
#    print(pmf*sum(list(values)))
#    print(prob, pmf)
    rmserr = np.sqrt(mean_squared_error(prob, pmf))

    plt.plot(k,pmf,color='b')
    
    if area == 0:
        if source == 0:
            plt.title('Probability of typhoons with landfalls in the Philippines \n source: JTWC, sample size = ' + str(thesum) + '\n mean = ' + str(mu) + ', RMS error = ' + str(rmserr))
        elif source == 1:
            plt.title('Probability of typhoons with landfalls in the Philippines \n source: CMA, sample size = ' + str(thesum) + '\n mean = ' + str(mu) + ', RMS error = ' + str(rmserr))
        elif source == 2:
            plt.title('Probability of typhoons with landfalls in the Philippines \n source: WMO, sample size = ' + str(thesum) + '\n mean = ' + str(mu) + ', RMS error = ' + str(rmserr))


    else:
        if source == 0:
            plt.title('Probability of typhoons with landfalls in Guangdong \n source: JTWC, sample size = ' + str(thesum) + '\n mean = ' + str(mu) + ', RMS error = ' + str(rmserr))
        elif source == 1:
            plt.title('Probability of typhoons with landfalls in Guangdong \n source: CMA, sample size = ' + str(thesum) + '\n mean = ' + str(mu) + ', RMS error = ' + str(rmserr))
        elif source == 2:
            plt.title('Probability of typhoons with landfalls in Guangdong \n source: WMO, sample size = ' + str(thesum) + '\n mean = ' + str(mu) + ', RMS error = ' + str(rmserr))

tic = time.time()
            
landfall_prob_plot(0,0)
landfall_prob_plot(0,1)
landfall_prob_plot(0,2)
landfall_prob_plot(1,0)
landfall_prob_plot(1,1)
landfall_prob_plot(1,2)

toc = time.time()
print(toc-tic)

In [ ]:
#Get daily rainfall amount from .nc file
def getfile(tcindex, area):
    filedirectory = 'IMERG/'+ str(tcindex) + str(area)

    totalprecipitation = np.zeros((450,379))
    
    #Sum daily rainfall amount from each file
    for file in os.listdir(filedirectory):
        if file.endswith('.nc'):
            nc_imerg = 'IMERG/' + str(tcindex) + str(area) + '/' + file
            imerg = Dataset(nc_imerg)

            precipitationCal = imerg.variables['precipitationCal'][:] #Rainfall
            lat_rainfall = imerg.variables['lat'][:] #Latitude range
            lon_rainfall = imerg.variables['lon'][:] #Longitude range
            totalprecipitation += precipitationCal
        
    return totalprecipitation, lat_rainfall, lon_rainfall

lat_rainfall = getfile(12733,1)[1]
lon_rainfall = getfile(12733,1)[2]

In [ ]:
tic = time.time() #Expected runtime ~15s

#Generate land mask for plotting rainfall contour map
def land_rainfall_contour(area):
    
    if area == 0:
        z_land_coord = z_land_coord_PHI
    else:
        extremeG_N = [118.3, 28.3]
        extremeG_S = [107.4, 18.9]
        extremeG_E = [120.0, 23.5]
        extremeG_W = [105.7, 25.5]

        m_top_G = (extremeG_N[1]-extremeG_W[1])/(extremeG_N[0]-extremeG_W[0])
        m_bot_G = (extremeG_S[1]-extremeG_E[1])/(extremeG_S[0]-extremeG_E[0])
        m_left_G = (extremeG_S[1]-extremeG_W[1])/(extremeG_S[0]-extremeG_W[0])
        m_right_G = (extremeG_N[1]-extremeG_E[1])/(extremeG_N[0]-extremeG_E[0])

        c_top_G = extremeG_N[1]-m_top_G*extremeG_N[0]
        c_bot_G = extremeG_S[1]-m_bot_G*extremeG_S[0]
        c_left_G = extremeG_S[1]-m_left_G*extremeG_S[0]
        c_right_G = extremeG_N[1]-m_right_G*extremeG_N[0]

        landmask_points = []

        for i in range(17100,18100): 
#             if i%50 == 0:
#                 print(i)
            for i2 in range(6500,7150):
                equation_top = m_top_G * landmask_LON[i,i2] + c_top_G
                equation_bot = m_bot_G * landmask_LON[i,i2] + c_bot_G 
                equation_left = m_left_G * landmask_LON[i,i2] + c_left_G
                equation_right  = m_right_G * landmask_LON[i,i2] + c_right_G

                if landmask_LAT[i,i2] <= equation_top and landmask_LAT[i,i2] <= equation_right and landmask_LAT[i,i2] >= equation_bot and landmask_LAT[i,i2] >= equation_left:
                    landmask_points.append(i)
                    landmask_points.append(i2)
        
        landmask_points = np.asarray(landmask_points)
        landmask_points = landmask_points.reshape(int(len(landmask_points)/2),2)
        
        
        z_land = []
        for i in range(len(landmask_points)):
#             if i%50 == 0:
#                 print(i)
            if z[landmask_points[i,1],landmask_points[i,0]] >= 0:
                z_land.append(landmask_points[i,0])
                z_land.append(landmask_points[i,1])

        z_land = np.asarray(z_land).reshape(int(len(z_land)/2),2)

        z_land_coord = np.zeros(z_land.shape)
        z_land_coord[:,0] = landmask_lon[z_land[:,0]]
        z_land_coord[:,1] = landmask_lat[z_land[:,1]]
    
        
    return z_land_coord

landforrainfall_PHI = land_rainfall_contour(0) #Generate for PHI
landforrainfall_GD = land_rainfall_contour(1) #Generate for GD
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()

#Plot rainfall contour
def rainfall_contour(tcindex, area, source):
    totalprecipitation, lat_rainfall, lon_rainfall = getfile(tcindex, area)
    LATrainfall, LONrainfall = np.meshgrid(lat_rainfall,lon_rainfall)
    
    if area == 0:
        if source == 0:
            coords = landfalldata_coord_PHI2014_jtwc[str(tcindex)] #TC coord (before & after)
        elif source == 1:
            coords = landfalldata_coord_PHI2014_cma[str(tcindex)]
        elif source == 2:
            coords = landfalldata_coord_PHI2014_wmo[str(tcindex)]
    else:
        if source == 0:
            coords = landfalldata_coord_GD2014_jtwc[str(tcindex)] #TC coord (before & after)
        elif source == 1:
            coords = landfalldata_coord_GD2014_cma[str(tcindex)]
        elif source == 2:
            coords = landfalldata_coord_GD2014_wmo[str(tcindex)]
    
    plt.figure()
    plt.scatter(landforrainfall_PHI[:,0],landforrainfall_PHI[:,1],color='k',alpha=0.6)
    plt.scatter(landforrainfall_GD[:,0],landforrainfall_GD[:,1],color='k', alpha=0.6)

    plt.contourf(LONrainfall, LATrainfall, totalprecipitation, cmap = 'jet', alpha=0.6) #Contour plot of rainfall
    plt.colorbar()
    plt.scatter(coords[:,1],coords[:,0],color='r', alpha= 0.5)
    
    if area == 0:
        if source == 0:
            plt.title('Rainfall contour map of ' + str(tcindex) + ' when it makes landfall in the Philippines \n source: JTWC')
        elif source == 1:
            plt.title('Rainfall contour map of ' + str(tcindex) + ' when it makes landfall in the Philippines \n source: CMA')
        elif source == 2:
            plt.title('Rainfall contour map of ' + str(tcindex) + ' when it makes landfall in the Philippines \n source: WMO')
    else:
        if source == 0:
            plt.title('Rainfall contour map of ' + str(tcindex) + ' when it makes landfall in Guangdong \n source: JTWC')
        elif source == 1:
            plt.title('Rainfall contour map of ' + str(tcindex) + ' when it makes landfall in Guangdong \n source: CMA')
        elif source == 2:
            plt.title('Rainfall contour map of ' + str(tcindex) + ' when it makes landfall in Guangdong \n source: WMO')


    plt.show()
    
#     fig = plt.figure()
#     ax = fig.add_subplot(111, projection='3d')
#     ax.plot_surface(LONrainfall,LATrainfall,totalprecipitation)
    
rainfall_contour(12862,0,1)
rainfall_contour(12862,1,1)

toc=time.time()
print(toc-tic)

In [ ]:
#Rainfall distribution within (radius)km of TC centre
def rainfall_distribution(area, source, tcindex, only_on_land = False, radius = 500):
    totalprecipitation, lat_rainfall, lon_rainfall = getfile(tcindex, area)
    epsilon = 1e-5 #Very small number to compare numbers with rounding error
    dist_list = []
    rainfall_list = []
    if area == 0:
        if source == 0:
            coords = landfalldata_coord_PHI2014_jtwc[str(tcindex)]
        elif source == 1:
            coords = landfalldata_coord_PHI2014_cma[str(tcindex)]
        elif source == 2:
            coords = landfalldata_coord_PHI2014_wmo[str(tcindex)]            
        land = landforrainfall_PHI
    else:
        if source == 0:
            coords = landfalldata_coord_GD2014_jtwc[str(tcindex)]
        elif source == 1:
            coords = landfalldata_coord_GD2014_cma[str(tcindex)]
        elif source == 2:
            coords = landfalldata_coord_GD2014_wmo[str(tcindex)]
        land = landforrainfall_GD
    landfall_coords_lon = coords[1][1]
    landfall_coords_lat = coords[1][0]
    for i in range(len(totalprecipitation)):
        for j in range(len(totalprecipitation[0])):
            rainfall_lat = lat_rainfall[j]
            rainfall_lon = lon_rainfall[i]
            dist = gd.geodesic([rainfall_lat, rainfall_lon],[landfall_coords_lat, landfall_coords_lon]).km
            if dist <= radius:
                if only_on_land == False:
                    dist_list.append(dist)
                    rainfall_list.append(totalprecipitation[i,j])
                else:
                    #Find all land points with same latitude
                    landpoint_onlat_temp1 = np.where(land[:,1] <= rainfall_lat + epsilon)
                    landpoint_onlat_temp2 = np.where(land[:,1] >= rainfall_lat - epsilon)
                    landpoint_onlat_index = np.intersect1d(landpoint_onlat_temp1,landpoint_onlat_temp2) #Index of points in landmask array where latitude is same as TClat
                    landpoint_onlat_lon = land[landpoint_onlat_index,0] #Lon of landmask points with same lat as TClat

                    if landpoint_onlat_lon.size != 0:
                        landpoint_onlat_maxlon = max(landpoint_onlat_lon) #Find easternmost land point on same latitude
                        landpoint_onlat_minlon = min(landpoint_onlat_lon) #Find westernmost land point on same latitude

                        #Check whether TC is on land or ocean
                        if landpoint_onlat_minlon-epsilon <= rainfall_lon <= landpoint_onlat_maxlon+epsilon:
                            dist_list.append(dist)
                            rainfall_list.append(totalprecipitation[i,j])
                        
    return dist_list, rainfall_list #Distance from TC centre, amount of rainfall

#Plot the rainfall distribution of each point
def plot_rainfall_distribution(area, source, tcindex, only_on_land = False):
    dist_list, rainfall_list = rainfall_distribution(area, source, tcindex, only_on_land)
    plt.figure()
    if area == 0:
        plt.title('Rainfall distribution of ' + str(tcindex) + ' in the Philippines')
    else:
        plt.title('Rainfall distribution of ' + str(tcindex) + ' in Guangdong')
        
    plt.xlabel('Distance to TC (km)')
    plt.ylabel('Precipitation (mm)')
    plt.scatter(dist_list, rainfall_list,color = 'b')
    plt.show()

In [ ]:
tic = time.time() #Expected runtime ~50s
#%matplotlib tk
plot_rainfall_distribution(1,1,12862)
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
#%matplotlib tk
plot_rainfall_distribution(1,1,12862, True)
toc = time.time()
print(toc-tic)

In [ ]:
#Input radii of 18m/s wind (R18) data, also known as radii of 34kts wind (R34)

r18dict_GD_jtwc = {} #Values in nautical mile, change to km
r18dict_GD_jtwc['11290'] = [] #Utor 2001, no data, from bwp0602001.txt
r18dict_GD_jtwc['11298'] = []#[float((60+90+120+60)*1.852/4), 'no data']#Yutu 2001, from bwp102001.txt
r18dict_GD_jtwc['11318'] = [float((70+70+40+40)*1.852/4), float((85+85+60+60)*1.852/4)] #Nari 2001, from bwp202001.txt
r18dict_GD_jtwc['11410'] = [float((60+120+120+70)*1.852/4), float((70+120+120+70)*1.852/4)] #Kammuri 2002, from bwp162002.txt
r18dict_GD_jtwc['11417'] = [float((60+70+70+60)*1.852/4), float((60+70+70+60)*1.852/4)] #Vongfong 2002, from bwp202002.txt
r18dict_GD_jtwc['11431'] = [] #Hagupit 2002, no data, from bwp232002.txt
r18dict_GD_jtwc['11508'] = []#[float((160+160+140+140)*1.852/4), 'no data'] #Imbudo 2003, from bwp092003.txt
r18dict_GD_jtwc['11520'] = [float((120+105+100+105)*1.852/4), float((120+105+100+105)*1.852/4)] #Krovanh 2003, from bwp122003.txt
r18dict_GD_jtwc['11528'] = [float((190+190+170+160)*1.852/4), float((170+170+145+135)*1.852/4)] #Dujuan 2003, from bwp142004.txt
r18dict_GD_jtwc['11605'] = [float((40+40+40+40)*1.852/4), float((40+40+40+40)*1.852/4)] #Kompasu 2004, from bwp122004.txt
r18dict_GD_jtwc['11623'] = [float((110+135+135+90)*1.852/4), float((110+135+135+90)*1.852/4)]#Aere 2004, from bwp202004.txt
r18dict_GD_jtwc['11725'] = [float((80+80+75+70)*1.852/4), float((80+80+75+70)*1.852/4)] #Sanvu 2005, from bwp102005.txt
r18dict_GD_jtwc['11810'] = [float((140+140+120+120)*1.852/4), float((140+140+120+100)*1.852/4)] #Chanchu 2006, from bwp022006.txt
r18dict_GD_jtwc['11814'] = [] #Jelawat 2006, no data, from bwp032006.txt
r18dict_GD_jtwc['11826'] = [float((140+120+120+105)*1.852/4), float((140+120+100+100)*1.852/4)] #Prapiroon 2006, from bwp072006.txt
r18dict_GD_jtwc['11842'] = [] #TD0823 2006, no data, from bwp132006.txt
r18dict_GD_jtwc['11850'] = [] #TD0912 2006, no data, from bwp152006.txt
r18dict_GD_jtwc['11999'] = [float((70+70+60+60)*1.852/4), float((70+65+60+65)*1.852/4)] #Neoguri 2008, from bwp022008.txt
r18dict_GD_jtwc['12012'] = [float((90+90+80+75)*1.852/4), float((90+90+80+75)*1.852/4)] #Fengshen 2008, from bwp072008.txt
r18dict_GD_jtwc['12025'] = [float((90+75+75+85)*1.852/4), float((80+70+70+80)*1.852/4)] #Kamuri 2008, from bwp102008.txt
r18dict_GD_jtwc['12034'] = [float((55+55+55+55)*1.852/4), float((55+55+55+55)*1.852/4)] #Nuri 2008, from bwp132008.txt
r18dict_GD_jtwc['12048'] = [float((155+135+130+155)*1.852/4), float((155+135+130+155)*1.852/4)] #Hagupit 2008, from bwp182008.txt
r18dict_GD_jtwc['12052'] = [] #Higos 2008, no data, from bwp212008.txt
r18dict_GD_jtwc['12110'] = [] #Nangka 2009, no data, from bwp042009.txt
r18dict_GD_jtwc['12114'] = [] #Soudelor 2009, no data, from bwp052009.txt
r18dict_GD_jtwc['12118'] = [float((85+70+55+60)*1.852/4), float((85+70+55+60)*1.852/4)] #Molave 2009, from bwp072009.txt
r18dict_GD_jtwc['12121'] = [] #Goni 2009, no data, from bwp082009.txt
r18dict_GD_jtwc['12146'] = [float((130+75+95+80)*1.852/4), float((130+75+95+80)*1.852/4)] #Koppu 2009, from bwp162009.dat
r18dict_GD_jtwc['12215'] = [float((105+95+75+95)*1.852/4), float((105+95+75+95)*1.852/4)] #Chanthu 2010, from bwp042010.txt
r18dict_GD_jtwc['12238'] = [float((120+130+100+85)*1.852/4), float((120+130+100+85)*1.852/4)] #Fanapi 2010, from bwp122010.txt
r18dict_GD_jtwc['12245'] = [] #Fourteen 2010, no data
r18dict_GD_jtwc['12294'] = [] #Sarika 2011, no data
r18dict_GD_jtwc['12296'] = [] #Haima 2011, no data
r18dict_GD_jtwc['12393'] = []#[float((70+55+55+70)*1.852/4), 'no data'] #Doksuri 2012, from bwp072012.dat
r18dict_GD_jtwc['12399'] = [float((110+95+95+119)*1.852/4), float((110+95+95+119)*1.852/4)] #Vincente 2012, from bwp092012.dat
r18dict_GD_jtwc['12408'] = [float((80+70+70+90)*1.852/4), float((80+70+70+80)*1.852/4)] #Kai-tak 2012, from bwp142012.dat
r18dict_GD_jtwc['12485'] = [float((70+65+60+60)*1.852/4), float((70+65+60+60)*1.852/4)] #Rumbia 2013, from bwp062013.dat
r18dict_GD_jtwc['12498'] = [float((125+130+125+110)*1.852/4), float((125+130+125+110)*1.852/4)] #Utor 2013, from bwp112013.dat
r18dict_GD_jtwc['12520'] = [float((155+140+160+165)*1.852/4), float((155+140+160+165)*1.852/4)] #Usagi 2013
r18dict_GD_jtwc['12594'] = [float((90+90+80+80)*1.852/4), float((90+90+80+80)*1.852/4)] #Hagibis 2014, from bwp072014.dat
r18dict_GD_jtwc['12600'] = [float((115+100+100+115)*1.852/4), float((115+100+100+115)*1.852/4)] #Rammasun 2014, from bwp092014.dat
r18dict_GD_jtwc['12621'] = [] #Fourteen 2014, from bwp142014.dat, no data
r18dict_GD_jtwc['12624'] = [float((155+150+140+150)*1.852/4), float((155+150+140+150)*1.852/4)] #Kalmaegi 2014, from bwp152014.dat
r18dict_GD_jtwc['12688'] = [float((75+70+70+75)*1.852/4), float((75+60+60+75)*1.852/4)] #Linfa 2015, from bwp102015.dat
r18dict_GD_jtwc['12733'] = [float((135+120+120+135)*1.852/4), float((135+120+120+135)*1.852/4)] #Mujigae 2015, from bwp222015.dat
r18dict_GD_jtwc['12776'] = [] #TD0526 2016, from bwp012016.dat, no data
r18dict_GD_jtwc['12793'] = [float((150+150+150+150)*1.852/4), float((150+150+150+130)*1.852/4)] #Nida 2016, from bwp062016.dat
r18dict_GD_jtwc['12837'] = [float((150+120+120+150)*1.852/4), float((150+120+120+150)*1.852/4)] #Haima 2016, from bwp252016.dat
r18dict_GD_jtwc['12861'] = [float((150+115+95+140)*1.852/4), float((100+100+95+95)*1.852/4)] #Hato 2017, from bwp152017.dat
r18dict_GD_jtwc['12862'] = [] #Mangkhut 2018, no data

r18dict_GD_jtwc_2014 = {}
r18dict_GD_jtwc_2014['12594'] = [float((90+90+80+80)*1.852/4), float((90+90+80+80)*1.852/4)] #Hagibis 2014, from bwp072014.dat
r18dict_GD_jtwc_2014['12600'] = [float((115+100+100+115)*1.852/4), float((115+100+100+115)*1.852/4)] #Rammasun 2014, from bwp092014.dat
r18dict_GD_jtwc_2014['12621'] = [] #Fourteen 2014, from bwp142014.dat, no data
r18dict_GD_jtwc_2014['12624'] = [float((155+150+140+150)*1.852/4), float((155+150+140+150)*1.852/4)] #Kalmaegi 2014, from bwp152014.dat
r18dict_GD_jtwc_2014['12688'] = [float((75+70+70+75)*1.852/4), float((75+60+60+75)*1.852/4)] #Linfa 2015, from bwp102015.dat
r18dict_GD_jtwc_2014['12733'] = [float((135+120+120+135)*1.852/4), float((135+120+120+135)*1.852/4)] #Mujigae 2015, from bwp222015.dat
r18dict_GD_jtwc_2014['12776'] = [] #TD0526 2016, from bwp012016.dat, no data
r18dict_GD_jtwc_2014['12793'] = [float((150+150+150+150)*1.852/4), float((150+150+150+130)*1.852/4)] #Nida 2016, from bwp062016.dat
r18dict_GD_jtwc_2014['12837'] = [float((150+120+120+150)*1.852/4), float((150+120+120+150)*1.852/4)] #Haima 2016, from bwp252016.dat
r18dict_GD_jtwc_2014['12861'] = [float((150+115+95+140)*1.852/4), float((100+100+95+95)*1.852/4)] #Hato 2017, from bwp152017.dat
r18dict_GD_jtwc_2014['12862'] = [] #Mangkhut 2018, no data

r18dict_GD_cma = {} #Values in km
r18dict_GD_cma['12594'] = [] #no data
r18dict_GD_cma['12600'] = [] #no data
r18dict_GD_cma['12621'] = [] #no data
r18dict_GD_cma['12624'] = [] #no data
r18dict_GD_cma['12688'] = [float((120+150+120+100)/4), float((120+150+120+100)/4)]
r18dict_GD_cma['12733'] = [float((200+200+200+200)/4), float((200+200+200+200)/4)]
r18dict_GD_cma['12776'] = [] #no data
r18dict_GD_cma['12793'] = [float((300+260+260+300)/4), float((260+260+170+170)/4)]
r18dict_GD_cma['12837'] = []#[float((380+350+380+320)/4), 'no data']
r18dict_GD_cma['12861'] = [float((280+220+260+240)/4), float((280+220+260+240)/4)]
r18dict_GD_cma['12862'] = []#[float((400+300+300+250)/4), 'no data']

r18dict_PHI_jtwc = {} #Values in nautical mile, change to km
r18dict_PHI_jtwc['12566'] = [] #Kajiki 2014, from bwp022014.dat, no data
r18dict_PHI_jtwc['12580'] = [] #Four 2014, no data
r18dict_PHI_jtwc['12600'] = [float((80+80+85+85)*1.852/4), float((80+80+85+85)*1.852/4)] #Rammasun 2014, from bwp092014.dat
r18dict_PHI_jtwc['12624'] = [float((110+120+120+110)*1.852/4),float((110+120+120+110)*1.852/4)] #Kalmaegi 2014, from bwp152014.dat
r18dict_PHI_jtwc['12627'] = [float((50+70+55+55)*1.852/4), float((55+40+40+55)*1.852/4)] #Fung-wong 2014, from bwp162014.dat
r18dict_PHI_jtwc['12645'] = [] #Sinlaku 2014, from bwp212014.dat, no data
r18dict_PHI_jtwc['12646'] = [float((125+110+110+125)*1.852/4), float((125+110+110+125)*1.852/4)] #Hagupit 2014, from bwp222014.dat
r18dict_PHI_jtwc['12649'] = ['no data', float((40+40+40+40)*1.852/4)] #Jangmi 2014, from bwp232014.dat
r18dict_PHI_jtwc['12651'] = [float((75+70+75+90)*1.852/4), float((75+70+75+80)*1.852/4)] #Mekkhala 2015, from bwp012015.dat
r18dict_PHI_jtwc['12669'] = [] #Maysak 2015, from bwp042015.dat, no data
r18dict_PHI_jtwc['12688'] = [float((80+65+65+80)*1.852/4), float((80+65+65+80)*1.852/4)] #Linfa 2015, from bwp102015.dat
r18dict_PHI_jtwc['12733'] = [] #Mujigae 2015, from bwp222015.dat, no data
r18dict_PHI_jtwc['12738'] = [float((110+155+95+85)*1.852/4), float((110+155+95+85)*1.852/4)] #Koppu 2015, from bwp242015.dat
r18dict_PHI_jtwc['12754'] = [float((135+75+75+110)*1.852/4), float((135+75+75+110)*1.852/4)] #Melor 2015, bwp282015.dat
r18dict_PHI_jtwc['12836'] = [float((150+110+110+150)*1.852/4), float((160+110+120+170)*1.852/4)] #Sarika 2016, from bwp242016.dat
r18dict_PHI_jtwc['12837'] = [float((220+200+190+200)*1.852/4), float((230+210+190+200)*1.852/4)] #Haima 2016, from bwp252016.dat
r18dict_PHI_jtwc['12846'] = [float((50+50+50+50)*1.852/4), float((50+50+50+50)*1.852/4)] #Tokage 2016, from bwp292016.dat
r18dict_PHI_jtwc['12852'] = [float((120+80+90+110)*1.852/4), float((110+80+80+120)*1.852/4)] #Nock-ten 2016
r18dict_PHI_jtwc['12862'] = [] #Mangkhut 2018, no data

r18dict_PHI_cma = {} #Values in km
r18dict_PHI_cma['12566'] = [] #no data
r18dict_PHI_cma['12580'] = [] #no data
r18dict_PHI_cma['12600'] = [] #no data
r18dict_PHI_cma['12624'] = [] #no data
r18dict_PHI_cma['12627'] = [] #no data
r18dict_PHI_cma['12645'] = [] #no data
r18dict_PHI_cma['12646'] = [] #no data
r18dict_PHI_cma['12649'] = [] #no data
r18dict_PHI_cma['12651'] = [] #no data
r18dict_PHI_cma['12669'] = [float((200+150+50+100)/4), float((150+130+70+70)/4)]
r18dict_PHI_cma['12688'] = [float((250+220+250+200)/4), float((260+230+260+200)/4)]
r18dict_PHI_cma['12733'] = [] #no data
r18dict_PHI_cma['12738'] = [float((350+350+350+300)/4), float((350+350+350+300)/4)]
r18dict_PHI_cma['12754'] = [float((250+230+200+250)/4), float((250+230+200+250)/4)]
r18dict_PHI_cma['12836'] = [float((260+240+200+240)/4), float((230+230+260+260)/4)]
r18dict_PHI_cma['12837'] = [float((380+350+380+320)/4), float((380+350+380+320)/4)]
r18dict_PHI_cma['12846'] = [] #no data
r18dict_PHI_cma['12852'] = [float((280+200+240+280)/4), float((280+220+220+240)/4)]
r18dict_PHI_cma['12862'] = [float((600+500+350+500)/4), float((550+480+480+400)/4)]

In [ ]:
#Plot log rain vs log pd
def plot_lograin_logpd(area, source, only_on_land = False, intersect = False, same_size_as_r18 = False):
    if area == 0:
        if source == 0:
            pressuredict = landfalldata_pres_PHI2014_jtwc.copy()
        elif source == 1:
            pressuredict = landfalldata_pres_PHI2014_cma.copy()
        elif source == 2:
            pressuredict = landfalldata_pres_PHI2014_wmo.copy()
        
        pressuredict.pop('12566', None) #No rainfall data for 12566
        pressuredict.pop('12580', None) #No rainfall data for 12580
    else:
        if source == 0:
            pressuredict = landfalldata_pres_GD2014_jtwc.copy()
        elif source == 1:
            pressuredict = landfalldata_pres_GD2014_cma.copy()
        elif source == 2:
            pressuredict = landfalldata_pres_GD2014_wmo.copy()
            
        pressuredict.pop('12860',None)
            
    if intersect == True:
        if area == 0:
            key_a = list(landfalldata_pres_PHI2014_jtwc.keys())
            key_b = list(landfalldata_pres_PHI2014_cma.keys())
            key_c = list(landfalldata_pres_PHI2014_wmo.keys())
        else:
            key_a = list(landfalldata_pres_GD2014_jtwc.keys())
            key_b = list(landfalldata_pres_GD2014_cma.keys())
            key_c = list(landfalldata_pres_GD2014_wmo.keys())
        key_intersect_list = np.intersect1d(np.intersect1d(key_a, key_b),key_c)
        
        if '12566' in key_intersect_list:
            key_intersect_list = np.delete(key_intersect_list,np.where(key_intersect_list=='12566'))
        if '12580' in key_intersect_list:
            key_intersect_list = np.delete(key_intersect_list,np.where(key_intersect_list=='12580'))
        if '12860' in key_intersect_list:
            key_intersect_list = np.delete(key_intersect_list,np.where(key_intersect_list=='12860'))
            
        print(key_intersect_list)
        tcindexes = np.asarray(key_intersect_list)
        
    else:
        print(list(pressuredict.keys()))
        tcindexes = np.asarray(list(pressuredict.keys()), dtype=int)
    
    rainfall_total_list = []
    pdlist = []
    
    if same_size_as_r18 == True:
        for tc in tcindexes:
            print(tc)
            if (source == 0 and str(tc) in list(r18dict_GD_jtwc_2014.keys()) and r18dict_GD_jtwc_2014[str(tc)] != []) or (source == 1 and str(tc) in list(r18dict_GD_cma.keys()) and r18dict_GD_cma[str(tc)] != []):
                rainfall_list = rainfall_distribution(area, source, tc, only_on_land)[1]
                total_rainfall = sum(rainfall_list) #Total rainfall within 500km of TC centre
                rainfall_total_list.append(total_rainfall)
                pressure = pressuredict[str(tc)][1] #Get TC pressure after landfall
                pd = 1011-pressure
                pdlist.append(pd)
    else:
        for tc in tcindexes:
            print(tc)
            rainfall_list = rainfall_distribution(area, source, tc, only_on_land)[1]
            total_rainfall = sum(rainfall_list) #Total rainfall within 500km of TC centre
            rainfall_total_list.append(total_rainfall)
            pressure = pressuredict[str(tc)][1] #Get TC pressure after landfall
            pd = 1011-pressure
            pdlist.append(pd)
        
    pdlist = np.asarray(pdlist)
    rainfall_total_list = np.asarray(rainfall_total_list)
    
    logpdlist = np.log(pdlist)
    lograinfalllist = np.log(rainfall_total_list)
        
    plt.figure()
    plt.scatter(logpdlist, lograinfalllist,color='b')
    slope, intercept, rvalue, pvalue, stderr = linregress(logpdlist, lograinfalllist) #Linear fit
    xrange = np.linspace(min(logpdlist),max(logpdlist),1000)
    plt.plot(xrange, slope*xrange + intercept, 'r-') #Plot linear fit
    samplesize = len(lograinfalllist)

    plt.xlabel('log(pressure deficit)')
    if only_on_land == True:
        plt.ylabel('log(total volume of rain within 500km on land)')
    else:
        plt.ylabel('log(total volume of rain within 500km)')
    if area == 0:
        if source == 0:
            plt.title('Rainfall against pressure deficit of TC when making landfall in the Philippines, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 1:
            plt.title('Rainfall against pressure deficit of TC when making landfall in the Philippines, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 2:
            plt.title('Rainfall against pressure deficit of TC when making landfall in the Philippines, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
    else:
        if source == 0:
            plt.title('Rainfall against pressure deficit of TC when making landfall in Guangdong, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='x-small')
        elif source == 1:
            plt.title('Rainfall against pressure deficit of TC when making landfall in Guangdong, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='x-small')
        elif source == 2:
            plt.title('Rainfall against pressure deficit of TC when making landfall in Guangdong, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='x-small')
    plt.show()

    return logpdlist, lograinfalllist, slope, intercept, rvalue, pvalue, stderr

In [ ]:
tic = time.time()
plot_lograin_logpd(0, 0, True, True)
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
plot_lograin_logpd(0, 1, True, True)
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
plot_lograin_logpd(0, 2, True)
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
plot_lograin_logpd(1, 0, False, False)
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
plot_lograin_logpd(1, 1, False, False)
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
plot_lograin_logpd(1, 2, False, False)
toc = time.time()
print(toc-tic)

In [ ]:
#Plot log rain vs log R18
def plot_lograin_logr18(area, source, only_on_land = False, intersect = False):
    if area == 0:
        if source == 0:
            r18dict = r18dict_PHI_jtwc.copy()
        elif source == 1:
            r18dict = r18dict_PHI_cma.copy()
    else:
        if source == 0:
            r18dict = r18dict_GD_jtwc_2014.copy()
        elif source == 1:
            r18dict = r18dict_GD_cma.copy()

    if intersect == True:
        if area == 0:
            dict_a = r18dict_PHI_jtwc.copy()
            dict_b = r18dict_PHI_cma.copy()
        else:
            dict_a = r18dict_GD_jtwc_2014.copy()
            dict_b = r18dict_GD_cma.copy()
        for index in list(dict_a.keys()):
            if dict_a[index] == []:
                dict_a.pop(index,None)
        for index in list(dict_b.keys()):
            if dict_b[index] == []:
                dict_b.pop(index,None)
        key_a = list(dict_a.keys())
        key_b = list(dict_b.keys())
        key_intersect_list = np.intersect1d(key_a, key_b)
        print(key_intersect_list)
        tcindexes = np.asarray(key_intersect_list)        
    else:
        print(list(r18dict.keys()))
        tcindexes = np.asarray(list(r18dict.keys()), dtype=int)
    
    rainfall_total_list = []
    r18list = []
    for tc in tcindexes:
        print(tc)
        if r18dict[str(tc)] != []:
            r18 = r18dict[str(tc)][1] #Get TC R18 after landfall
            if type(r18) != str:
                r18list.append(r18)
                rainfall_list = rainfall_distribution(area, source, tc, only_on_land)[1]
                total_rainfall = sum(rainfall_list) #Total rainfall within 500km of TC centre
                rainfall_total_list.append(total_rainfall)
                
        
    r18list = np.asarray(r18list)
    rainfall_total_list = np.asarray(rainfall_total_list)
    
    logr18list = np.log(r18list)
    lograinfalllist = np.log(rainfall_total_list)
        
    plt.figure()
    plt.scatter(logr18list, lograinfalllist,color='b')
    slope, intercept, rvalue, pvalue, stderr = linregress(logr18list, lograinfalllist) #Linear fit
    xrange = np.linspace(min(logr18list),max(logr18list),1000)
    plt.plot(xrange, slope*xrange + intercept, 'r-') #Plot linear fit
    samplesize = len(lograinfalllist)
    
    plt.xlabel('log(R18)')
    if only_on_land == True:
        plt.ylabel('log(total volume of rain within 500km on land)')
    else:
        plt.ylabel('log(total volume of rain within 500km)')
    if area == 0:
        if source == 0:
            plt.title('Rainfall against R18 of TC when making landfall in the Philippines, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 1:
            plt.title('Rainfall against R18 of TC when making landfall in the Philippines, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 2:
            plt.title('Rainfall against R18 of TC when making landfall in the Philippines, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')

    else:
        if source == 0:
            plt.title('Rainfall against R18 of TC when making landfall in Guangdong, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 1:
            plt.title('Rainfall against R18 of TC when making landfall in Guangdong, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 2:
            plt.title('Rainfall against R18 of TC when making landfall in Guangdong, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
    plt.show()
    
    return logr18list, lograinfalllist, slope, intercept, rvalue, pvalue, stderr

In [ ]:
tic = time.time()
plot_lograin_logr18(0,0,False,True)
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
plot_lograin_logr18(0,1,False,True)
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
plot_lograin_logr18(1,0,False,False)
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
plot_lograin_logr18(1,1,False,False)
toc = time.time()
print(toc-tic)

In [ ]:
#Plot log rain vs (log pd & log r18)
def plot_lograin_logpdlogr18(area, source, only_on_land = False, intersect = False):
    if area == 0:
        if source == 0:
            pressuredict = landfalldata_pres_PHI2014_jtwc
            r18dict = r18dict_PHI_jtwc
        elif source == 1:
            pressuredict = landfalldata_pres_PHI2014_cma
            r18dict = r18dict_PHI_cma
        
    else:
        if source == 0:
            pressuredict = landfalldata_pres_GD2014_jtwc
            r18dict = r18dict_GD_jtwc_2014
        elif source == 1:
            pressuredict = landfalldata_pres_GD2014_cma
            r18dict = r18dict_GD_cma
            
    if intersect == True:
        if area == 0:
            dict_a = r18dict_PHI_jtwc
            dict_b = r18dict_PHI_cma
        else:
            dict_a = r18dict_GD_jtwc_2014
            dict_b = r18dict_GD_cma
        for index in list(dict_a.keys()):
            if dict_a[index] == []:
                dict_a.pop(index,None)
        for index in list(dict_b.keys()):
            if dict_b[index] == []:
                dict_b.pop(index,None)
        key_a = list(dict_a.keys())
        key_b = list(dict_b.keys())
        key_intersect_list = np.intersect1d(key_a, key_b)
        print(key_intersect_list)
        tcindexes = np.asarray(key_intersect_list)
    
    else:  
        print(list(r18dict.keys()))
        tcindexes = np.asarray(list(r18dict.keys()), dtype=int)
    
    rainfall_total_list = []
    r18list = []
    pdlist = []
    for tc in tcindexes:
        print(tc)
        if r18dict[str(tc)] != [] and tc != 12566 and tc != 12580: #No rainfall data for 12566 and 12580
            r18 = r18dict[str(tc)][1] #Get TC R18 after landfall
            if type(r18) != str:
                pressure = pressuredict[str(tc)][1] #Get TC pressure after landfall
                pd = 1010-pressure
                pdlist.append(pd)
                r18list.append(r18)
                rainfall_list = rainfall_distribution(area, source, tc, only_on_land)[1]
                total_rainfall = sum(rainfall_list) #Total rainfall within 500km of TC centre
                rainfall_total_list.append(total_rainfall)
                
    pdlist = np.asarray(pdlist)
    r18list = np.asarray(r18list)
    rainfall_total_list = np.asarray(rainfall_total_list)
    
    if intersect == False:
        if only_on_land == False:
            if area == 0:
                if source == 0:
                    slope_pd = 0.3201 #Use values obtained from previous individual plots
                    slope_r18 = 0.1615
                elif source == 1:
                    slope_pd = 0.2925
                    slope_r18 = 1.5331
            else:
                if source == 0:
                    slope_pd = 0.4518
                    slope_r18 = 1.7545
                elif source == 1:
                    slope_pd = 0.8237
                    slope_r18 = 1.7018
        else:
            if area == 0:
                if source == 0:
                    slope_pd = 0.3703
                    slope_r18 = -0.0532
                elif source == 1:
                    slope_pd = 0.3594
                    slope_r18 = 1.304
            else:
                if source == 0:
                    slope_pd = 0.5051 #Check
                    slope_r18 = 1.5141 #Check
                elif source == 1:
                    slope_pd = 0.5579
                    slope_r18 = 1.0801
    
    logpdlist = np.log(pdlist**slope_pd)
    logr18list = np.log(r18list**slope_r18)
    lograinfalllist = np.log(rainfall_total_list)
    
    loglist = logpdlist + logr18list
        
    plt.figure()
    plt.scatter(loglist, lograinfalllist,color='b')
    slope, intercept, rvalue, pvalue, stderr = linregress(loglist, lograinfalllist) #Linear fit
    xrange = np.linspace(min(loglist),max(loglist),1000)
    plt.plot(xrange, slope*xrange + intercept, 'r-') #Plot linear fit
    samplesize = len(lograinfalllist)
    plt.xlabel('log(R18^a) + log(pressure deficit^b)')
    plt.ylabel('log(total volume of rain within 500km)')
    if area == 0:
        if source == 0:
            plt.title('Rainfall against pressure deficit and R18 of TC when making landfall in the Philippines, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue) + ', a = ' + str(slope_pd) + ', b = ' + str(slope_r18),fontsize = 'x-small')
        elif source == 1:
            plt.title('Rainfall against pressure deficit and R18 of TC when making landfall in the Philippines, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue) + ', a = ' + str(slope_pd) + ', b = ' + str(slope_r18),fontsize = 'x-small')
        elif source == 2:
            plt.title('Rainfall against pressure deficit and R18 of TC when making landfall in the Philippines, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue) + ', a = ' + str(slope_pd) + ', b = ' + str(slope_r18),fontsize = 'x-small')

    else:
        if source == 0:
            plt.title('Rainfall against pressure deficit and R18 of TC when making landfall in Guangdong, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue) + ', a = ' + str(slope_pd) + ', b = ' + str(slope_r18),fontsize = 'x-small')
        elif source == 1:
            plt.title('Rainfall against pressure deficit and R18 of TC when making landfall in Guangdong, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue) + ', a = ' + str(slope_pd) + ', b = ' + str(slope_r18),fontsize = 'x-small')
        elif source == 2:
            plt.title('Rainfall against pressure deficit and R18 of TC when making landfall in Guangdong, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue) + ', a = ' + str(slope_pd) + ', b = ' + str(slope_r18),fontsize = 'x-small')
    plt.show()
    
    return loglist, lograinfalllist, slope, intercept, rvalue, pvalue, stderr

In [ ]:
tic = time.time()
plot_lograin_logpdlogr18(0,0,True)
plot_lograin_logpdlogr18(0,0,False)
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
plot_lograin_logpdlogr18(0,1,True)
plot_lograin_logpdlogr18(0,1,False)
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
#plot_lograin_logpdlogr18(1,0,True)
plot_lograin_logpdlogr18(1,0,False)
toc = time.time()
print(toc-tic)

In [ ]:
tic = time.time()
#plot_lograin_logpdlogr18(1,1,True)
plot_lograin_logpdlogr18(1,1,False)
toc = time.time()
print(toc-tic)

In [ ]:
#GD damage data from EM-DAT, figures taken under "China"
damage_GD = {} #Total deaths, total damage in USD, 0 means no data
damage_GD['8967'] = [188, 0] #Joe 1980
damage_GD['9304'] = [30, 0] #Ellen 1983, no data
# damage_GD['9319'] = 0 #Joe 1983, no data
# damage_GD['9386'] = 0 #Wynne 1984, no data
# damage_GD['9391'] = 0 #Betty 1984, no data
# damage_GD['9406'] = 0 #Gerald 1984, no data
# damage_GD['9415'] = 0 #June 1984, no data
# daamge_GD['9509'] = 0 #Hal 1985, no data
# damage_GD['9542'] = 0 #Tess 1985, no data
# damage_GD['9554'] = 0 #Winona 1985, no data
# damage_GD['9625'] = 0 #Mac 1986, no data
damage_GD['9634'] = [172, 38e7] #Peggy 1986
# damage_GD['9638'] = 0 #Not named 1986, no data
# damage_GD['9673'] = 0 #Ellen 1986, no data
# damage_GD['9728'] = 0 #Ruth 1987, no data?
# damage_GD['9782'] = 0 #Lynn 1987, no data
# damage_GD['9826'] = 0 #Vanessa 1988, no data
# damage_GD['9830'] = 0 #Warren 1988, no data
damage_GD['9874'] = [6, 0] #Kit 1989, no data?
# damage_GD['9875'] = 0 #Mamie:Mimie 1988, no data
damage_GD['9944'] = [84, 0] #Brenda 1989, no data?
# damage_GD['9961'] = 0 #Gordon 1989, no data
# damage_GD['10069'] = 0 #Nathan 1990, no data
damage_GD['10083'] = [108, 83e6] #Tasha 1990
damage_GD['10096'] = [144, 157000e3] #Yancy 1990
damage_GD['10192'] = [0, 0] #Brendan 1991, no data
damage_GD['10212'] = [0, 0] #Joel 1991, no data
damage_GD['10221'] = [0, 0] #Nat 1991, no data
damage_GD['10302'] = [0, 0] #Faye 1992, no data
damage_GD['10318'] = [0, 0] #Mark 1992, no data
damage_GD['10324'] = [0, 0] #Omar 1992, no data
damage_GD['10417'] = [10, 219670e3] #Koryn 1993
damage_GD['10436'] = [28, 433920e3] #Tasha 1993
damage_GD['10449'] = [10, 12610e3] #Abe 1993
damage_GD['10452'] = [7, 263670e3] #Becky 1993
damage_GD['10454'] = [1, 286370e3] #Dot 1993
damage_GD['10466'] = [0, 15890e3] #Ira 1993
damage_GD['10518'] = [74, 700e3] #Russ 1994
damage_GD['10520'] = [0, 0] #Sharon 1994, no data
damage_GD['10553'] = [0, 0] #Harry 1994, no data
damage_GD['10640'] = [26, 16e7] #Helen 1995
damage_GD['10645'] = [0, 0] #Irving 1995, no data
damage_GD['10653'] = [445, 87000e3] #Kent 1995
damage_GD['10657'] = [0, 0] #Nina 1995, no data
damage_GD['10669'] = [0, 0] #Sibyl 1995, no data
damage_GD['10675'] = [0, 0] #Ted 1995, no data
damage_GD['10779'] = [38, 1e8] #Willie 1996
damage_GD['10881'] = [65, 579700e3] #Victor 1997
damage_GD['10891'] = [7, 33e7] #Zita 1997
damage_GD['10980'] = [0, 0] #Penny 1998, no data
damage_GD['11077'] = [0, 0] #Maggie 1999, no data
damage_GD['11092'] = [0, 0] #TS0725 1999, no data
damage_GD['11106'] = [5, 18000e3] #Sam 1999
damage_GD['11115'] = [192, 277900e3] #Wendy 1999
damage_GD['11119'] = [18, 64300e3] #York 1999
damage_GD['11127'] = [0, 0] #Cam 1999, no data
damage_GD['11218'] = [47, 169e6] #Maria 2000
damage_GD['11290'] = [33, 2743000e3] #Utor 2001
damage_GD['11298'] = [0, 85000e3] #Yutu 2001
damage_GD['11313'] = [4, 213e6] #Fitow 2001
damage_GD['11318'] = [0, 0] #Nari 2001, no data
damage_GD['11410'] = [0, 0] #Kammuri 2002, no data
damage_GD['11417'] = [0, 0] #Vongfong 2002, no data
damage_GD['11431'] = [25, 0] #Hagupit 2002
damage_GD['11438'] = [0, 0] #Mekkhala 2002, no data
damage_GD['11508'] = [20, 1e8] #Imbudo 2003
damage_GD['11520'] = [0, 0] #Krovanh 2003, no data
damage_GD['11528'] = [38, 241000e3] #Dujuan 2003
damage_GD['11605'] = [0, 0] #Kompasu 2004, no data
damage_GD['11623'] = [2, 5000e3] #Aere 2004
damage_GD['11725'] = [0, 0] #Sanvu 2005, no data
damage_GD['11810'] = [23, 475000e3] #Chanchu 2006
damage_GD['11814'] = [0, 0] #Jelawat 2006, no data
damage_GD['11826'] = [89, 9e8] #Prapiroon 2006
damage_GD['11928'] = [0, 0] #Pabuk 2007, no data
damage_GD['11999'] = [25, 49000e3] #Neoguri 2008
damage_GD['12012'] = [14, 175000e3] #Fengshen 2008
damage_GD['12025'] = [0, 80000e3] #Kammuri 2008
damage_GD['12034'] = [4, 58000e3] #Nuri 2008
damage_GD['12048'] = [12, 824000e3] #Hagupit 2008
damage_GD['12052'] = [0, 0] #Higos 2008, no data
damage_GD['12110'] = [0, 0] #Nangka 2009, no data
damage_GD['12114'] = [0, 0] #Soudelor 2009, no data
damage_GD['12118'] = [0, 0] #Molave 2009, no data
damage_GD['12121'] = [9, 7000e3] #Goni 2009
damage_GD['12146'] = [13, 295001e3] #Koppu 2009
damage_GD['12215'] = [0, 0] #Chanthu 2010, no data
damage_GD['12227'] = [0, 0] #Lionrock 2010, no data
damage_GD['12238'] = [75, 298285e3] #Fanapi 2010
damage_GD['12294'] = [23, 0] #Sarika 2011
damage_GD['12296'] = [0, 0] #Haima 2011, no data
damage_GD['12393'] = [0, 0] #Doksuri 2012, no data
damage_GD['12399'] = [8, 329000e3] #Vincente 2012
damage_GD['12408'] = [2, 262000e3] #Kai-tak 2012
damage_GD['12485'] = [7, 177000e3] #Rumbia 2013
damage_GD['12498'] = [88, 2120000e3] #Utor 2013
damage_GD['12520'] = [20, 0] #Usagi 2013
damage_GD['12594'] = [0, 131000e3] #Hagibis 2014
damage_GD['12600'] = [71, 4232973e3] #Rammasun 2014
damage_GD['12621'] = [0, 0] #Fourteen TD0907 2014, no data
damage_GD['12624'] = [9, 2900000e3] #Kalmaegi 2014
damage_GD['12688'] = [0, 213000e3] #Linfa 2015
damage_GD['12733'] = [20, 4200000e3] #Mujigae 2015
damage_GD['12776'] = [0, 0] #TD0526  2016, no data
damage_GD['12793'] = [0, 123000e3] #Nida 2016
damage_GD['12837'] = [0, 0] #Haima 2016, no data
damage_GD['12861'] = [8, 3500000e3] #Hato 2017
damage_GD['12862'] = [0, 770000e3] #Mangkhut 2018

damage_GD_after2014 = {} #Total deaths, total damage in USD, 0 means no data
damage_GD_after2014['12594'] = [0, 131000e3] #Hagibis 2014
damage_GD_after2014['12600'] = [71, 4232973e3] #Rammasun 2014
damage_GD_after2014['12621'] = [0, 0] #Fourteen TD0907 2014, no data
damage_GD_after2014['12624'] = [9, 2900000e3] #Kalmaegi 2014
damage_GD_after2014['12688'] = [0, 213000e3] #Linfa 2015
damage_GD_after2014['12733'] = [20, 4200000e3] #Mujigae 2015
damage_GD_after2014['12776'] = [0, 0] #TD0526  2016, no data
damage_GD_after2014['12793'] = [0, 123000e3] #Nida 2016
damage_GD_after2014['12837'] = [0, 0] #Haima 2016, no data
damage_GD_after2014['12861'] = [8, 3500000e3] #Hato 2017
damage_GD_after2014['12862'] = [0, 770000e3] #Mangkhut 2018

In [ ]:
#Plot damage vs pd
def plot_damage_pd_GD(source, death_or_damage, after2014 = False, logdamage = False, logpd = False, intersect = False):
    if after2014 == False:
        damagedict = damage_GD
        if source == 0:
            pressuredict = landfalldata_pres_GD_jtwc
        elif source == 1:
            pressuredict = landfalldata_pres_GD_cma
        elif source == 2:
            pressuredict = landfalldata_pres_GD_wmo
    else:
        damagedict = damage_GD_after2014
        if source == 0:
            pressuredict = landfalldata_pres_GD2014_jtwc
        elif source == 1:
            pressuredict = landfalldata_pres_GD2014_cma
        elif source == 2:
            pressuredict = landfalldata_pres_GD2014_wmo
    
    if intersect == True:
        key_a = list(landfalldata_pres_GD_jtwc.keys())
        key_b = list(landfalldata_pres_GD_cma.keys())
        key_c = list(landfalldata_pres_GD_wmo.keys())
        key_intersect_list = np.intersect1d(np.intersect1d(key_a, key_b), key_c)
        
    else:
        key_intersect_list = pressuredict.keys()
        
    tcindexes = np.asarray(list(damagedict.keys()), dtype=int)
    
    damagelist = []
    pdlist = []
    for tc in tcindexes:
#        print(tc)
        if death_or_damage == 0: #Total deaths
            damage = damagedict[str(tc)][0]
        else:
            damage = damagedict[str(tc)][1]
        if damage != 0 and str(tc) in key_intersect_list:
            pres = pressuredict[str(tc)][1] #Get TC pressure after landfall
            pd = 1010-pres
            pdlist.append(pd)
            damagelist.append(damage)
                
        
    pdlist = np.asarray(pdlist)
    damagelist = np.asarray(damagelist)
    
    if logdamage == True:
        damagelist = np.log(damagelist)
    if logpd == True:
        pdlist = np.log(pdlist)
        
    plt.figure()
    plt.scatter(pdlist, damagelist,color='b')
    slope, intercept, rvalue, pvalue, stderr = linregress(pdlist, damagelist) #Linear fit
    xrange = np.linspace(min(pdlist),max(pdlist),1000)
    plt.plot(xrange, slope*xrange + intercept, 'r-') #Plot linear fit
    samplesize = len(damagelist)
    if logpd == True:
        plt.xlabel('log(pressure deficit)')
    else:
        plt.xlabel('Pressure deficit (hPa)')
    if death_or_damage == 0:
        if logdamage == True:
            plt.ylabel('log(total deaths)')
        else:
            plt.ylabel('Total deaths')
    else:
        if logdamage == True:
            plt.ylabel('log(damage)')
        else:
            plt.ylabel('Damage (USD)')

    if death_or_damage == 0:
        if source == 0:
            plt.title('Total deaths against pressure deficit of TC when making landfall in Guangdong, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 1:
            plt.title('Total deaths against pressure deficit of TC when making landfall in Guangdong, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 2:
            plt.title('Total deaths against pressure deficit of TC when making landfall in Guangdong, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
    else:
        if source == 0:
            plt.title('Damage against pressure deficit of TC when making landfall in Guangdong, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 1:
            plt.title('Damage against pressure deficit of TC when making landfall in Guangdong, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 2:
            plt.title('Damage against pressure deficit of TC when making landfall in Guangdong, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')    
    plt.show()
    
    return pdlist, damagelist, slope, intercept, rvalue, pvalue, stderr

In [ ]:
tic = time.time()
# plot_damage_pd_GD(0)
# plot_damage_pd_GD(1)
# plot_damage_pd_GD(2)
# plot_damage_pd_GD(0,1,False,logpd=False,logdamage=False,False)
# plot_damage_pd_GD(1,1,False,False,False,False)
# plot_damage_pd_GD(2,1,False,False,False,False)
plot_damage_pd_GD(0,0,False,logdamage=True,logpd=True, intersect = False)
plot_damage_pd_GD(1,0,False,True,True,False)
plot_damage_pd_GD(2,0,False,True,True,False)
# plot_damage_pd_GD(0, False, True)
# plot_damage_pd_GD(1, False, True)
# plot_damage_pd_GD(2, False, True)
toc = time.time()
print(toc-tic)

In [ ]:
#Plot damage vs wind
def plot_damage_wind_GD(source, death_or_damage, after2014 = False, logdamage = False, logwind = False, intersect = False):
    if after2014 == False:
        damagedict = damage_GD
        if source == 0:
            winddict = landfalldata_wind_GD_jtwc
        elif source == 1:
            winddict = landfalldata_wind_GD_cma
        elif source == 2:
            winddict = landfalldata_wind_GD_wmo
    else:
        damagedict = damage_GD_after2014
        if source == 0:
            winddict = landfalldata_wind_GD2014_jtwc
        elif source == 1:
            winddict = landfalldata_wind_GD2014_cma
        elif source == 2:
            winddict = landfalldata_wind_GD2014_wmo
            
    if intersect == True:
        key_a = list(landfalldata_wind_GD_jtwc.keys())
        key_b = list(landfalldata_wind_GD_cma.keys())
        key_c = list(landfalldata_wind_GD_wmo.keys())
        key_intersect_list = np.intersect1d(np.intersect1d(key_a, key_b), key_c)
        
    else:
        key_intersect_list = winddict.keys()
        
    
#    print(list(damagedict.keys()))
    tcindexes = np.asarray(list(damagedict.keys()), dtype=int)
    
    damagelist = []
    windlist = []
    for tc in tcindexes:
        if death_or_damage == 0:
            damage = damagedict[str(tc)][0]
        else:
            damage = damagedict[str(tc)][1]
        if damage != 0 and str(tc) in key_intersect_list:
            if len(winddict[str(tc)]) == 1:
                wind = winddict[str(tc)][0] #Get TC wind speed after landfall
            elif len(winddict[str(tc)]) == 2:
                wind = winddict[str(tc)][1] #Get TC wind speed after landfall
            windlist.append(wind)
            damagelist.append(damage)
                
        
    windlist = np.asarray(windlist)
    damagelist = np.asarray(damagelist)
    
    if logwind == True:
        windlist = np.log(windlist)
    if logdamage == True:
        damagelist = np.log(damagelist)



    plt.figure()
    plt.scatter(windlist, damagelist,color='b')
    slope, intercept, rvalue, pvalue, stderr = linregress(windlist, damagelist) #Linear fit
    xrange = np.linspace(min(windlist),max(windlist),1000)
    plt.plot(xrange, slope*xrange + intercept, 'r-') #Plot linear fit
    samplesize = len(damagelist)
    
    if logwind == True:
        plt.xlabel('log(wind speed)')
    else:
        plt.xlabel('Wind speed (kts)')
    if logdamage == True:
        if death_or_damage == 0:
            plt.ylabel('log(total deaths)')
        else:
            plt.ylabel('log(damage)')
    else:
        if death_or_damage == 0:
            plt.ylabel('Total deaths')
        else:
            plt.ylabel('Damage (USD)')

    if death_or_damage == 0:
        if source == 0:
            plt.title('Total deaths against maximum wind speed of TC when making landfall in Guangdong, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 1:
            plt.title('Total deaths against maximum wind speed of TC when making landfall in Guangdong, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 2:
            plt.title('Total deaths against maximum wind speed of TC when making landfall in Guangdong, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
    else:
        if source == 0:
            plt.title('Damage against maximum wind speed of TC when making landfall in Guangdong, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 1:
            plt.title('Damage against maximum wind speed of TC when making landfall in Guangdong, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 2:
            plt.title('Damage against maximum wind speed of TC when making landfall in Guangdong, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')

    plt.show()
    
    return windlist, damagelist, slope, intercept, rvalue, pvalue, stderr

In [ ]:
tic = time.time()
plot_damage_wind_GD(0, 0,False, logdamage = True, logwind = True,intersect = False)
plot_damage_wind_GD(1, 0,False, True, True, False)
plot_damage_wind_GD(2, 0,False, True, True, False)
# plot_damage_wind_GD(0, 0,False, True, True,False)
# plot_damage_wind_GD(1, 0,False, False, True,False)
# plot_damage_wind_GD(2, 0,False, False, True,False)
toc = time.time()
print(toc-tic)

In [ ]:
#Plot damage vs rainfall
def plot_damage_rainfall_GD(source, death_or_damage, only_on_land = False, logdamage = False, lograin = False):
    damagedict = damage_GD_after2014
    print(list(damagedict.keys()))
    tcindexes = np.asarray(list(damagedict.keys()), dtype=int)
    
    damagelist = []
    rainfall_total_list = []
    for tc in tcindexes:
        print(tc)
        if death_or_damage == 0:
            damage = damagedict[str(tc)][0]
        else:
            damage = damagedict[str(tc)][1]
        if damage != 0:
            if (source == 0 and str(tc) in landfalldata_coord_GD2014_jtwc.keys()) or (source == 1 and str(tc) in landfalldata_coord_GD2014_cma.keys()) :
                damagelist.append(damage)
                rainfall_list = rainfall_distribution(1, source, tc, only_on_land)[1]
                total_rainfall = sum(rainfall_list) #Total rainfall within 500km of TC centre
                rainfall_total_list.append(total_rainfall)
                
        
    rainfall_total_list = np.asarray(rainfall_total_list)
    damagelist = np.asarray(damagelist)
    
    if lograin == True:
        rainfall_total_list = np.log(rainfall_total_list)
    if logdamage == True:
        damagelist = np.log(damagelist)



    plt.figure()
    plt.scatter(rainfall_total_list, damagelist,color='b')
    slope, intercept, rvalue, pvalue, stderr = linregress(rainfall_total_list, damagelist) #Linear fit
    xrange = np.linspace(min(rainfall_total_list),max(rainfall_total_list),1000)
    plt.plot(xrange, slope*xrange + intercept, 'r-') #Plot linear fit
    samplesize = len(damagelist)
    
    if lograin == True:
        if only_on_land == True:
            plt.xlabel('log(total volume of rain within 500km on land)')
        else:
            plt.xlabel('log(total volume of rain within 500km)')
    else:
        if only_on_land == True:
            plt.xlabel('Total volume of rain within 500km on land (mm)')
        else:
            plt.xlabel('Total volume of rain within 500km (mm)')
    if logdamage == True:
        if death_or_damage == 0:
            plt.ylabel('log(total deaths)')
        else:
            plt.ylabel('log(damage)')
    else:
        if death_or_damage == 0:
            plt.ylabel('Total deaths')
        else:
            plt.ylabel('Damage (USD)')

    if death_or_damage == 0:
        if source == 0:
            plt.title('Total deaths against rainfall of TC when making landfall in Guangdong, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 1:
            plt.title('Total deaths against rainfall of TC when making landfall in Guangdong, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 2:
            plt.title('Total deaths against rainfall of TC when making landfall in Guangdong, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
    else:
        if source == 0:
            plt.title('Damage against rainfall of TC when making landfall in Guangdong, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 1:
            plt.title('Damage against rainfall of TC when making landfall in Guangdong, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 2:
            plt.title('Damage against rainfall of TC when making landfall in Guangdong, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')

    plt.show()
    
    return rainfall_total_list, damagelist, slope, intercept, rvalue, pvalue, stderr

In [ ]:
tic - time.time() #Expected runtime ~3-5 min for each graph
plot_damage_rainfall_GD(source=0, death_or_damage=0, only_on_land = False, logdamage = True, lograin = True)
plot_damage_rainfall_GD(1, 0, only_on_land = False, logdamage = True, lograin = True)
# plot_damage_rainfall_GD(source=0, death_or_damage=1, only_on_land = False, logdamage = False, lograin = False)
# plot_damage_rainfall_GD(1, 1, only_on_land = False, logdamage = False, lograin = False)
toc = time.time()
print(toc-tic)

In [ ]:
#Plot damage vs r18
def plot_damage_r18_GD(source, death_or_damage, logscale = False, intersect = False):
    damagedict = damage_GD
    if source == 0:
        r18dict = r18dict_GD_jtwc
    elif source == 1:
        r18dict = r18dict_GD_cma

    if intersect == True:
        key_a = list(r18dict_GD_jtwc.keys())
        key_b = list(r18dict_GD_cma.keys())
        key_intersect_list = np.intersect1d(key_a, key_b)
        
    else:
        key_intersect_list = r18dict.keys()
        
#    print(list(damagedict.keys()))
    tcindexes = np.asarray(list(damagedict.keys()), dtype=int)
    
    damagelist = []
    r18list = []
    for tc in tcindexes:
#        print(tc)
        if death_or_damage == 0:
            damage = damagedict[str(tc)][0]
        else:
            damage = damagedict[str(tc)][1]
        if str(tc) in key_intersect_list:
            if r18dict[str(tc)] != []:
                r18 = r18dict[str(tc)][1] #Get TC R18 after landfall
                if type(r18) != str and damage != 0 and str(tc) in r18dict.keys():
                    r18list.append(r18)
                    damagelist.append(damage)
        
    r18list = np.asarray(r18list)
    damagelist = np.asarray(damagelist)
    
    if logscale == True:
        r18list = np.log(r18list)
        damagelist = np.log(damagelist)



    plt.figure()
    plt.scatter(r18list, damagelist,color='b')
    slope, intercept, rvalue, pvalue, stderr = linregress(r18list, damagelist) #Linear fit
    xrange = np.linspace(min(r18list),max(r18list),1000)
    plt.plot(xrange, slope*xrange + intercept, 'r-') #Plot linear fit
    samplesize = len(damagelist)
    
    if logscale == True:
        if death_or_damage == 0:
            plt.xlabel('log(R18)')
            plt.ylabel('log(total deaths)')
        else:
            plt.xlabel('log(R18)')
            plt.ylabel('log(damage)')
    else:
        if death_or_damage == 0:
            plt.xlabel('R18 (km)')
            plt.ylabel('Total deaths')
        else:
            plt.xlabel('R18 (km)')
            plt.ylabel('Damage (USD)')
    if death_or_damage == 0:
        if source == 0:
            plt.title('Total deaths against R18 of TC when making landfall in Guangdong, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 1:
            plt.title('Total deaths against R18 of TC when making landfall in Guangdong, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 2:
            plt.title('Total deaths against R18 of TC when making landfall in Guangdong, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
    else:
        if source == 0:
            plt.title('Total damage against R18 of TC when making landfall in Guangdong, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 1:
            plt.title('Total damage against R18 of TC when making landfall in Guangdong, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 2:
            plt.title('Total damage against R18 of TC when making landfall in Guangdong, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
    
    plt.show()
    
    return r18list, damagelist, slope, intercept, rvalue, pvalue, stderr

In [ ]:
tic = time.time()
#plot_damage_r18_GD(0,1,False,True)
plot_damage_r18_GD(0,0,True,False)
#plot_damage_r18_GD(1,1,False,True)
plot_damage_r18_GD(1,0,True,False)
toc = time.time()
print(toc-tic)

In [ ]:
#Plot damage against numerous variables combined
def plot_damage_pdwindrainr18(variables,source,death_or_damage,only_on_land=False,logdamage=False,logvariables=False,intersect=False):
    #Variables should be a string list, e.g. ['pd', 'wind', 'rain']
    #logdamage/logvariable: 'True' if plot in log scale, 'False' otherwise
    if 'rain' in variables:
        damagedict = damage_GD_after2014 #No rainfall data before 2014
    else:
        damagedict = damage_GD
        
    if source == 0:
        pressuredict = landfalldata_pres_GD_jtwc
        winddict = landfalldata_wind_GD_jtwc
        r18dict = r18dict_GD_jtwc
    elif source == 1:
        pressuredict = landfalldata_pres_GD_cma
        winddict = landfalldata_wind_GD_cma
        r18dict = r18dict_GD_cma
    elif source == 2:
        pressuredict = landfalldata_pres_GD_wmo
        winddict = landfalldata_wind_GD_wmo
        r18dict = {}
   
    tcindexes = np.asarray(list(damagedict.keys()), dtype=int)
    print(tcindexes)
    
    pdlist = []
    windlist = []
    rainfall_total_list = []
    r18list = []
    damagelist = []
    for tc in tcindexes:
        print(tc)
        if death_or_damage == 0:
            damage = damagedict[str(tc)][0]
        else:
            damage = damagedict[str(tc)][1]
        if damage != 0:
            if str(tc) in winddict.keys() and str(tc) in r18dict.keys() and str(tc) in pressuredict.keys():
            #if True:
                if 'r18' in variables:
                    if r18dict[str(tc)] != []:
                        r18 = r18dict[str(tc)][1] #Get TC R18 after landfall
                        if type(r18) != str:
                            r18list.append(r18)
                        else:
                            r18list.append(0)
                    else:
                        r18list.append(0)
                if 'pd' in variables:
                    pres = pressuredict[str(tc)][1] #Get TC pressure after landfall
                    pd = 1010-pres
                    pdlist.append(pd)
                if 'wind' in variables:
                    wind = winddict[str(tc)][1]
                    windlist.append(wind)
                if 'rain' in variables:
                    rainfall_list = rainfall_distribution(1, source, tc, only_on_land)[1]
                    total_rainfall = sum(rainfall_list) #Total rainfall within 500km of TC centre
                    rainfall_total_list.append(total_rainfall) 
                damagelist.append(damage)
                
    
    if 'r18' in variables:
        r18_zeros = np.where(np.asarray(r18list) == 0) #Remove R18 missing data
        r18_zeros = list(r18_zeros[0])

        if r18_zeros != []:
            damagelist_index = [i for i in range(len(damagelist)) if i not in r18_zeros]
            damagelist_new = []
            for i in damagelist_index:
                damagelist_new.append(damagelist[i])
            damagelist = damagelist_new
            r18list_index = [i for i in range(len(r18list)) if i not in r18_zeros]
            r18list_new = []
            for i in r18list_index:
                r18list_new.append(r18list[i])
            r18list = r18list_new
            if 'pd' in variables:
                pdlist_index = [i for i in range(len(pdlist)) if i not in r18_zeros]
                pdlist_new = []
                for i in pdlist_index:
                    pdlist_new.append(pdlist[i])
                pdlist = pdlist_new
            if 'rain' in variables:
                rainfall_total_list = [x for x in rainfall_total_list if rainfall_total_list.index(x) not in r18_zeros]
            if 'wind' in variables:
                windlist_index = [i for i in range(len(windlist)) if i not in r18_zeros]
                windlist_new = []
                for i in windlist_index:
                    windlist_new.append(windlist[i])
                windlist = windlist_new

    
    pdlist = np.asarray(pdlist)
    windlist = np.asarray(windlist)
    rainfall_total_list = np.asarray(rainfall_total_list)
    r18list = np.asarray(r18list)
    
    variableslist = 0
    if logvariables == True:
        if 'pd' in variables:
            if death_or_damage == 0:
                if source == 0:
                    slope_pd = 0.3227 #Slope obtained from previous individual plots
                elif source == 1:
                    slope_pd = 0.4848         
            elif death_or_damage == 1:
                if source == 0:
                    slope_pd = 2.5911
                else:
                    slope_pd = 2.6996
            pdlist = np.log(pdlist**slope_pd)
            variableslist += pdlist
        if 'wind' in variables:
            if death_or_damage == 0:
                if source == 0:
                    slope_wind = -0.0216
                elif source == 1:
                    slope_wind = 0.8079
            elif death_or_damage == 1:
                if source == 0:
                    slope_wind = 2.8713
                elif source == 1:
                    slope_wind = 4.3707
            windlist = np.log(windlist**slope_wind)
            variableslist += windlist
        if 'rain' in variables:
            if death_or_damage == 0:
                if source == 0:
                    slope_rain = 0.1287
                elif source == 1:
                    slope_rain = 0.1275
            elif death_or_damage == 1:
                if source == 0:
                    slope_rain = 1.569
                elif source == 1:
                    slope_rain = 1.4513
            rainfall_total_list = np.log(rainfall_total_list**slope_rain)
            variableslist += rainfall_total_list
        if 'r18' in variables:
            if death_or_damage == 0:
                if source == 0:
                    slope_r18 = 1.406
                else:
                    slope_r18 = -4.1063
            elif death_or_damage == 1:
                if source == 0:
                    slope_r18 = 2.2525
                else:
                    slope_r18 = 2.9373
            r18list = np.log(r18list**slope_r18)
            variableslist += r18list
    if logdamage == True:
        damagelist = np.log(damagelist)

    
    plt.figure()
    plt.scatter(variableslist, damagelist,color='b')
    slope, intercept, rvalue, pvalue, stderr = linregress(variableslist, damagelist) #Linear fit
    xrange = np.linspace(min(variableslist),max(variableslist),100)
    plt.plot(xrange, slope*xrange + intercept, 'r-') #Plot linear fit
    samplesize = len(variableslist)
    
    if logvariables == True:
        plt.xlabel('log(sum of variables)')
    else:
        plt.xlabel('Sum of variables')
    if death_or_damage == 1:
        if logdamage == True:
            plt.ylabel('log(damage)')
        else:
            plt.ylabel('Damage (USD)')
    else:
        if logdamage == True:
            plt.ylabel('log(total deaths)')
        else:
            plt.ylabel('Total deaths')
    if death_or_damage == 0:
        if source == 0:
            plt.title('Total deaths against ' + str(variables) + ' of TC when making landfall in Guangdong, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 1:
            plt.title('Total deaths against ' + str(variables) + ' of TC when making landfall in Guangdong, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 2:
            plt.title('Total deaths against ' + str(variables) + ' of TC when making landfall in Guangdong, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')

    else:
        if source == 0:
            plt.title('Damage against ' + str(variables) + ' of TC when making landfall in Guangdong, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 1:
            plt.title('Damage against ' + str(variables) + ' of TC when making landfall in Guangdong, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
        elif source == 2:
            plt.title('Damage against ' + str(variables) + ' of TC when making landfall in Guangdong, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize = 'x-small')
    plt.show()
    
    return variableslist, damagelist, slope, intercept, rvalue, pvalue, stderr

In [ ]:
tic = time.time() #Expect longer runtime if 'rain' involved
plot_damage_pdwindrainr18(variables=['pd','rain'],source=0,logvariables=True,logdamage=True,intersect=False,death_or_damage=1,only_on_land=False)
plot_damage_pdwindrainr18(variables=['wind','rain'],source=0,logvariables=True,logdamage=True,intersect=False,death_or_damage=1,only_on_land=False)
#plot_damage_pdwindrainr18(variables=['pd','rain','r18'],source=0,logvariables=True,logdamage=True,intersect=False,death_or_damage=1,only_on_land=False)
#plot_damage_pdwindrainr18(variables=['wind','rain','r18'],source=0,logvariables=True,logdamage=True,intersect=False,death_or_damage=1,only_on_land=False)
toc = time.time()
print(toc-tic)

In [ ]:
#Plot pd vs max wind speed
def plot_pd_windspeed(area, source, logscale = False, intersect = False):
    windspeed_dict = get_landfall_data_beforeandafter(area, source, False)[1]
    pressure_dict = get_landfall_data_beforeandafter(area,source, False)[0]
    windspeed_before_list = []
    windspeed_after_list = []
    pres_before_list = []
    pres_after_list = []
    key_intersect1_list = np.intersect1d(list(get_landfall_data_beforeandafter(area,0,False)[1].keys()), list(get_landfall_data_beforeandafter(area,1,False)[1].keys()))
    key_intersect2_list = np.intersect1d(key_intersect1_list, list(get_landfall_data_beforeandafter(area,2,False)[1].keys()))
    key_intersect3_list = np.intersect1d(key_intersect2_list, list(get_landfall_data_beforeandafter(area,0,False)[0].keys()))
    key_intersect4_list = np.intersect1d(key_intersect3_list, list(get_landfall_data_beforeandafter(area,1,False)[0].keys()))
    key_intersect5_list = np.intersect1d(key_intersect4_list, list(get_landfall_data_beforeandafter(area,2,False)[0].keys()))
    
    if intersect == False:
        pres_wind_intersect = np.intersect1d(list(pressure_dict.keys()),list(windspeed_dict.keys()))
    else:
        pres_wind_intersect = key_intersect5_list
        
    for key in pres_wind_intersect:
        if len(windspeed_dict[key]) == 1 and windspeed_dict[key][0] != 0: 
            pres_after = pressure_dict[key][0]
            pd = 1010-pres_after
            if pd == 0:
                pres_after_list.append(1) #Prevent log(0) --> this will give log(pd) = 0
            else:
                pres_after_list.append(pd)
            windspeed_after_list. append(windspeed_dict[key][0])
        elif len(windspeed_dict[key]) != 1 and windspeed_dict[key][1] != 0: 
            windspeed_before_list.append(windspeed_dict[key][0])
            windspeed_after_list.append(windspeed_dict[key][1])  
            pres_before = pressure_dict[key][0]
            pres_after = pressure_dict[key][1]
            pdbefore = 1010-pres_before
            pdafter = 1010-pres_after
            if pdbefore == 0:
                pres_before_list.append(1)
            else:
                pres_before_list.append(pdbefore)
            if pdafter == 0:
                pres_after_list.append(1)
            else:
                pres_after_list.append(pdafter)
    
    pres_before_list = np.asarray(pres_before_list)
    pres_after_list = np.asarray(pres_after_list)
    windspeed_before_list = np.asarray(windspeed_before_list)
    windspeed_after_list = np.asarray(windspeed_after_list)
    
    if logscale == True:
        pres_before_list = np.log(pres_before_list)
        pres_after_list = np.log(pres_after_list)
        windspeed_before_list = np.log(windspeed_before_list)
        windspeed_after_list = np.log(windspeed_after_list)
         
    plt.figure()
    slope, intercept, rvalue, pvalue, stderr = linregress(pres_after_list, windspeed_after_list) #Linear fit
    xrange = np.linspace(min(pres_after_list),max(pres_after_list),1000)
    plt.plot(xrange, slope*xrange + intercept, 'r-') #Plot linear fit
    samplesize = len(pres_after_list)
    
    if logscale == True:
        plt.xlabel('log(pressure deficit)')
        plt.ylabel('log(wind speed)') 
    else:
        plt.xlabel('Pressure deficit(hPa)')
        plt.ylabel('Wind Speed (kt)') 
    plt.scatter(pres_after_list, windspeed_after_list,color = 'k') 
    if area == 0:
        if source == 0:
            plt.title('Maximum wind speed against central pressure deficit of typhoons at their landfalls in Philippines, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='small')
        elif source == 1:
            plt.title('Maximum wind speed against central pressure deficit of typhoons at their landfalls in Philippines, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='small')
        elif source == 2:
            plt.title('Maximum wind speed against central pressure deficit of typhoons at their landfalls in Philippines, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='small')
    else:    
        if source == 0:
            plt.title('Maximum wind speed against central pressure deficit of typhoons at their landfalls in GD, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='small')
        elif source == 1:
            plt.title('Maximum wind speed against central pressure deficit of typhoons at their landfalls in GD, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='small')
        elif source == 2:
            plt.title('Maximum wind speed against central pressure deficit of typhoons at their landfalls in GD, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='small')
    plt.show()

In [ ]:
plot_pd_windspeed(1, 0, True, True)
plot_pd_windspeed(1, 1, True, True)
plot_pd_windspeed(1, 2, True, True)

In [ ]:
'''INCOMPLETE'''
#Plot r18 vs max wind speed
def plot_r18_windspeed(area, source, logscale = False):
    windspeed_dict = get_landfall_data_beforeandafter(area, source, False)[1]
    if area == 0:
        if source == 0:
            r18_dict = r18dict_PHI_jtwc
        else:
            r18_dict = r18dict_PHI_cma
    else:
        if source == 0:
            r18_dict = r18dict_GD_jtwc
        else:
            r18_dict = r18dict_GD_cma
    windspeed_before_list = []
    windspeed_after_list = []
    pres_before_list = []
    pres_after_list = []
    key_intersect1_list = np.intersect1d(list(get_landfall_data_beforeandafter(area,0,False)[1].keys()), list(get_landfall_data_beforeandafter(area,1,False)[1].keys()))
    key_intersect2_list = np.intersect1d(key_intersect1_list, list(get_landfall_data_beforeandafter(area,2,False)[1].keys()))
    key_intersect3_list = np.intersect1d(key_intersect2_list, list(get_landfall_data_beforeandafter(area,0,False)[0].keys()))
    key_intersect4_list = np.intersect1d(key_intersect3_list, list(get_landfall_data_beforeandafter(area,1,False)[0].keys()))
    key_intersect5_list = np.intersect1d(key_intersect4_list, list(get_landfall_data_beforeandafter(area,2,False)[0].keys()))
    for key in key_intersect5_list:
        if len(windspeed_dict[key]) == 1: 
            windspeed_after_list. append(windspeed_dict[key][0])
        else: 
            windspeed_before_list.append(windspeed_dict[key][0])
            windspeed_after_list.append(windspeed_dict[key][1])  
        if len(pressure_dict[key]) == 1: 
            pres_after = pressure_dict[key][0]
            pres_after_list.append(1010-pres_after)
        else: 
            pres_before = pressure_dict[key][0]
            pres_after = pressure_dict[key][1]
            pres_before_list.append(1010-pres_before)
            pres_after_list.append(1010-pres_after)
    
    pres_before_list = np.asarray(pres_before_list)
    pres_after_list = np.asarray(pres_after_list)
    windspeed_before_list = np.asarray(windspeed_before_list)
    windspeed_after_list = np.asarray(windspeed_after_list)
    
    if logscale == True:
        pres_before_list = np.log(pres_before_list)
        pres_after_list = np.log(pres_after_list)
        windspeed_before_list = np.log(windspeed_before_list)
        windspeed_after_list = np.log(windspeed_after_list)
         
    plt.figure()
    slope, intercept, rvalue, pvalue, stderr = linregress(pres_after_list, windspeed_after_list) #Linear fit
    xrange = np.linspace(min(pres_after_list),max(pres_after_list),1000)
    plt.plot(xrange, slope*xrange + intercept, 'r-') #Plot linear fit
    samplesize = len(pres_after_list)
    if logscale == True:
        plt.xlabel('log(pressure deficit)')
        plt.ylabel('log(wind speed)') 
    else:
        plt.xlabel('Pressure deficit(hPa)')
        plt.ylabel('Wind Speed (kt)') 
    plt.scatter(pres_after_list, windspeed_after_list,color = 'k') 
    if area == 0:
        if source == 0:
            plt.title('Maximum wind speed against central pressure deficit of typhoons at their landfalls in Philippines, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='small')
        elif source == 1:
            plt.title('Maximum wind speed against central pressure deficit of typhoons at their landfalls in Philippines, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='small')
        elif source == 2:
            plt.title('Maximum wind speed against central pressure deficit of typhoons at their landfalls in Philippines, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='small')
    else:    
        if source == 0:
            plt.title('Maximum wind speed against central pressure deficit of typhoons at their landfalls in GD, source: JTWC \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='small')
        elif source == 1:
            plt.title('Maximum wind speed against central pressure deficit of typhoons at their landfalls in GD, source: CMA \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='small')
        elif source == 2:
            plt.title('Maximum wind speed against central pressure deficit of typhoons at their landfalls in GD, source: WMO \n sample size = ' + str(samplesize) + ', slope = ' + str(slope) + r'$\pm$' + str(stderr) + ', intercept = ' + str(intercept) + '\n r-value = ' + str(rvalue) + ', R-squared = ' + str(rvalue**2) + ', p-value = ' + str(pvalue),fontsize='small')
    plt.show()

In [ ]:
#Plot all variables on same plot (for GD)
def plot_variables_sameplot(source, dvariable, area = 1, ivariables = ['pd', 'wind', 'rain', 'r18']):
#dvariable is dependant variable, takes either 'rain', 'damage' or 'death'
#ivariables is list of independent variables, takes a list with elements 'pd', 'wind', 'rain' and/or 'r18'

    plt.figure()
    if dvariable == 'rain': #Plotting rain against variables
        
        if 'pd' in ivariables: #Include pressure deficit in plot
            if area == 0:
                if source == 0:
                    pressuredict = landfalldata_pres_PHI2014_jtwc.copy()
                elif source == 1:
                    pressuredict = landfalldata_pres_PHI2014_cma.copy()
                elif source == 2:
                    pressuredict = landfalldata_pres_PHI2014_wmo.copy()

                pressuredict.pop('12566', None) #No rainfall data for 12566
                pressuredict.pop('12580', None) #No rainfall data for 12580
            else:
                if source == 0:
                    pressuredict = landfalldata_pres_GD2014_jtwc.copy()
                elif source == 1:
                    pressuredict = landfalldata_pres_GD2014_cma.copy()
                elif source == 2:
                    pressuredict = landfalldata_pres_GD2014_wmo.copy()

                pressuredict.pop('12860', None) #12860 rainfall data not included
                
            print(list(pressuredict.keys()))
            tcindexes_pres = np.asarray(list(pressuredict.keys()), dtype=int)
            
            pdlist = []
            rainfall_total_list = []
            
            for tc in tcindexes_pres:
                print(tc)
                rainfall_list = rainfall_distribution(area, source, tc, only_on_land=False)[1]
                total_rainfall = sum(rainfall_list) #Total rainfall within 500km of TC centre
                rainfall_total_list.append(total_rainfall)
                pressure = pressuredict[str(tc)][1] #Get TC pressure after landfall
                pd = 1011-pressure #Reference set to 1020 to prevent log(0)
                pdlist.append(pd)
                
            pdlist = np.asarray(pdlist)
            rainfall_total_list = np.asarray(rainfall_total_list)

            logpdlist = np.log(pdlist)
            lograinfalllist = np.log(rainfall_total_list)

            plt.scatter(logpdlist, lograinfalllist,color='r',label = 'Pressure deficit (hPa)')
            slope, intercept, rvalue, pvalue, stderr = linregress(logpdlist, lograinfalllist) #Linear fit
            xrange = np.linspace(min(logpdlist),max(logpdlist),1000)
            plt.plot(xrange, slope*xrange + intercept, 'r--') #Plot linear fit
#             samplesize = len(lograinfalllist)
                
                
        if 'r18' in ivariables: #Include R18 in plot
            if area == 0:
                if source == 0:
                    r18dict = r18dict_PHI_jtwc.copy()
                elif source == 1:
                    r18dict = r18dict_PHI_cma.copy()
            else:
                if source == 0:
                    r18dict = r18dict_GD_jtwc_2014.copy()
                elif source == 1:
                    r18dict = r18dict_GD_cma.copy()
                    
            print(list(r18dict.keys()))
            tcindexes_r18 = np.asarray(list(r18dict.keys()), dtype=int)
            
            rainfall_total_list = []
            r18list = []
            
            for tc in tcindexes_r18:
                print(tc)
                if r18dict[str(tc)] != []:
                    r18 = r18dict[str(tc)][1] #Get TC R18 after landfall
                    if type(r18) != str:
                        r18list.append(r18)
                        rainfall_list = rainfall_distribution(area, source, tc, only_on_land=False)[1]
                        total_rainfall = sum(rainfall_list) #Total rainfall within 500km of TC centre
                        rainfall_total_list.append(total_rainfall)
            
            r18list = np.asarray(r18list)
            rainfall_total_list = np.asarray(rainfall_total_list)

            logr18list = np.log(r18list)
            lograinfalllist = np.log(rainfall_total_list)

            plt.scatter(logr18list, lograinfalllist,color='b',label = 'R18 (km)')
            slope, intercept, rvalue, pvalue, stderr = linregress(logr18list, lograinfalllist) #Linear fit
            xrange = np.linspace(min(logr18list),max(logr18list),1000)
            plt.plot(xrange, slope*xrange + intercept, 'b--') #Plot linear fit
#             samplesize = len(lograinfalllist)

    elif dvariable == 'damage' or dvariable == 'deaths': #Plot damage or deaths
        if area == 1:
            if 'pd' in ivariables:
                damagedict = damage_GD
                if source == 0:
                    pressuredict = landfalldata_pres_GD_jtwc
                elif source == 1:
                    pressuredict = landfalldata_pres_GD_cma
                elif source == 2:
                    pressuredict = landfalldata_pres_GD_wmo
                    
                tcindexes = np.asarray(list(damagedict.keys()), dtype=int)
    
                damagelist = []
                pdlist = []
                for tc in tcindexes:
                    if dvariable == 'damage':
                        damage = damagedict[str(tc)][1] #Get damage in USD
                    elif dvariable == 'deaths':
                        damage = damagedict[str(tc)][0] #Get death no.
                    if damage != 0 and str(tc) in pressuredict.keys():
                        pres = pressuredict[str(tc)][1] #Get TC pressure after landfall
                        pd = 1010-pres
                        pdlist.append(pd)
                        damagelist.append(damage)
                
                pdlist = np.asarray(pdlist)
                damagelist = np.asarray(damagelist)

                damagelist = np.log(damagelist)
                pdlist = np.log(pdlist)
                
                plt.scatter(pdlist, damagelist,color='r',s=15, label = 'Pressure deficit (hPa)')
                slope, intercept, rvalue, pvalue, stderr = linregress(pdlist, damagelist) #Linear fit
                xrange = np.linspace(min(pdlist),max(pdlist),1000)
                plt.plot(xrange, slope*xrange + intercept, 'r--') #Plot linear fit
#                 samplesize = len(damagelist)

            if 'wind' in ivariables:
                damagedict = damage_GD
                if source == 0:
                    winddict = landfalldata_wind_GD_jtwc
                elif source == 1:
                    winddict = landfalldata_wind_GD_cma
                elif source == 2:
                    winddict = landfalldata_wind_GD_wmo
                    
                tcindexes = np.asarray(list(damagedict.keys()), dtype=int)
    
                damagelist = []
                windlist = []
                for tc in tcindexes:
                    if dvariable == 'damage':
                        damage = damagedict[str(tc)][1]
                    elif dvariable == 'deaths':
                        damage = damagedict[str(tc)][0]
                    if damage != 0 and str(tc) in winddict.keys():
                        if len(winddict[str(tc)]) == 1:
                            wind = winddict[str(tc)][0] #Get TC max wind after landfall
                        elif len(winddict[str(tc)]) == 2:
                            wind = winddict[str(tc)][1] #Get TC max wind after landfall
                        windlist.append(wind)
                        damagelist.append(damage)


                windlist = np.asarray(windlist)
                damagelist = np.asarray(damagelist)

                windlist = np.log(windlist)
                damagelist = np.log(damagelist)
                
                plt.scatter(windlist, damagelist,color='y',s=15, label = 'Maximum wind speed (kts)')
                slope, intercept, rvalue, pvalue, stderr = linregress(windlist, damagelist) #Linear fit
                xrange = np.linspace(min(windlist),max(windlist),1000)
                plt.plot(xrange, slope*xrange + intercept, 'y--') #Plot linear fit
#                 samplesize = len(damagelist)

            if 'rain' in ivariables:
                damagedict = damage_GD_after2014
                print(list(damagedict.keys()))
                tcindexes = np.asarray(list(damagedict.keys()), dtype=int)

                damagelist = []
                rainfall_total_list = []
                for tc in tcindexes:
                    print(tc)
                    if dvariable == 'damage':
                        damage = damagedict[str(tc)][1]
                    elif dvariable == 'deaths':
                        damage = damagedict[str(tc)][0]
                    if damage != 0:
                        if (source == 0 and str(tc) in landfalldata_coord_GD2014_jtwc.keys()) or (source == 1 and str(tc) in landfalldata_coord_GD2014_cma.keys()) :
                            damagelist.append(damage)
                            rainfall_list = rainfall_distribution(1, source, tc, only_on_land=False)[1]
                            total_rainfall = sum(rainfall_list) #Total rainfall within 500km of TC centre
                            rainfall_total_list.append(total_rainfall/1000)


                rainfall_total_list = np.asarray(rainfall_total_list)
                damagelist = np.asarray(damagelist)

                rainfall_total_list = np.log(rainfall_total_list)
                damagelist = np.log(damagelist)
                
                plt.scatter(rainfall_total_list, damagelist,color='g',s=15,label='Daily rainfall within 500km of TC centre (m)')
                slope, intercept, rvalue, pvalue, stderr = linregress(rainfall_total_list, damagelist) #Linear fit
                xrange = np.linspace(min(rainfall_total_list),max(rainfall_total_list),1000)
                plt.plot(xrange, slope*xrange + intercept, 'g--') #Plot linear fit
#                 samplesize = len(damagelist)

            if 'r18' in ivariables:
                damagedict = damage_GD
                if source == 0:
                    r18dict = r18dict_GD_jtwc
                elif source == 1:
                    r18dict = r18dict_GD_cma

                tcindexes = np.asarray(list(damagedict.keys()), dtype=int)

                damagelist = []
                r18list = []
                for tc in tcindexes:
                    if dvariable == 'damage':
                        damage = damagedict[str(tc)][1]
                    else:
                        damage = damagedict[str(tc)][0]
                    if str(tc) in r18dict.keys():
                        if r18dict[str(tc)] != []:
                            r18 = r18dict[str(tc)][1] #Get TC R18 after landfall
                            if type(r18) != str and damage != 0 and str(tc) in r18dict.keys():
                                r18list.append(r18)
                                damagelist.append(damage)

                r18list = np.asarray(r18list)
                damagelist = np.asarray(damagelist)

                r18list = np.log(r18list)
                damagelist = np.log(damagelist)

                plt.scatter(r18list, damagelist,color='b',s=15, label = 'R18 (km)')
                slope, intercept, rvalue, pvalue, stderr = linregress(r18list, damagelist) #Linear fit
                xrange = np.linspace(min(r18list),max(r18list),1000)
                plt.plot(xrange, slope*xrange + intercept, 'b--') #Plot linear fit
#                 samplesize = len(damagelist)
    
    plt.xlabel('log(variable)')
    plt.ylabel('log(' + str(dvariable) + ')')
    if dvariable == 'rain':
        plt.legend(fontsize='small')
    elif dvariable == 'damage':
        plt.legend(fontsize='x-small')
    elif dvariable == 'deaths':
        plt.legend(fontsize='xx-small')

In [ ]:
tic = time.time()
plot_variables_sameplot(source=0,ivariables=['pd','wind','rain','r18'],dvariable = 'damage')
#plot_variables_sameplot(source=0,ivariables=['pd','wind','rain','r18'],dvariable = 'deaths')
#plot_variables_sameplot(source=1,ivariables=['pd','r18'],dvariable = 'rain')
toc = time.time()
print(toc-tic)